Generating plots that explore the disordered LOPC model. This notebook is for generating publication-ready plots, which will be automatically exported to the LaTeX directory. To follow the process of exploring the data, look at the other notebooks.

In [ ]:
# computation
import lumapi
import numpy as np
import warnings
import xarray as xr
# import xyzpy as xyz
# from xyzpy.gen.combo_runner import multi_concat
from multilayer_simulator.lumerical_classes import LumericalOscillator, format_stackrt, format_stackfield
from multilayer_simulator.helpers.mixins import convert_wavelength_and_frequency
import dask
from functools import partial
from pathlib import Path
import sys
from tqdm import tqdm
# plotting
import hvplot.xarray
import holoviews as hv
from holoviews import dim, opts
import colorcet
import panel as pn
import panel.widgets as pnw
from bokeh.io import export_png, export_svg
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from scipy.signal import find_peaks
from bokeh.models import PrintfTickFormatter

In [ ]:
hv.extension("bokeh", inline=False, case_sensitive_completion=True)  # use matplotlib because rendering bokeh to svg is broken
pn.config.throttled = True  # don't update interactive plots until mouse is unclicked

# default_color_cycle = hv.Cycle("Colorblind")  # Ruth doesn't like the inclusion of yellow, which is fair enough
default_color_cycle = hv.Cycle(colorcet.glasbey_dark)
default_dash_cycle = hv.Cycle(["solid", "dashed", "dashdot", "dotted", "dotdash"])
universal_opts = dict(fontscale=2, title="")
matplotlib_opts = dict(fig_inches=5, aspect=2, fig_latex=True)
bokeh_opts = dict(width=700, height=300)
opts.defaults(opts.Curve(**universal_opts|bokeh_opts, color=default_color_cycle, line_width=1.5),
              opts.Scatter(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Slope(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Area(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Overlay(**universal_opts|bokeh_opts),
              opts.Layout(**universal_opts|bokeh_opts),
              opts.GridSpace(**universal_opts|bokeh_opts),
              )

xarray_engine='h5netcdf'

In [ ]:
root = Path.cwd().parent.parent.parent  # depth of parents depends on if this is running in JupyterLab or Notebook

In [ ]:
code_path = root / r"research"

In [ ]:
data_path = code_path / r"notebooks/data"

In [ ]:
archive_path = root / r"thesis/LaTeX/chapters/chapter_2"

In [ ]:
fig_path = archive_path / "fig_chapter_2"

In [ ]:
if not code_path in sys.path:
    sys.path.append(str(code_path))
from LOPC import LOPC
from LOPC.helpers import (
    assign_derived_attrs,
    restack,
    enhancement_factor,
    # combo_length,
    # estimate_combo_run_time,
    linewidth_calculator,
    lopc_data,
    spectrum,
    normalise_over_dim,
    integrate_da,
    sel_or_integrate,
    find_optimum_coords,
    plot_secondary,
    pre_process_for_plots,
    vlines,
    coordinate_string,
    plot_da,
    plot_var,
    plot_optimum_over_dim,
    plot_field,
    visualise_multilayer,
    complex_elements,
    indexer_from_dataset,
    fix_bin_labels,
    mean_and_std,
    assign_high_from_mean_and_std,
    assign_low_from_mean_and_std,
    assign_high_and_low,
    max_min_pos,
)

Turn on auto-archiving of cells and Holoviews outputs. See the user guide [here](https://holoviews.org/user_guide/Exporting_and_Archiving.html).

Might need to install `ipympl`.

EDIT: This does not work but I'm leaving this here so a future researcher can avoid the rabbithole I fell down.

In [ ]:
# # This is the idiomatic way to record all generated figures with holoviews
# # This does NOT work in JupyterLab: see https://github.com/holoviz/holoviews/issues/3570
# # This also does not work in Jupyter Notebook
# # It's just utterly broken

# hv.archive.auto(root=str(archive_path), export_name="fig_chapter_2") 

Due to a bug that I still do not fully understand, `hv.save` raises
```
RuntimeError: Neither firefox and geckodriver nor a variant of chromium browser and chromedriver are available on system PATH. You can install the former with 'conda install -c conda-forge firefox geckodriver'.
```
The below code is part of a workaround. (The second part is to pass `web_driver` to `export_png(hv.render(fig), ..., webdriver=web_driver)` rather than `hv.save(fig)`, which is supposed to just work.)

EDIT: `hv.save(fig)` seems to be working for now.

In [ ]:
# options = Options()
# options.add_argument('-headless')
# web_driver = Firefox(
#     options=options,
# )

# Load/define datasets

## Basic datasets

In [ ]:
# plt = 50, 100
uncorrelated_ds_01_04_short = xr.open_mfdataset(
    [data_path / r"run_103/LOPC_01.nc",
     data_path / r"run_103/LOPC_02.nc",
     data_path / r"run_103/LOPC_03.nc",
     data_path / r"run_103/LOPC_04.nc",
    ],
    engine=xarray_engine,
    lock=False,
)

# plt = 150, 200, 250
uncorrelated_ds_01_04_long = xr.open_mfdataset(
    data_path / r"run_101/LOPC.nc",
    engine=xarray_engine,
    lock=False,
)

uncorrelated_ds_01_04 = xr.concat(
    [uncorrelated_ds_01_04_short, uncorrelated_ds_01_04_long],
    dim="passive_layer_thickness",
)
uncorrelated_ds_01_04 = assign_derived_attrs(
    uncorrelated_ds_01_04,
    per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
)

In [ ]:
# plt = 50, 100
uncorrelated_ds_05_short = xr.open_mfdataset(
    (data_path / "run_103").glob("LOPC_05_*"),
    chunks={"run": 100},
    concat_dim="run",
    combine="nested",
    engine=xarray_engine,
    lock=False,
)

# plt = 150, 200, 250
uncorrelated_ds_05_long = xr.open_mfdataset(
    (data_path / "run_102").glob("*"),
    chunks={"run": 100},
    concat_dim="run",
    combine="nested",
    engine=xarray_engine,
    lock=False,
)

uncorrelated_ds_05 = xr.concat(
    [uncorrelated_ds_05_short, uncorrelated_ds_05_long],
    dim="passive_layer_thickness",
)
# assign each run a unique coordinate
uncorrelated_ds_05 = uncorrelated_ds_05.assign_coords(
    run=range(len(uncorrelated_ds_05.run))
)
uncorrelated_ds_05 = assign_derived_attrs(uncorrelated_ds_05, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"])

In [ ]:
uncorrelated_aggregated_ds = xr.open_mfdataset(
    data_path / r"run_199/LOPC_aggregated.nc",
    engine=xarray_engine,
    lock=False,
)
uncorrelated_aggregated_ds = assign_derived_attrs(uncorrelated_aggregated_ds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"])

In [ ]:
correlated_run_number_long = 201
long_run_path = data_path / f"run_{correlated_run_number_long}"
correlated_run_number_short = 203
short_run_path = data_path / f"run_{correlated_run_number_short}"

# plt = 50, 100
correlated_ds_01_04_short = xr.open_mfdataset(
    [
        short_run_path / "LOPC_01.nc",
        short_run_path / "LOPC_02.nc",
        short_run_path / "LOPC_03.nc",
        short_run_path / "LOPC_04.nc",
    ],
    engine=xarray_engine,
    lock=False,
)

# plt = 150, 200, 250
correlated_ds_01_04_long = xr.open_mfdataset(
    [
        long_run_path / "LOPC_01.nc",
        long_run_path / "LOPC_02.nc",
        long_run_path / "LOPC_03.nc",
        long_run_path / "LOPC_04.nc",
    ],
    engine=xarray_engine,
    lock=False,
)

correlated_ds_01_04 = xr.concat(
    [correlated_ds_01_04_short, correlated_ds_01_04_long],
    dim="passive_layer_thickness",
)
correlated_ds_01_04 = assign_derived_attrs(
    correlated_ds_01_04,
    per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
)

In [ ]:
# plt = 50, 100
correlated_ds_05_short = xr.open_mfdataset(
    short_run_path.glob("LOPC_05_*.nc"),
    chunks={"run": 100},
    concat_dim="run",
    combine="nested",
    engine=xarray_engine,
    lock=False,
)

# plt = 150, 200, 250
correlated_ds_05_long = xr.open_mfdataset(
    long_run_path.glob("LOPC_05_*.nc"),
    chunks={"run": 100},
    concat_dim="run",
    combine="nested",
    engine=xarray_engine,
    lock=False,
)

correlated_ds_05 = xr.concat(
    [correlated_ds_05_short, correlated_ds_05_long],
    dim="passive_layer_thickness",
)
# assign each run a unique coordinate
correlated_ds_05 = correlated_ds_05.assign_coords(run=range(len(correlated_ds_05.run)))
correlated_ds_05 = assign_derived_attrs(
    correlated_ds_05, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"]
)

In [ ]:
# plt = 50, 100
correlated_ds_10_short = xr.open_mfdataset(
    short_run_path.glob("LOPC_10_*.nc"),
    chunks={"run": 100},
    concat_dim="run",
    combine="nested",
    engine=xarray_engine,
    lock=False,
)

# plt = 150, 200, 250
correlated_ds_10_long = xr.open_mfdataset(
    long_run_path.glob("LOPC_10_*.nc"),
    chunks={"run": 100},
    concat_dim="run",
    combine="nested",
    engine=xarray_engine,
    lock=False,
)

correlated_ds_10 = xr.concat(
    [correlated_ds_10_short, correlated_ds_10_long],
    dim="passive_layer_thickness",
)
# assign each run a unique coordinate
correlated_ds_10 = correlated_ds_10.assign_coords(run=range(len(correlated_ds_10.run)))
correlated_ds_10 = assign_derived_attrs(
    correlated_ds_10, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"]
)

In [ ]:
# num_periods = 5, 10, 15, 20, 25
correlated_aggregated_ds_short = xr.open_mfdataset(
    data_path.glob(r"run_211/LOPC_0*.nc"),
    engine=xarray_engine,
    lock=False,
)

# num_periods = 30
correlated_aggregated_ds_long = xr.open_mfdataset(
    data_path.glob(r"run_211/LOPC_N30_*.nc"),
    engine=xarray_engine,
    lock=False,
)

correlated_aggregated_ds = xr.concat(
    [correlated_aggregated_ds_short, correlated_aggregated_ds_long], dim="num_periods"
)

correlated_aggregated_ds = assign_derived_attrs(
    correlated_aggregated_ds,
    per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
)

In [ ]:
# chunks for plotting or integrating over angle
undisordered_run_number = 2
pds_chunks = {
    "frequency": 256,
    "excitonic_layer_thickness": 16,
    "passive_layer_thickness": 32,
    "theta": 16,
    "num_periods": 1,
}

periodic_ds = xr.open_mfdataset(
    data_path / f"run_{undisordered_run_number}/LOPC.nc",
    engine=xarray_engine,
    lock=False,
    chunks=pds_chunks,
)
periodic_ds = assign_derived_attrs(
    periodic_ds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"]
)

## Synthesised datasets

In [ ]:
disorder_dim = "disorder_correlation"

singles_ds_01_04 = xr.concat(
    [
        correlated_ds_01_04.expand_dims({disorder_dim: ["correlated"]}),
        uncorrelated_ds_01_04.expand_dims({disorder_dim: ["uncorrelated"]}),
    ],
    dim=disorder_dim,
    combine_attrs="drop_conflicts",
)

singles_ds_05 = xr.concat(
    [
        correlated_ds_05.expand_dims({disorder_dim: ["correlated"]}),
        uncorrelated_ds_05.expand_dims({disorder_dim: ["uncorrelated"]}),
    ],
    dim=disorder_dim,
    combine_attrs="drop_conflicts",
)

singles_ds_10 = correlated_ds_10.expand_dims({disorder_dim: ["correlated"]}).reindex(
    {disorder_dim: ["correlated", "uncorrelated"]}
)

In [ ]:
indexer_from_singles_ds = partial(indexer_from_dataset, drop=["run", "layer_number", "delta", disorder_dim])

In [ ]:
# loading the data into memory might save time later
singles_pds = periodic_ds.sel(indexer_from_singles_ds(singles_ds_01_04)).expand_dims(
    {
        "run": [0],
        disorder_dim: singles_ds_01_04[disorder_dim].values,
        "delta": [0.0],
    }
)  # .load()

A `singles_ds` is defined below for convenience. There is a surprising difference in the performance of different ways to concatenate the datasets. Arranging the `disorder_dim` in alphabetical order to avoid resorting makes a 2x difference. Using the reindexed `singles_ds_10` versus letting `align` do the work also makes a difference.

In general the use of this dataset should be avoided because accessing the data for delta<0.4 is 10x slower. This is due to having to load a bunch of nans because of the run number disparity. This dataset is okay for exploring the space conveniently but shouldn't be used often.

In [ ]:
%%time
with dask.config.set(**{"array.slicing.split_large_chunks": False}):
    singles_ds = xr.concat(
        xr.align(
            singles_pds,
            singles_ds_01_04,
            singles_ds_05,
            correlated_ds_10.expand_dims({disorder_dim: ["correlated"]}),
            join="outer",
            exclude="delta",
        ),
        dim="delta",
        combine_attrs="drop_conflicts",
    )

In [ ]:
# %%time
# singles_ds.sel(delta=0.5, frequency=0, theta=0, method="nearest").Rs.load()

```
CPU times: total: 3.5 s
Wall time: 3.45 s
```

In [ ]:
# %%time
# # slightly less efficient overall
# with dask.config.set(**{"array.slicing.split_large_chunks": False}):
#     singles_ds = xr.concat(
#         xr.align(
#             singles_ds_01_04,
#             singles_ds_05,
#             singles_ds_10,
#             join="outer",
#             exclude="delta",
#         ),
#         dim="delta",
#         combine_attrs="drop_conflicts",
#     )

```
CPU times: total: 1.55 s
Wall time: 1.61 s
```

In [ ]:
# %%time
# singles_ds.sel(delta=0.5, frequency=0, theta=0, method="nearest").Rs.load()

```
CPU times: total: 5.03 s
Wall time: 6.57 s
```

In [ ]:
# %%time
# singles_ds_10.sel(delta=1, frequency=0, theta=0, method="nearest").Rs.load()

```
CPU times: total: 19 s
Wall time: 5.58 s
```

In [ ]:
# %%time
# singles_ds_01_04.sel(delta=0.1, frequency=0, theta=0, method="nearest").Rs.load()

```
CPU times: total: 1.08 s
Wall time: 1.58 s
```

In [ ]:
# %%time
# singles_ds_10.sel(disorder_correlation="correlated").sel(frequency=0, theta=0, method="nearest").Rs.load()

```
CPU times: total: 21.9 s
Wall time: 6.8 s
```

In [ ]:
# %%time
# correlated_ds_10.sel(frequency=0, theta=0, method="nearest").Rs.load()

```
CPU times: total: 1.02 s
Wall time: 2.44 s
```

In [ ]:
# stats_ds = xr.concat(
#     [
#         # could be done more efficiently with renaming but using mean_and_std guarantees I use the same code paths
#         mean_and_std(singles_pds),
#         mean_and_std(singles_ds_01_04.drop_dims("layer_number")),
#         mean_and_std(singles_ds_05.drop_dims("layer_number")),
#         # delay reindex to AFTER mean_and_std
#         mean_and_std(
#             correlated_ds_10.expand_dims({disorder_dim: ["correlated"]})
#         # explicit reindex saves time over automatic inference
#         ).reindex({disorder_dim: ["correlated", "uncorrelated"]}),
#         # mean_and_std(singles_ds_10.drop_dims("layer_number")),  # slower version of above
#     ],
#     dim="delta",
#     combine_attrs="drop_conflicts",
# )

To speed up later computations, we store the intermediate `stats_ds` dataset. We strip out the per-oscillator metrics, because those can be recovered from the bare RTA using the `assign_per_oscillator` helper function, which just does a simple divison. Dividing the mean by a given value divides the standard deviation by the same value.

We could potentially also remove the unpolarised RTA, if the standard deviation of `R` is just the mean of the standard deviations of `Rs` and `Rp`. Preliminary investigations seem to indicate this, but I'm not confident it's universally true, because `Rs` and `Rp` are not obviously 'independent' from each other.

In [ ]:
# stats_var_keys = list(stats_ds.data_vars.keys())

# stats_var_keys_no_per_oscillator = [x for x in stats_var_keys if "per_oscillator" not in x]

# stats_ds.sel(disorder_correlation="correlated")[stats_var_keys_no_per_oscillator].to_netcdf(data_path/"run_299/LOPC_stats_correlated.nc", engine=xarray_engine)

# (
#     stats_ds
#     .sel(disorder_correlation="uncorrelated")
#     .sel(delta=slice(0, 0.5))[
#         stats_var_keys_no_per_oscillator
#     ]
# ).to_netcdf(data_path / "run_299/LOPC_stats_uncorrelated.nc", engine=xarray_engine)

In [ ]:
stats_ds = xr.open_mfdataset(
    [data_path / "run_299/LOPC_stats_correlated.nc",
     data_path / "run_299/LOPC_stats_uncorrelated.nc",],
    concat_dim=disorder_dim,
    combine="nested",
    engine=xarray_engine,
    lock=False,
    chunks=None,
)

In [ ]:
LOPC_var_keys = list(singles_pds.data_vars.keys())

In [ ]:
basic_RTA_var_keys = [x for x in LOPC_var_keys if "per_oscillator" not in x]

In [ ]:
stats_ds = assign_high_and_low(stats_ds, var_key=basic_RTA_var_keys)

In [ ]:
# # you might naively think that this should do the same as the above cell, but the
# # nested align/concat makes accessing the data MUCH slower
# stats_ds = xr.concat(
#     [
#         # could be done more efficiently with renaming but using mean_and_std guarantees I use the same code paths
#         mean_and_std(singles_pds),
#         mean_and_std(singles_ds.drop_dims("layer_number")),
#     ],
#     dim="delta",
#     combine_attrs="drop_conflicts",
# )

In [ ]:
aggregated_ds = xr.concat(
    [
        correlated_aggregated_ds.expand_dims({disorder_dim: ["correlated"]}),
        uncorrelated_aggregated_ds.expand_dims({disorder_dim: ["uncorrelated"]}),
    ],
    dim=disorder_dim,
    combine_attrs="drop_conflicts",
)

In [ ]:
indexer_from_aggregated_ds = partial(indexer_from_dataset, drop=["delta", disorder_dim])

In [ ]:
# loading the data into memory might save time later
aggregated_pds = periodic_ds.sel(indexer_from_aggregated_ds(aggregated_ds)).expand_dims(
    {
        "run": 1,
        disorder_dim: aggregated_ds[disorder_dim].values,
        "delta": [0.0],
    }
)  # .load()

In [ ]:
aggregated_ds = xr.concat(
    [aggregated_pds, aggregated_ds],
    dim="delta",
    combine_attrs="drop_conflicts",
)

In [ ]:
restack_plt_to_period = partial(
    restack,
    start_idxs=["passive_layer_thickness", "excitonic_layer_thickness"],
    end_idxs=["period", "excitonic_layer_thickness"],
)

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     restacked_pds = restack_plt_to_period(periodic_ds)  # the delta=0 case
#     restacked_spds = restack_plt_to_period(singles_pds)  # periodic_ds indexed by singles_ds, below

#     restacked_uads = restack_plt_to_period(uncorrelated_aggregated_ds) # uncorrelated delta<0.15, higher precision in elt and period
#     restacked_cads = restack_plt_to_period(correlated_aggregated_ds) # correlated delta<0.15, higher precision in elt and period
#     restacked_ads = restack_plt_to_period(aggregated_ds)

#     restacked_sds_01_04 = restack_plt_to_period(singles_ds_01_04)  # individual run data delta<0.4, coarse
#     restacked_sds_05 = restack_plt_to_period(singles_ds_05)  # individual run data delta=0.5, coarse
#     # restacked_sds_10 = restack_plt_to_period(singles_ds_10)  # individual run data delta=1, coarse, inefficient - never use this!
#     restacked_cds_10 = restack_plt_to_period(correlated_ds_10)  # much faster version of above without unnecessary reindexing
#     restacked_sds = restack_plt_to_period(singles_ds)  # merge of the above - convenient but inefficient due to ragged structure
#     restacked_stats_ds = restack_plt_to_period(stats_ds)  # calculated mean and standard deviation of individual run data

Just some plots to test everything is working:

In [ ]:
# restacked_stats_ds.Rs_mean.sel(
#     theta=0, num_periods=20, period=300, delta=[0.1]
# ).squeeze().hvplot.line(x="frequency", by="disorder_correlation").opts(
#     legend_position="top_right"
# )

In [ ]:
# restacked_stats_ds.Rs_mean.sel(theta=0, num_periods=20, period=300).sel(
#     delta=slice(0.1, None)
# ).squeeze().hvplot.line(x="frequency", by="disorder_correlation", row="delta").opts(
#     opts.Curve(xticks=1, show_legend=False), opts.GridSpace(shared_xaxis=True)
# )

In [ ]:
# var = pnw.Select(options = list(restacked_ads.data_vars.keys()))

# xs = {
#     "period": pnw.DiscreteSlider,
#     "excitonic_layer_thickness": pnw.DiscreteSlider,
#     "num_periods": pnw.DiscreteSlider,
#     "delta": pnw.DiscreteSlider,
#     "theta": pnw.DiscreteSlider,
# }

# restacked_ads.interactive(loc="left")[var].sel(**xs).squeeze(drop=True).hvplot(
#     x="frequency",
#     by="disorder_correlation",
# )

### Load reference slab dataset

Note for future maintainers: the following code specifies a larger set of thicknesses that is actually covered by the disorder simulations, but that's fine because the latter is contained within the former, and arithmetic using both datasets will only use the intersection.

In [ ]:
# useful variables
total_excitonic_thicknesses = np.unique(periodic_ds.total_excitonic_thickness)
total_passive_thicknesses = np.unique(periodic_ds.total_passive_thickness)
total_thicknesses = np.unique(periodic_ds.total_thickness)

In [ ]:
ref = xr.open_mfdataset(
    data_path / f"run_{undisordered_run_number}/ref.nc",
    engine=xarray_engine,
    lock=False,
    # chunks={'theta': 16},
)

Note: `period=False` is an important option because otherwise it ends up a coordinate of `total_excitonic_thickness` and causes a conflict after binary operations with `ds`.

In [ ]:
ref = assign_derived_attrs(ref, period=False, total_excitonic_thickness=False, total_passive_thickness=False, total_thickness=False)
# ref

In [ ]:
# compressed reference slab without passive layer
crs_1 = (
    ref.sel(
        remove_last_layer=1,
        passive_layer_thickness=0,
        excitonic_layer_thickness=total_excitonic_thicknesses,
        drop=True,
    )
    .squeeze(drop=True)
    .rename(excitonic_layer_thickness="total_excitonic_thickness")
    # .load()
)

# # compressed reference slab with the passive layer at the end
# crs_0 = (
#     ref.sel(
#         remove_last_layer=0,
#         passive_layer_thickness=total_passive_thicknesses,
#         excitonic_layer_thickness=total_excitonic_thicknesses,
#         drop=True,
#     )
#     .squeeze(drop=True)
#     .rename(
#         excitonic_layer_thickness="total_excitonic_thickness",
#         passive_layer_thickness="total_passive_thickness",
#     )
# )

### Calculate derived variables

In [ ]:
# # dataset memo
# periodic_ds  # the delta=0 case
# singles_pds  # periodic_ds indexed by singles_ds, below

# uncorrelated_aggregated_ds # uncorrelated delta<0.15, higher precision in elt and period
# correlated_aggregated_ds # correlated delta<0.15, higher precision in elt and period
# aggregated_ds # combination of above

# singles_ds_01_04  # individual run data delta<0.4, coarse
# singles_ds_05  # individual run data delta=0.5, coarse
# singles_ds_10  # individual run data delta=1, coarse, inefficient - never use this!
# correlated_ds_10  # much faster version of above without unnecessary reindexing
# singles_ds  # merge of the above - convenient but inefficient due to ragged structure
# stats_ds  # calculated mean and standard deviation of individual run data

# # + restacked versions of all the above

#### Normalise by CRS

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     norm_1 = enhancement_factor(ds, ref=crs_1, common_dim="total_excitonic_thickness", method="groupby")

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     restacked_norm_1 = restack_plt_to_period(norm_1)

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     norm_1_05 = enhancement_factor(ds_05, ref=crs_1, common_dim="total_excitonic_thickness", method="groupby")

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     restacked_norm_1_05 = restack_plt_to_period(norm_1_05)

#### Difference with CRS

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     gb_tet = ds.groupby('total_excitonic_thickness')

In [ ]:
# # 'biology' absorptance enhancement factor: normalise by reference slab type 1: compressed reference slab w/o passive layer
# diff_1 = gb_tet - crs_1

In [ ]:
# diff_1 = assign_derived_attrs(
#     dataset=diff_1,
#     unpolarised=True,
#     absorption=False,
#     period=False,
#     total_excitonic_thickness=False,
#     total_passive_thickness=False,
#     total_thickness=False,
#     N_tot=False,
#     per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
# )

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     restacked_diff_1 = restack_plt_to_period(diff_1)

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     gb_tet_05 = ds_05.groupby('total_excitonic_thickness')

In [ ]:
# # 'biology' absorptance enhancement factor: normalise by reference slab type 1: compressed reference slab w/o passive layer
# diff_1_05 = gb_tet_05 - crs_1

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     restacked_diff_1_05 = restack_plt_to_period(diff_1_05)

# Prepare for Plots

## Pre-processing

In [ ]:
blue = hv.Cycle.default_cycles['default_colors'][0]
red = hv.Cycle.default_cycles['default_colors'][1]
yellow = hv.Cycle.default_cycles['default_colors'][2]
green = hv.Cycle.default_cycles['default_colors'][3]

In [ ]:
RTA_cycle = hv.Cycle([blue, yellow, red])

In [ ]:
# change default colour cycle
hv.Cycle.default_cycles["default_colors"] = colorcet.glasbey_dark

In [ ]:
wavelengths_in_nanometres = np.linspace(480, 880, 256)
wavelengths = wavelengths_in_nanometres * 1e-9
frequencies = convert_wavelength_and_frequency(wavelengths)
angles = np.linspace(0, 86, 64)

In [ ]:
default_oscillator_params = {
    "N": 1e26,
    "permittivity": 2.2,
    "lorentz_resonance_wavelength": 680,
    "lorentz_linewidth": 7.5e13,
}

In [ ]:
unpolarised_RTA = ["R", "T", "A"]
s_polarised_RTA = ["Rs", "Ts", "As"]
p_polarised_RTA = ["Rp", "Tp", "Ap"]
reflectance = ["Rs", "Rp", "R"]
transmittance = ["Ts", "Tp", "T"]
absorptance = ["As", "Ap", "A"]
per_oscillator_RTA = ["R_per_oscillator", "T_per_oscillator", "A_per_oscillator"]

In [ ]:
pre_process_for_plots = partial(pre_process_for_plots, strict=False)

In [ ]:
# WARNING: all these datasets will be fundamentally changed after this cell, to the extent that it can't be run twice
# For consistency, keep important calculations in the preceding section!
periodic_ds = pre_process_for_plots(periodic_ds)  # the delta=0 case
singles_pds = pre_process_for_plots(singles_pds)  # periodic_ds indexed by singles_ds, below

uncorrelated_aggregated_ds = pre_process_for_plots(uncorrelated_aggregated_ds) # uncorrelated delta<0.15, higher precision in elt and period
correlated_aggregated_ds = pre_process_for_plots(correlated_aggregated_ds) # correlated delta<0.15, higher precision in elt and period
aggregated_ds = pre_process_for_plots(aggregated_ds) # combination of above

singles_ds_01_04 = pre_process_for_plots(singles_ds_01_04)  # individual run data delta<0.4, coarse
singles_ds_05 = pre_process_for_plots(singles_ds_05)  # individual run data delta=0.5, coarse
# singles_ds_10 = pre_process_for_plots(singles_ds_10)  # individual run data delta=1, coarse, inefficient - never use this!
correlated_ds_10 = pre_process_for_plots(correlated_ds_10)  # much faster version of above without unnecessary reindexing
singles_ds = pre_process_for_plots(singles_ds)  # merge of the above - convenient but inefficient due to ragged structure
stats_ds = pre_process_for_plots(stats_ds)  # calculated mean and standard deviation of individual run data

# + restacked versions of all the above
ref = pre_process_for_plots(ref)
crs_1 = pre_process_for_plots(crs_1)
# norm_1 = pre_process_for_plots(norm_1)
# restacked_norm_1 = pre_process_for_plots(restacked_norm_1)
# diff_1 = pre_process_for_plots(diff_1)
# restacked_diff_1 = pre_process_for_plots(restacked_diff_1)

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    restacked_pds = restack_plt_to_period(periodic_ds)  # the delta=0 case
    restacked_spds = restack_plt_to_period(singles_pds)  # periodic_ds indexed by singles_ds, below

    restacked_uads = restack_plt_to_period(uncorrelated_aggregated_ds) # uncorrelated delta<0.15, higher precision in elt and period
    restacked_cads = restack_plt_to_period(correlated_aggregated_ds) # correlated delta<0.15, higher precision in elt and period
    restacked_ads = restack_plt_to_period(aggregated_ds)

    restacked_sds_01_04 = restack_plt_to_period(singles_ds_01_04)  # individual run data delta<0.4, coarse
    restacked_sds_05 = restack_plt_to_period(singles_ds_05)  # individual run data delta=0.5, coarse
    # restacked_sds_10 = restack_plt_to_period(singles_ds_10)  # individual run data delta=1, coarse, inefficient - never use this!
    restacked_cds_10 = restack_plt_to_period(correlated_ds_10)  # much faster version of above without unnecessary reindexing
    restacked_sds = restack_plt_to_period(singles_ds)  # merge of the above - convenient but inefficient due to ragged structure
    restacked_stats_ds = restack_plt_to_period(stats_ds)  # calculated mean and standard deviation of individual run data

In [ ]:
og_names = list(crs_1.data_vars.keys())

In [ ]:
mean_names = [f"{var}_mean" for var in og_names]

In [ ]:
std_names = [f"{var}_std" for var in og_names]

In [ ]:
mean_rename = {old_var: new_var for new_var, old_var in zip(mean_names, og_names)}

In [ ]:
std_assignation = {new_var: (lambda x, var=old_var: x[var]) for new_var, old_var in zip(std_names, og_names)}

In [ ]:
# manipulated to give the correct enhancement factors when used in ratio with stats_ds
crs_1_stats_norm = crs_1.assign(std_assignation).rename(mean_rename)

In [ ]:
std_assignation = {new_var: (lambda x: xr.zeros_like(x[old_var])) for new_var, old_var in zip(std_names, og_names)}

In [ ]:
# manipulated to give the correct difference factors when subtracted from stats_ds
crs_1_stats_diff = crs_1.assign(std_assignation).rename(mean_rename)

In [ ]:
norm_1 = partial(enhancement_factor, ref=crs_1, common_dim="total_excitonic_thickness", method="groupby")

In [ ]:
def restacked_norm_1(ds):
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        return restack_plt_to_period(norm_1(ds))

In [ ]:
# norm_1_stats = partial(enhancement_factor, ref=crs_1_stats_norm, common_dim="total_excitonic_thickness", method="groupby")

In [ ]:
def norm_1_stats(ds, ref=crs_1_stats_norm, common_dim="total_excitonic_thickness", method="groupby", assign_high=True, assign_low=True, high_low_var_key=basic_RTA_var_keys):
    normed_stats = enhancement_factor(ds, ref=ref, common_dim=common_dim, method=method)
    if assign_high:
        normed_stats = assign_high_from_mean_and_std(normed_stats, var_key=high_low_var_key)
    if assign_low:
        normed_stats = assign_low_from_mean_and_std(normed_stats, var_key=high_low_var_key)
        
    return normed_stats

In [ ]:
def restacked_norm_1_stats(ds):
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        return restack_plt_to_period(norm_1_stats(ds))

In [ ]:
def diff_1(ds):
    diff_1 = ds.groupby("total_excitonic_thickness") - crs_1

    diff_1 = assign_derived_attrs(
        dataset=diff_1,
        unpolarised=True,
        absorption=False,
        period=False,
        total_excitonic_thickness=False,
        total_passive_thickness=False,
        total_thickness=False,
        N_tot=False,
        per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
    )
    
    return diff_1

In [ ]:
def restacked_diff_1(ds):
    return restack_plt_to_period(diff_1(ds))

In [ ]:
def diff_1_stats(ds):
    diff_1 = ds.groupby("total_excitonic_thickness") - crs_1_stats_diff

    diff_1 = assign_derived_attrs(
        dataset=diff_1,
        unpolarised=True,
        absorption=False,
        period=False,
        total_excitonic_thickness=False,
        total_passive_thickness=False,
        total_thickness=False,
        N_tot=False,
        per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
    )
    
    return diff_1

In [ ]:
def restacked_diff_1_stats(ds):
    return restack_plt_to_period(diff_1_stats(ds))

In [ ]:
for n, c in stats_ds.coords.items():
    print(n, c.attrs)

In [ ]:
for n, c in restacked_norm_1(stats_ds).coords.items():
    print(n, c.attrs)

In [ ]:
period_dim = hv.Dimension("period", label="Λ", unit="nm")
wavelength_dim = hv.Dimension("wavelength", label="λ", unit="nm")
real_index_dim = hv.Dimension("n")
imag_index_dim = hv.Dimension("k", label="ϰ")
delta_dim = hv.Dimension("delta", label="Δ")
elt_dim = hv.Dimension("excitonic_layer_thickness", label="d_e", unit="nm")

### Useful lines

#### Function to do linewidth maths

In [ ]:
def linewidth_calculator_factory(centre, linewidth):
    return partial(linewidth_calculator, centre=centre, linewidth=linewidth)

#### Lorentz lines

I want some sort of metric for 'near the resonance' and 'far from the resonance'. The natural unit of distance in this instance is the linewidth. The linewidth is given in rad/s so there need to be some conversions to get the equivalent lines in the plots by wavelength, but they are roughly symmetrical around the peak wavelength.

Based on the plots of the refractive index below, I think I will consider 'near' to be 'within two linewidths', and 'far' to be 'at least four linewidths away'.

In [ ]:
# resonance_line = hv.VLine(680, label='LO resonance wavelength').opts(line_dash='dotted')

# Convert from rad/s to Hz
lorentz_linewidth_frequency = default_oscillator_params["lorentz_linewidth"] / (2*np.pi)

In [ ]:
lorentz_line_frequency = linewidth_calculator_factory(convert_wavelength_and_frequency(680e-9), lorentz_linewidth_frequency)

def lorentz_line_wavelength(x=None):
    x = -x if x is not None else x
    return convert_wavelength_and_frequency(lorentz_line_frequency(x))

In [ ]:
def lorentz_vlines(x=0, scale=1, mode='wavelength', **kwargs):
    if mode == 'wavelength':
        line_func = lorentz_line_wavelength
    elif mode == 'frequency':
        line_func = lorentz_line_frequency
    else:
        raise TypeError(f"mode should be 'wavelength' or 'frequency', not {mode}")
        
    match x:
        case [*xs]:
            line_pos = [line_func(x)/scale for x in xs]
        case x:
            line_pos = line_func(x)/scale
            
    return vlines(line_pos, **kwargs)

### Useful functions

#### Select a wavelength or wavelength range based on the distance from the resonance in linewidths.

In [ ]:
def select_lorentz_line(da, lorentz_line=0, window_radius=0):
    if window_radius == 0:
        wavelength = lorentz_line_wavelength(lorentz_line) * 1e9
        wavelength_sel_method = "nearest"
    else:
        wavelength = slice(
            lorentz_line_wavelength(lorentz_line - window_radius) * 1e9,
            lorentz_line_wavelength(lorentz_line + window_radius) * 1e9,
        )
        wavelength_sel_method = None
    da = da.sel(wavelength=wavelength, method=wavelength_sel_method)
    
    return da

In [ ]:
def cross_section(
    dataset,
    variable=None,
    wavelength=None,
    lorentz_line=0,
    window_radius=0,
    theta=0,
    integrate_angle=None,
    normalisation=1,
):
    """Select and integrate"""
    da = dataset[variable] if variable is not None else dataset
    integration_dims = []

    if not integrate_angle:
        if theta is not None:
            da = da.sel(theta=theta, method="nearest")
    else:  # integrate_angle must be a float, so that (theta, integrate_angle) is a slice syntax
        da = da.sel(theta=slice(theta, integrate_angle))
        integration_dims.append("theta")

    match wavelength:
        case None:
            da = select_lorentz_line(
                da, lorentz_line=lorentz_line, window_radius=window_radius
            )

            if window_radius != 0:
                integration_dims.append("wavelength")
        case (start, stop, *_):
            da = da.sel(wavelength=slice(*wavelength))
            integration_dims.append("wavelength")
        case _:
            da = sel_slice_or_nearest(da, "wavelength", wavelength)

    if integration_dims:  # if the list isn't empty
        da = integrate_da(
            da, integration_dims, weighting=1, normalisation=normalisation
        )

    return da

#### Calculate integrated enhancement factor

In [ ]:
def integrated_enhancement_factor(ds, ref, common_dim, method="groupby", assign_high=False, assign_low=False, high_low_var_key=basic_RTA_var_keys, **kwargs):
    if "normalisation" not in kwargs:  # unless normalisation is specified for some reason...
        kwargs |= {"normalisation": None}  # don't do it, because it happens automatically from taking the ratio
    ds = cross_section(ds, **kwargs)
    ref = cross_section(ref, **kwargs)
    ef = enhancement_factor(ds=ds, ref=ref, common_dim=common_dim, method=method)
    
    # this bit is only relevant if I'm working with the stats dataset
    if assign_high:
        ef = assign_high_from_mean_and_std(ef, var_key=high_low_var_key)
    if assign_low:
        ef = assign_low_from_mean_and_std(ef, var_key=high_low_var_key)
    
    return ef

In [ ]:
def restacked_ief(*args, **kwargs):
    return restack_plt_to_period(integrated_enhancement_factor(*args, **kwargs))

#### Plot a comparison of the reflectance and absorptance of the LOPC with that of the reference slab.

In [ ]:
opts_R = [opts.Curve(color=blue, ylim=(0,1)), opts.Image(cmap='viridis', clim=(0,1)), opts.QuadMesh(cmap='viridis', clim=(0,1)), opts.Violin(violin_fill_color=blue)]

def plot_R(variable="R", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_R)
    return plot

# # test
# plot_R(dataset=restacked_ds.mean("run", keep_attrs=True).sel(period=200, excitonic_layer_thickness=50, num_periods=10).squeeze(), x="wavelength", y="theta").opts(cmap="cividis", clim=(None, None))

In [ ]:
opts_T = [opts.Curve(color=yellow, ylim=(0,1)), opts.Image(cmap='cividis', clim=(0,1)), opts.QuadMesh(cmap='cividis', clim=(0,1)), opts.Violin(violin_fill_color=yellow)]

def plot_T(variable="T", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_T)
    return plot

In [ ]:
opts_A = [opts.Curve(color=red, ylim=(0,1)), opts.Image(cmap='inferno', clim=(0,1)), opts.QuadMesh(cmap='inferno', clim=(0,1)), opts.Violin(violin_fill_color=red)]

def plot_A(variable="A", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_A)
    return plot

In [ ]:
def plot_vars_to_funcs(plot_vars):
    var_func_mapping = {
        "R": plot_R,
        "T": plot_T,
        "A": plot_A,
        "Rs": partial(plot_R, variable="Rs"),
        "Ts": partial(plot_T, variable="Ts"),
        "As": partial(plot_A, variable="As"),
        "Rp": partial(plot_R, variable="Rp"),
        "Tp": partial(plot_T, variable="Tp"),
        "Ap": partial(plot_A, variable="Ap"),
        "R_per_oscillator": partial(plot_R, variable="R_per_oscillator"),
        "T_per_oscillator": partial(plot_T, variable="T_per_oscillator"),
        "A_per_oscillator": partial(plot_A, variable="A_per_oscillator"),
        "Rs_per_oscillator": partial(plot_R, variable="Rs_per_oscillator"),
        "Ts_per_oscillator": partial(plot_T, variable="Ts_per_oscillator"),
        "As_per_oscillator": partial(plot_A, variable="As_per_oscillator"),
        "Rp_per_oscillator": partial(plot_R, variable="Rp_per_oscillator"),
        "Tp_per_oscillator": partial(plot_T, variable="Tp_per_oscillator"),
        "Ap_per_oscillator": partial(plot_A, variable="Ap_per_oscillator"),
    }

    plot_funcs = []
    for var in plot_vars:
        try:
            func = var_func_mapping[var]
        except KeyError:
            func = partial(plot_var, variable=var)
        plot_funcs.append(func)

    return plot_funcs

In [ ]:
# new version
def plot_RTA(
    period,
    excitonic_layer_thickness,
    num_periods,
    theta,
    title="",
    include=["LOPC", "CRS_1"],
    plot_vars=["R", "T", "A"],
    label_override=None,
    label_append=None,
):
    label_field = None  # for debugging
    label_append = "" if label_append is None else label_append

    P = period
    t = excitonic_layer_thickness
    N = num_periods

    plot_funcs = [
        partial(func, x="wavelength", label_field=label_field)
        for func in plot_vars_to_funcs(plot_vars)
    ]
    curves = []
    if "LOPC" in include:
        lopc_label = " (LOPC)" if label_override is None else label_override
        lopc_label += label_append
        lopc_sel = restacked_ds.sel(
            period=P, excitonic_layer_thickness=t, num_periods=N
        ).squeeze()
        lopc_sel = sel_or_integrate(lopc_sel, "theta", theta, normalisation=1)
        lopc_curves = [
            plot_func(dataset=lopc_sel, label_append=lopc_label).opts(line_dash="solid")
            for plot_func in plot_funcs
        ]
        curves += lopc_curves
    if "CRS_1" in include:
        crs_1_label = " (CRS)" if label_override is None else label_override
        crs_1_label += label_append
        crs_1_sel = crs_1.sel(total_excitonic_thickness=t * N).squeeze()
        crs_1_sel = sel_or_integrate(crs_1_sel, "theta", theta, normalisation=1)
        crs_1_curves = [
            plot_func(dataset=crs_1_sel, label_append=crs_1_label).opts(
                line_dash="dashed"
            )
            for plot_func in plot_funcs
        ]
        curves += crs_1_curves
    if "FRS_1" in include:
        frs_1_label = " (FRS)" if label_override is None else label_override
        frs_1_label += label_append
        frs_1_sel = frs_1.sel(total_thickness=(P + t) * N).squeeze()
        frs_1_sel = sel_or_integrate(frs_1_sel, "theta", theta, normalisation=1)
        frs_1_curves = [
            plot_func(dataset=frs_1_sel, label_append=frs_1_label).opts(
                line_dash="dotted"
            )
            for plot_func in plot_funcs
        ]
        curves += frs_1_curves

    overlay = hv.Overlay(curves).opts(
        opts.Curve(
            ylim=(0, 1),
            ylabel="Intensity",
            title=f"{title}{coordinate_string(period=P, excitonic_layer_thickness=t, num_periods=N, theta=theta)}",
        ),
    )

    return overlay


# # test
# display(
#     plot_RTA(200, 50, 20, 0, "test\n", include=["LOPC", "CRS_1"]).opts(
#         legend_position="right"
#     )
# )

# display(
#     plot_RTA(
#         200,
#         50,
#         20,
#         (10, 50),
#         "test RA only\n",
#         include=["LOPC", "CRS_1"],
#         plot_vars=["R", "A"],
#     ).opts(opts.Overlay(legend_position="right"))
# )

# display(
#     plot_RTA(
#         200,
#         50,
#         20,
#         75,
#         "test\n",
#         include=["LOPC"],
#         plot_vars=["R_per_oscillator", "A_per_oscillator"],
#         label_append=" test",
#         label_override="OVERRIDDEN",
#     ).opts(opts.Curve(ylim=(None, None)), opts.Overlay(legend_position="right"))
# )

#### Plot a comparison of normal incidence to integrated

In [ ]:
def plot_comparison(*comparison_params: tuple[dict, list["opts"]], plot_func=plot_RTA, **shared_params):
    param_opts = [(shared_params|comp_params, comp_opts) for comp_params, comp_opts in comparison_params]
    plots = [plot_func(**comp_params).opts(*comp_opts) for comp_params, comp_opts in param_opts]
    return plots

In [ ]:
def compare_RTA(*args, opts_cycle=None, plot_func=plot_RTA, **shared_params):
    default_opts = [
        [opts.Curve(line_dash=style)]
        for style in ["solid", "dashed", "dotted", "dotdash", "dashdot"]
    ]
    opts_cycle = default_opts if opts_cycle is None else opts_cycle

    # comparison_params = list(zip(args, opts_cycle))

    plots = plot_comparison(
        *zip(args, opts_cycle), plot_func=plot_func, **shared_params
    )
    overlay = hv.Overlay(plots).opts(opts.Overlay(legend_position="right"))

    return overlay


# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
#     "include": ["LOPC"],
# }
# compare_RTA({"theta": (0, 75), "label_override": " (integrated)"}, {"theta": 0, "label_override": " (θ = 0)"}, **shared_params)

In [ ]:
compare_RTA_normal_vs_integrated = partial(compare_RTA, {"theta": (0, 75), "label_override": " (integrated)"}, {"theta": 0, "label_override": " (θ = 0)"}, include= ["LOPC"],)

# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
# }
# compare_RTA_normal_vs_integrated(**shared_params)

#### Plot the RTA of the structures in 2D

In [ ]:
def plot_RTA_2D(
    period,
    excitonic_layer_thickness,
    num_periods,
    theta=(0, 75),
    title="",
    include=["LOPC", "CRS_1"],
):
    P = period
    t = excitonic_layer_thickness
    N = num_periods

    plots = []
    if "LOPC" in include:
        lopc_sel = restacked_ds.sel(
            period=P, excitonic_layer_thickness=t, num_periods=N
        ).squeeze()
        lopc_sel = lopc_sel.sel(theta=slice(*theta))
        plots.append(
            lopc_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (LOPC)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            lopc_sel["T"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Transmittance (LOPC)")
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            lopc_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (LOPC)")
            .opts(opts.Image(cmap="inferno"))
        )
    if "CRS_1" in include:
        crs_1_sel = crs_1.sel(total_excitonic_thickness=t * N).squeeze()
        crs_1_sel = crs_1_sel.sel(theta=slice(*theta))
        plots.append(
            crs_1_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (CRS)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            crs_1_sel["T"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Transmittance (CRS)")
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            crs_1_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (CRS)")
            .opts(opts.Image(cmap="inferno"))
        )
    if "FRS_1" in include:
        frs_1_sel = frs_1.sel(total_thickness=(P + t) * N).squeeze()
        frs_1_sel = frs_1_sel.sel(theta=slice(*theta))
        plots.append(
            frs_1_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (FRS)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            frs_1_sel["T"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Transmittance (FRS)")
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            frs_1_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (FRS)")
            .opts(opts.Image(cmap="inferno"))
        )

    layout = hv.Layout(plots).opts(
        opts.Image(
            clim=(0, 1),
            clabel="Intensity",
        ),
        opts.Layout(
            title=f"{title}{coordinate_string(period=P, excitonic_layer_thickness=t, num_periods=N)}",
        ),
    )

    return layout

# # test
# display(plot_RTA_2D(200, 50, 20, (0, 90), "test\n", include=["LOPC", "CRS_1"]).opts(opts.Image(frame_width=200)).cols(3))

# display(plot_RTA_2D(200, 40, 20, (10, 50), "test\n", include=["LOPC", "CRS_1", "FRS_1"]).opts(opts.Image(frame_width=200)).cols(3))

#### Plot an enhancement factor.

In [ ]:
def plot_ef(
    variable,
    dataset,
    sel=None,
    sel_method=None,
    title="",
    *,
    x="wavelength",
    y=None,
):
    sel = {} if sel is None else sel
    da = dataset[variable].sel(**sel, method=sel_method).squeeze()
    if y is None:
        plot = da.hvplot(x=x, label=f"{variable} enhancement factor")
        plot *= hv.HLine(1).opts(line_dash="dotted")
    else:
        plot = da.hvplot(
            kind="image",
            x=x,
            y=y,
            label=f"{variable} enhancement factor",
            clim=(0.5, 1.5),
        )
    plot = plot.opts(
        opts.Curve(
            title=f"{title}{coordinate_string(**sel)}",
        ),
        opts.Overlay(
            title=f"{title}{coordinate_string(**sel)}",
        ),
    )

    return plot


# # test
# sel_1 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 30}
# sel_2 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 0}
# sel_3 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10}
# display(
#     (
#         plot_ef("As", restacked_norm_1, sel_1, "nearest", "test\n")
#         + plot_ef("As", restacked_norm_2, sel_2, title="test2\n")
#     ).cols(1)
# )
# display(
#     plot_ef("As", restacked_norm_1, sel_3, title="test3\n", x="theta", y="wavelength").opts(clim=(0, 2), cmap="RdBu_r")
# )

#### Test plot_optimum_over_dim

In [ ]:
# foo, bar = plot_optimum_over_dim(restacked_ds.A.sel(theta=0, wavelength=660, method="nearest"), "period", "excitonic_layer_thickness", "num_periods", "max")

In [ ]:
# foo, bar = plot_optimum_over_dim(integrate_da(restacked_ds.A, "theta", normalisation=1).sel(wavelength=660, method="nearest"), "period", "excitonic_layer_thickness", "num_periods", "max")

#### Find and plot the min or max over any dimension.

In [ ]:
def wrapped_2D_plot(
    variable,
    dataset,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=None,
    period_stop=None,
    integrate_angle=None,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    dim=None,  # automatically assign if dataset recognised
):
    plots = []

    if str(dataset) == "restacked_ds":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_ds.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    if str(dataset) == "restacked_norm_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_norm_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    if str(dataset) == "restacked_diff_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_diff_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    da = dataset[variable]

    if not integrate_angle:
        da = da.sel(theta=theta, method="nearest")
    else:  # integrate_angle must be a float, so that (theta, integrate_angle) is a slice syntax
        da = da.sel(theta=slice(theta, integrate_angle))
        da = integrate_da(da, "theta", weighting=1, normalisation=1)

    if period_start < period_stop:
        da = da.sel(period=slice(period_start, period_stop))
    else:  # otherwise no data is selected and everything breaks
        da = da.sel(period=slice(period_start, None))
    da = select_lorentz_line(da, lorentz_line=lorentz_line, window_radius=window_radius)

    vline_locs = [0]

    if window_radius == 0:
        wavelength = float(da.wavelength)
        title = f"{optimise.capitalize()}imum {variable} at {wavelength:.0f} nm"
        if lorentz_line != 0:  # don't put two lines over the resonance
            vline_locs.append(lorentz_line)
    else:
        wavelength_start = float(da.wavelength[0])
        wavelength_stop = float(da.wavelength[-1])
        # make it easier to compare values
        da = integrate_da(
            da, "wavelength", weighting=1, normalisation=1
        )  # replaces the below two lines
        # da = normalise_over_dim(da, "wavelength", 1)
        # da = da.integrate("wavelength")
        title = f"{optimise.capitalize()}imum integrated {variable} between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
        vline_locs.append(lorentz_line - window_radius)
        vline_locs.append(lorentz_line + window_radius)

    plot_1, optimum_coords = plot_optimum_over_dim(
        da,
        dim=dim,
        x="excitonic_layer_thickness",
        y="num_periods",
        optimise=optimise,
    )

    P = float(optimum_coords["period"])
    t = float(optimum_coords["excitonic_layer_thickness"])
    N = float(optimum_coords["num_periods"])
    try:  # this should work if not integrating over theta
        th = float(optimum_coords["theta"])
    except:  # probably the problem is that theta doesn't exist because I integrated over it already
        th = (theta, integrate_angle)
    lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
        opts.VLine(line_color=green, line_dash="dotted"),
    )

    # give the resonance line a special colour
    lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

    plot_1.opts(
        opts.QuadMesh(cmap=cmap),
        opts.Points(color="red"),
        opts.Overlay(title=f"{title}\nOptimal period: {P:.0f}"),
    )

    plots.append(plot_1)

    if "RTA_normal" in extra_plots:  # plot RTA at theta=0
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=0
        )
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=th
        )
        new_plot *= lorentz_lines
        plots.append(new_plot)

    if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
        sel = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
            "theta": 0,
        }
        new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if (
        "norm_1_int" in extra_plots
    ):  # plot enhancement factor at theta OR integrating over theta
        try:  # this should work if not integrating over theta
            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "theta": th,
            }
            new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
            new_plot *= lorentz_lines
        except:  # if integrating, we need to do the integral *before* normalising
            ds_int = sel_or_integrate(ds, dim="theta", val=th)
            crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
            norm = enhancement_factor(
                ds_int,
                ref=crs_1_int,
                common_dim="total_excitonic_thickness",
                method="groupby",
            )
            restacked_norm = restack_plt_to_period(norm)
            # replaces the lines below
            #             # this should all get separated out into its own function
            #             crs_1_like_ds = crs_1.sel(
            #                 total_excitonic_thickness=ds.total_excitonic_thickness
            #             )

            #             ds_int = sel_or_integrate(ds, dim="theta", val=th)
            #             crs_1_int = sel_or_integrate(crs_1_like_ds, dim="theta", val=th)
            #             norm = ds_int / crs_1_int
            #             restacked_norm = norm.stack(multiperiod=['passive_layer_thickness', 'excitonic_layer_thickness']).set_index(multiperiod=['period', 'excitonic_layer_thickness']).unstack()

            sel = {"period": P, "excitonic_layer_thickness": t, "num_periods": N}
            new_plot = plot_ef(variable="A", dataset=restacked_norm, sel=sel)
            new_plot *= lorentz_lines
            sel["theta"] = th
            new_plot = new_plot.opts(opts.Overlay(title=f"{coordinate_string(**sel)}"))
        plots.append(new_plot)

    return hv.Layout(plots).cols(1)

#### Plot the E-field, overlayed with the refractive index profile and layer boundaries.

In [ ]:
# # sometimes this errors on the first call for some reason
# fdtd = lumapi.FDTD()

# oscillator = LumericalOscillator(fdtd)

# def wrap_plot_field(
#     wavelength=680,
#     excitonic_layer_thickness=50,
#     passive_layer_thickness=None,
#     period=None,
#     num_periods=5,
#     theta=0,
#     delta=0,
#     run=0,
#     disorder_correlation="correlated",
#     add_first_layer=True,
#     z_scale=1e-9,
#     ri_lower=1.35,
#     ri_upper=1.65,
#     **field_kwargs,
# ):
#     match passive_layer_thickness, period:
#         case None, None:
#             raise ValueError(
#                 "exactly one of passive_layer_thickness or period must be supplied"
#             )
#         case _, None:
#             period = passive_layer_thickness + excitonic_layer_thickness
#         case None, period:
#             passive_layer_thickness = period - excitonic_layer_thickness
#         case _, _:
#             raise ValueError(
#                 "exactly one of passive_layer_thickness or period must be supplied"
#             )

#     default_field_kwargs = dict(
#         wavelength=wavelength,
#         lumerical_session=fdtd,
#         oscillator=oscillator,
#         ri_lower=ri_lower,
#         ri_upper=ri_upper,
#         z_scale=z_scale,
#         excitonic_layer_thickness=excitonic_layer_thickness,
#         passive_layer_thickness=passive_layer_thickness,
#         num_periods=num_periods,
#         angles=theta,
#         add_first_layer=add_first_layer,  # add dummy passive layer to allow disorder on first excitonic layer
#         **default_oscillator_params
#     )

#     if delta != 0:
#         temp_dxs = np.array(
#             [
#                 singles_ds.sel(
#                     delta=delta,
#                     excitonic_layer_thickness=excitonic_layer_thickness,
#                     num_periods=num_periods,
#                     passive_layer_thickness=passive_layer_thickness,
#                     run=run,
#                     disorder_correlation=disorder_correlation,
#                 )
#                 .sel(layer_number=slice(0, num_periods))
#                 .dxs.squeeze()
#                 .compute()
#                 .data
#             ]
#         )

#         default_field_kwargs |= dict(
#             copy_layers=True,
#             apply_disorder=True,
#             delta=1,
#             delta_mode="exact",
#             disorder_type="dxs",
#             dxs=temp_dxs,
#             correlated=True,  # False-> uncorrelated disorder; always use True if applying dxs directly
#             retain_runs=False,  # True-> return concat'd array; False-> return mean array
#         )

#     field_kwargs = default_field_kwargs | field_kwargs

#     with warnings.catch_warnings():
#         warnings.filterwarnings("ignore", "Can't deepcopy")
#         field = plot_field(**field_kwargs)

#     unperturbed_layer_pos = (np.arange(0, num_periods) * period) + (
#         excitonic_layer_thickness / 2 + passive_layer_thickness * add_first_layer
#     )
#     unperturbed_layer_pos *= 1e-9 / z_scale  # default scale if z_scale=1 is metres
#     unperturbed_layer_markers = vlines(list(unperturbed_layer_pos))

#     the_plot = (field * unperturbed_layer_markers).opts(
#         opts.VSpan(color="gray"),
#         opts.VLine(line_dash="dotted", color="black"),
#         opts.Curve(
#             # xlabel=r"\(z (m)\)",  # maybe in some future version of holoviews latex labels will work again :'(
#             # ylabel=r"\(|E|^2\)",
#             xlabel="z (nm)",  # obviously the units here should be matched to z_scale
#             ylabel="|E|^2",
#         ),
#     )

#     return the_plot

#### More plots!

In [ ]:
def plot_singles(
    var,
    run,
    disorder_correlation,
    num_periods,
    period,
    delta,
    theta,
    wavelength_int_range=(None, None),
    integrate_angle=None,
    mode="raw",
):
    plot_funcs = [partial(func, label_field=None) for func in plot_vars_to_funcs(var)]

    match mode:
        case "raw":
            int_func = partial(
                cross_section,
                wavelength=slice(None, None),
                theta=theta,
                integrate_angle=integrate_angle,
                normalisation=1,
            )
            temp_sds = int_func(restacked_sds)
            temp_pds = int_func(restacked_pds)
            temp_stats_ds = int_func(restacked_stats_ds)
            fom_label = f"{var[0]}"
        case "ef":
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", "invalid value")
                int_func = partial(
                    restacked_ief,
                    common_dim="total_excitonic_thickness",
                    method="vectorized_indexing",
                    wavelength=slice(None, None),
                    theta=theta,
                    integrate_angle=integrate_angle,
                    # normalisation=1,
                )
                temp_sds = int_func(singles_ds, ref=crs_1)
                temp_pds = int_func(periodic_ds, ref=crs_1)
                temp_stats_ds = int_func(
                    stats_ds, ref=crs_1_stats_norm, assign_high=True, assign_low=True
                )
            fom_label = f"γ{var[0]}"
        case _:
            raise NotImplementedError(f"valid options for mode are 'raw' or 'ef', not {mode}")

    temp_sds = (
        temp_sds[var]
        .sel(run=run)
        .sel(
            disorder_correlation=disorder_correlation,
            num_periods=num_periods,
            period=period,
            delta=delta,
        )
        .squeeze()
        .load()
    )

    the_plot_1 = temp_sds.hvplot(x="wavelength", by="run", legend=False, alpha=0.5)

    ####
    temp_pds = (
        temp_pds[var]
        .sel(
            num_periods=num_periods,
            period=period,
            excitonic_layer_thickness=excitonic_layer_thickness,
        )
        .squeeze()
        .load()
    )

    the_plot_2 = (temp_pds.hvplot(x="wavelength") * lorentz_vlines(0, scale=1e-9)).opts(
        opts.Curve(
            line_dash="dashed",
            color="black",
            line_width=2,
            ylabel=fom_label,
        ),
        opts.VLine(
            line_dash="dotted",
            color=yellow,
        ),
    )

    if mode == "ef":
        the_plot_2 *= hv.HLine(1).opts(opts.HLine(line_dash="dotted", color="black"))

    ####
    temp_mmds = max_min_pos(
        temp_sds.drop_vars(
            list(
                variable
                for variable in temp_sds.variables.keys()
                if variable not in [*var, "run", "wavelength"]
            )
        ),
        var_key=var,
        dim="wavelength",
    )

    the_plot_3 = temp_mmds.hvplot(
        kind="scatter",
        x=f"{var[0]}_max_wavelength",
        y=f"{var[0]}_max",
        by="run",
        groupby=[],
        alpha=0.5,
        marker="x",
        size=100,
        color="gray",
        legend=False,
    ).opts(opts.Scatter(line_width=2, xlabel="λ (nm)"))

    ####
    match mode:
        case "raw":
            # if I'm working with the raw FOMs it's fine to integrate the existing dataset again
            temp_sds_int = integrate_da(
                temp_sds.sel(wavelength=slice(*wavelength_int_range)),
                dim="wavelength",
                normalisation=1,
            )
            temp_pds_int = integrate_da(
                temp_pds.sel(wavelength=slice(*wavelength_int_range)),
                dim="wavelength",
                normalisation=1,
            )
        case "ef":
            # if I'm working with the EF then I NEED to go back to the original dataset
            # and do the integral before calculating the enhancement factor
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", "invalid value")
                # I can re-use the int_func defined before though
                temp_sds_int = (
                    int_func(
                        singles_ds[var],
                        ref=crs_1,
                        wavelength=wavelength_int_range,
                        assign_high=False,
                        assign_low=False,  # don't need to waste CPU cycles computing these
                    )
                    .sel(run=run)
                    .sel(
                        disorder_correlation=disorder_correlation,
                        num_periods=num_periods,
                        period=period,
                        delta=delta,
                    )
                    .squeeze()
                    .load()
                )
                temp_pds_int = (
                    int_func(
                        periodic_ds[var],
                        ref=crs_1,
                        wavelength=wavelength_int_range,
                        assign_high=False,
                        assign_low=False,  # don't need to waste CPU cycles computing these
                    )
                    .sel(
                        num_periods=num_periods,
                        period=period,
                        excitonic_layer_thickness=excitonic_layer_thickness,
                    )
                    .squeeze()
                    .load()
                )

    scatter = hv.Scatter(
        (
            temp_mmds[f"{var[0]}_max"],
            temp_sds_int[var[0]],
        )
    ).opts(color="gray")

    slope = hv.Slope.from_scatter(scatter).opts(line_dash="dashed")

    the_plot_4 = (scatter * slope) * (
        (
            hv.Scatter(
                (
                    temp_pds[var[0]].max().data,
                    temp_pds_int[var[0]].data,
                ),
            ).opts(color=red, marker="o", size=10, line_width=2)
        ).opts(opts.Scatter(xlabel=f"Max {fom_label}", ylabel=f"{fom_label}_tot"))
    )

    ####
    # I decided that I don't want violin plots anymore and this is the easiest way of excising them
    violin_plots = [
        hv.Text(0, 0, "violin dummy")
    #     plot_func(
    #         variable=var_key,
    #         dataset=temp_sds_int,
    #         kind="violin",
    #     )
    #     for plot_func, var_key in zip(plot_funcs, var)
    ]

    box_plots = [
        hv.Text(1, 1, "box dummy")
        # hv.BoxWhisker(
        #     [temp_pds_int[var_key].data],
        #     label=var_key,
        # )
        # for var_key in var
    ]

    the_plot_5 = hv.Layout(
        [violin * box for violin, box in zip(violin_plots, box_plots)]
    ).opts(opts.BoxWhisker(whisker_line_width=2), opts.Violin(ylabel=""))

    ####
    temp_stats_ds = (
        temp_stats_ds.drop_dims("layer_number")
        .sel(
            disorder_correlation=disorder_correlation,
            num_periods=num_periods,
            period=period,
            delta=delta,
        )
        .squeeze()
    )

    the_plot_6 = hv.Overlay(
        [
            plot_func(
                variable=f"{var_key}_mean",
                dataset=temp_stats_ds,
                x="wavelength",
                legend=False,
            )
            for plot_func, var_key in zip(plot_funcs, var)
        ]
    ).opts(opts.Overlay(show_legend=False))

    ####
    the_plot_7 = hv.Overlay(
        [
            temp_stats_ds.hvplot.area(
                x="wavelength",
                y=f"{var_key}_low",
                y2=f"{var_key}_high",
                color="gray",
                alpha=0.2,
            )
            for var_key in var
        ]
    )

    return (
        the_plot_1,
        the_plot_2,
        the_plot_3,
        the_plot_4,
        the_plot_5,
        the_plot_6,
        the_plot_7,
    )

In [ ]:
def wrapped_plot_singles(
    variable,
    disorder_correlation,
    period,
    num_periods,
    delta,
    theta,
    integrate_angle,
    mode,
    run=slice(0, 1),
    wavelength_int_range=(None, None),
    save_figs=False,
):
    display_plots = []
    var = [variable]

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", "divide")
        the_plots = plot_singles(
            var=var,
            run=run,
            disorder_correlation=disorder_correlation,
            num_periods=num_periods,
            period=period,
            delta=delta,
            theta=theta,
            integrate_angle=integrate_angle,
            mode=mode,
            wavelength_int_range=wavelength_int_range,
        )

        overlay_1 = (
            hv.Overlay(the_plots[:3])
            .select(run=slice(0, 8))
            .opts(opts.Scatter(alpha=1), clone=True)
        )
        display_plots.append(overlay_1)

        overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
            opts.Curve(
                xlim=(480, 880), ylim=(None, None)
            )  # I don't understand why this is needed but otherwise the ylim defaults to (0, 1)
        )
        display_plots.append(overlay_2)

        max_vs_integral = the_plots[3].opts(opts.Scatter(apply_ranges=True))
        display_plots.append(max_vs_integral)

        # violin_plot = the_plots[4]
        # display(violin_plot)

        if save_figs:
            P = period
            elt = 50
            N = num_periods
            D = delta
            disorder_code = disorder_correlation[0]
            D = str(delta).replace(".", "")

            hv.save(
                overlay_1,
                filename=fig_path
                / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
                fmt="png",
                toolbar=False,
            )

            hv.save(
                overlay_2,
                filename=fig_path
                / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
                fmt="png",
                toolbar=False,
            )

            hv.save(
                max_vs_integral,
                filename=fig_path
                / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
                fmt="png",
                toolbar=False,
            )

    return hv.Layout(display_plots).cols(1)

In [ ]:
def plot_curve_max_min(dataset, variable, dim="wavelength", **hvplot_kwargs):
    """
    Scatter plot of max or min points on Curve.
    """
    default_hvplot_kwargs = dict(
        kind="scatter",
        x=f"{variable}_max_{dim}",
        y=f"{variable}_max",
        by=["run"],
        groupby=[],
        alpha=1,
        marker="o",
        size=100,
        color=blue,
        legend=False,
    )

    hvplot_kwargs = default_hvplot_kwargs|hvplot_kwargs
    
    temp_mmds = max_min_pos(
        dataset.drop_vars(
            list(
                var
                for var in dataset.variables.keys()
                if var not in [variable, dim, *hvplot_kwargs["by"], *hvplot_kwargs["groupby"]]
            )
        ),
        var_key=variable,
        dim=dim,
    )

    the_plot = temp_mmds.hvplot(**hvplot_kwargs)
    
    return the_plot

In [ ]:
def sel_slice_or_nearest(ds, dim, val):
    match val:
        case None:
            pass
        case _:
            try:
                ds = ds.sel(**{dim: val})
            except KeyError:
                ds = ds.sel(**{dim: val}, method="nearest")
    return ds

In [ ]:
def set_of_strings(str_or_container):
    if isinstance(str_or_container, str):
        return {str_or_container}
    else:
        return set(str_or_container)

In [ ]:
def compare_integrated_stats(
    variable,
    disorder_correlation=None,
    num_periods=None,
    period=None,
    delta=None,
    excitonic_layer_thickness=50,
    wavelength=None,
    lorentz_line=0,
    window_radius=0,
    theta=0,
    integrate_angle=None,
    normalisation=1,
    include=["DLOPC", "CRS"],
    color_cycle=default_color_cycle,
    dash_cycle=default_dash_cycle,
    mode="raw",
    **hvplot_kwargs,
):
    plots = []

    match mode:
        case "raw":
            ds = restacked_stats_ds[f"{variable}_mean"]
            for dim, val in zip(
                [
                    "disorder_correlation",
                    "num_periods",
                    "period",
                    "delta",
                    "excitonic_layer_thickness",
                ],
                [
                    disorder_correlation,
                    num_periods,
                    period,
                    delta,
                    excitonic_layer_thickness,
                ],
            ):
                ds = sel_slice_or_nearest(ds, dim, val)
            ds = cross_section(
                ds,
                wavelength=wavelength,
                lorentz_line=lorentz_line,
                window_radius=window_radius,
                theta=theta,
                integrate_angle=integrate_angle,
                normalisation=normalisation,
            )
            fom_label = variable
        case "ef":
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", "invalid value")
                int_func = partial(
                    restacked_ief,
                    ref=crs_1_stats_norm,
                    common_dim="total_excitonic_thickness",
                    method="vectorized_indexing",
                    wavelength=wavelength,
                    lorentz_line=lorentz_line,
                    window_radius=window_radius,
                    theta=theta,
                    integrate_angle=integrate_angle,
                )
                ds = stats_ds
                ds = restacked_ief(
                    ds,
                    ref=crs_1_stats_norm,
                    common_dim="total_excitonic_thickness",
                    method="vectorized_indexing",
                    wavelength=wavelength,
                    lorentz_line=lorentz_line,
                    window_radius=window_radius,
                    theta=theta,
                    integrate_angle=integrate_angle,
                )
                ds = ds[f"{variable}_mean"]
                for dim, val in zip(
                    [
                        "disorder_correlation",
                        "num_periods",
                        "period",
                        "delta",
                        "excitonic_layer_thickness",
                    ],
                    [
                        disorder_correlation,
                        num_periods,
                        period,
                        delta,
                        excitonic_layer_thickness,
                    ],
                ):
                    ds = sel_slice_or_nearest(ds, dim, val)
                ds = ds.compute()
            fom_label = f"γ{variable}"
        case _:
            raise NotImplementedError(
                f"valid options for mode are 'raw' or 'ef', not {mode}"
            )

    ds = ds.squeeze()

    if "DLOPC" in include:
        plots.append(
            ds.hvplot(**hvplot_kwargs).opts(
                opts.Curve(ylabel=fom_label, color=color_cycle, line_dash=dash_cycle)
            )
        )

    if "CRS" in include:
        match mode:
            case "raw":
                temp_crs = crs_1[variable].assign_coords(
                    {
                        "num_periods": lambda x: x.total_excitonic_thickness
                        / excitonic_layer_thickness
                    }
                )
                temp_crs = temp_crs.swap_dims(total_excitonic_thickness="num_periods")
                temp_crs = sel_slice_or_nearest(temp_crs, "num_periods", num_periods)
                temp_crs = cross_section(
                    temp_crs,
                    wavelength=wavelength,
                    lorentz_line=lorentz_line,
                    window_radius=window_radius,
                    theta=theta,
                    integrate_angle=integrate_angle,
                    normalisation=normalisation,
                )
                temp_crs = temp_crs.squeeze()

                # only pass kwargs to hvplot if they're relevant to the CRS
                filtered_kwargs = {
                    k: v
                    for k, v in hvplot_kwargs.items()
                    if set_of_strings(v).issubset(temp_crs.coords.keys())
                }

                plots.append(
                    temp_crs.hvplot(**filtered_kwargs).opts(
                        opts.Curve(color="black", line_dash="dotted")
                    )
                )
            case "ef":
                plots.append(
                    hv.HLine(1).opts(
                        opts.HLine(color="black", line_dash="dotted", apply_ranges=True)
                    )
                )

    return hv.Overlay(plots)

In [ ]:
def compare_integrated_stats(
    variable,
    disorder_correlation=None,
    num_periods=None,
    period=None,
    delta=None,
    excitonic_layer_thickness=50,
    wavelength=None,
    lorentz_line=0,
    window_radius=0,
    theta=0,
    integrate_angle=None,
    normalisation=1,
    include=["DLOPC", "CRS"],
    color_cycle=default_color_cycle,
    dash_cycle=default_dash_cycle,
    mode="raw",
    include_std=False,
    **hvplot_kwargs,
):
    plots = []

    match mode:
        case "raw":
            ds = restacked_stats_ds
            for dim, val in zip(
                [
                    "disorder_correlation",
                    "num_periods",
                    "period",
                    "delta",
                    "excitonic_layer_thickness",
                ],
                [
                    disorder_correlation,
                    num_periods,
                    period,
                    delta,
                    excitonic_layer_thickness,
                ],
            ):
                ds = sel_slice_or_nearest(ds, dim, val)
            ds = cross_section(
                ds,
                wavelength=wavelength,
                lorentz_line=lorentz_line,
                window_radius=window_radius,
                theta=theta,
                integrate_angle=integrate_angle,
                normalisation=normalisation,
            )
            fom_label = variable
        case "ef":
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", "invalid value")
                int_func = partial(
                    restacked_ief,
                    ref=crs_1_stats_norm,
                    common_dim="total_excitonic_thickness",
                    method="vectorized_indexing",
                    wavelength=wavelength,
                    lorentz_line=lorentz_line,
                    window_radius=window_radius,
                    theta=theta,
                    integrate_angle=integrate_angle,
                )
                ds = stats_ds
                ds = restacked_ief(
                    ds,
                    ref=crs_1_stats_norm,
                    common_dim="total_excitonic_thickness",
                    method="vectorized_indexing",
                    wavelength=wavelength,
                    lorentz_line=lorentz_line,
                    window_radius=window_radius,
                    theta=theta,
                    integrate_angle=integrate_angle,
                    assign_high=include_std,
                    assign_low=include_std,
                )
                ds = ds
                for dim, val in zip(
                    [
                        "disorder_correlation",
                        "num_periods",
                        "period",
                        "delta",
                        "excitonic_layer_thickness",
                    ],
                    [
                        disorder_correlation,
                        num_periods,
                        period,
                        delta,
                        excitonic_layer_thickness,
                    ],
                ):
                    ds = sel_slice_or_nearest(ds, dim, val)
                # ds = ds.compute()
            fom_label = f"γ{variable}"
        case _:
            raise NotImplementedError(
                f"valid options for mode are 'raw' or 'ef', not {mode}"
            )

    ds = ds.squeeze()

    if "DLOPC" in include:
        plots.append(
            ds[f"{variable}_mean"].hvplot.line(**hvplot_kwargs).opts(
                opts.Curve(ylabel=fom_label, color=color_cycle, line_dash=dash_cycle)
            )
        )
        
        if include_std:
            plots.append(
            ds.hvplot.area(y=f"{variable}_low", y2=f"{variable}_high", **hvplot_kwargs).opts(
                opts.Area(ylabel=fom_label, color=color_cycle, line_dash=dash_cycle, alpha=0.2)
            )
        )

    if "CRS" in include:
        match mode:
            case "raw":
                temp_crs = crs_1[variable].assign_coords(
                    {
                        "num_periods": lambda x: x.total_excitonic_thickness
                        / excitonic_layer_thickness
                    }
                )
                temp_crs = temp_crs.swap_dims(total_excitonic_thickness="num_periods")
                temp_crs = sel_slice_or_nearest(temp_crs, "num_periods", num_periods)
                temp_crs = cross_section(
                    temp_crs,
                    wavelength=wavelength,
                    lorentz_line=lorentz_line,
                    window_radius=window_radius,
                    theta=theta,
                    integrate_angle=integrate_angle,
                    normalisation=normalisation,
                )
                temp_crs = temp_crs.squeeze()

                # only pass kwargs to hvplot if they're relevant to the CRS
                filtered_kwargs = {
                    k: v
                    for k, v in hvplot_kwargs.items()
                    if set_of_strings(v).issubset(temp_crs.coords.keys())
                }

                plots.append(
                    temp_crs.hvplot(**filtered_kwargs).opts(
                        opts.Curve(color="black", line_dash="dotted")
                    )
                )
            case "ef":
                plots.append(
                    hv.HLine(1).opts(
                        opts.HLine(color="black", line_dash="dotted", apply_ranges=True)
                    )
                )

    return hv.Overlay(plots)

In [ ]:
def compare_integrated_singles(
    variable,
    disorder_correlation=None,
    num_periods=None,
    period=None,
    delta=None,
    run=None,
    excitonic_layer_thickness=50,
    wavelength=None,
    lorentz_line=0,
    window_radius=0,
    theta=0,
    integrate_angle=None,
    normalisation=1,
    include=["DLOPC", "CRS"],
    color_cycle=default_color_cycle,
    dash_cycle=default_dash_cycle,
    mode="raw",
    maximise=None,
    **hvplot_kwargs,
):
    plots = []

    match mode:
        case "raw":
            ds = restacked_sds[variable]
            for dim, val in zip(
                [
                    "disorder_correlation",
                    "num_periods",
                    "period",
                    "delta",
                    "run",
                    "excitonic_layer_thickness",
                ],
                [
                    disorder_correlation,
                    num_periods,
                    period,
                    delta,
                    run,
                    excitonic_layer_thickness,
                ],
            ):
                ds = sel_slice_or_nearest(ds, dim, val)
            ds = cross_section(
                ds,
                wavelength=wavelength,
                lorentz_line=lorentz_line,
                window_radius=window_radius,
                theta=theta,
                integrate_angle=integrate_angle,
                normalisation=normalisation,
            )
            
            if maximise is not None:
                ds = ds.max(dim=maximise)
            
            fom_label = variable
        case "ef":
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", "invalid value")
                ds = singles_ds
                ds = restacked_ief(
                    ds,
                    ref=crs_1,
                    common_dim="total_excitonic_thickness",
                    method="vectorized_indexing",
                    wavelength=wavelength,
                    lorentz_line=lorentz_line,
                    window_radius=window_radius,
                    theta=theta,
                    integrate_angle=integrate_angle,
                )
                ds = ds[f"{variable}"]
                for dim, val in zip(
                    [
                        "disorder_correlation",
                        "num_periods",
                        "period",
                        "delta",
                        "run",
                        "excitonic_layer_thickness",
                    ],
                    [
                        disorder_correlation,
                        num_periods,
                        period,
                        delta,
                        run,
                        excitonic_layer_thickness,
                    ],
                ):
                    ds = sel_slice_or_nearest(ds, dim, val)
                
                if maximise is not None:
                    ds = ds.max(dim=maximise)
                ds = ds.compute()
            fom_label = f"γ{variable}"
        case _:
            raise NotImplementedError(
                f"valid options for mode are 'raw' or 'ef', not {mode}"
            )

    ds = ds.squeeze()

    if "DLOPC" in include:
        plots.append(
            ds.hvplot(**hvplot_kwargs).opts(
                opts.Curve(ylabel=fom_label, color=color_cycle, line_dash=dash_cycle)
            )
        )

    if "CRS" in include:
        match mode:
            case "raw":
                temp_crs = crs_1[variable].assign_coords(
                    {
                        "num_periods": lambda x: x.total_excitonic_thickness
                        / excitonic_layer_thickness
                    }
                )
                temp_crs = temp_crs.swap_dims(total_excitonic_thickness="num_periods")
                temp_crs = sel_slice_or_nearest(temp_crs, "num_periods", num_periods)
                temp_crs = cross_section(
                    temp_crs,
                    wavelength=wavelength,
                    lorentz_line=lorentz_line,
                    window_radius=window_radius,
                    theta=theta,
                    integrate_angle=integrate_angle,
                    normalisation=normalisation,
                )
                temp_crs = temp_crs.squeeze()

                # only pass kwargs to hvplot if they're relevant to the CRS
                filtered_kwargs = {
                    k: v
                    for k, v in hvplot_kwargs.items()
                    if set_of_strings(v).issubset(temp_crs.coords.keys())
                }

                plots.append(
                    temp_crs.hvplot(**filtered_kwargs).opts(
                        opts.Curve(color="black", line_dash="dotted")
                    )
                )
            case "ef":
                plots.append(
                    hv.HLine(1).opts(
                        opts.HLine(color="black", line_dash="dotted", apply_ranges=True)
                    )
                )

    return hv.Overlay(plots)

# Plot!

In [ ]:
# restacked_norm_1(aggregated_ds)["A"].sel(
#     period=210, excitonic_layer_thickness=20, num_periods=5, theta=0
# ).squeeze().hvplot(x="wavelength", groupby=["disorder_correlation", "delta"])

In [ ]:
# the_plot = restacked_norm_1(aggregated_ds)["A"].sel(
#     period=210, excitonic_layer_thickness=20, num_periods=5, theta=0
# ).squeeze().hvplot(x="wavelength", y="delta", row="disorder_correlation")

In [ ]:
# the_plot.opts(opts.Image(cmap="RdBu_r"))

In [ ]:
# the_plot = restack_plt_to_period(aggregated_ds)["R"].sel(
#     period=210, excitonic_layer_thickness=20, num_periods=5, theta=0
# ).squeeze().hvplot(x="wavelength", y="delta", row="disorder_correlation")

In [ ]:
# the_plot.opts(opts.Image(cmap="viridis"))

In [ ]:
the_plot = restack_plt_to_period(aggregated_ds)["R"].sel(
    period=210, excitonic_layer_thickness=20, theta=0
).squeeze().hvplot(x="wavelength", groupby=["delta", "disorder_correlation", "num_periods"])

In [ ]:
the_plot

In [ ]:
the_plot = restack_plt_to_period(aggregated_ds)["A"].sel(
    period=210, excitonic_layer_thickness=20, num_periods=5,
).squeeze().integrate("wavelength").hvplot(x="theta", by=["disorder_correlation", "delta"])

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = integrate_da(
    restack_plt_to_period(stats_ds)["A_mean"]
    .sel(
        period=200,
        num_periods=5,
    )
    .squeeze(),
    dim="wavelength",
    normalisation=1,
).hvplot(x="theta", by=["disorder_correlation", "delta"])

In [ ]:
the_plot_2 = integrate_da(
    crs_1_stats_norm["A_mean"].sel(total_excitonic_thickness=250),
    dim="wavelength",
    normalisation=1,
).hvplot(x="theta", line_color="black", line_dash="dashed")

In [ ]:
(the_plot*the_plot_2).opts(opts.Overlay(legend_cols=2))

In [ ]:
the_plot = restack_plt_to_period(stats_ds)["A_mean"].sel(
    period=300, num_periods=5,
).squeeze().integrate("wavelength").hvplot(x="theta", by=["disorder_correlation", "delta"])

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restack_plt_to_period(stats_ds)["A_mean"].sel(
    period=200, num_periods=5, theta=0
).squeeze().hvplot(x="wavelength", by="delta", row="disorder_correlation")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_norm_1_stats(stats_ds)["A_mean"].sel(
    period=100, num_periods=5, theta=0, disorder_correlation="correlated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_norm_1_stats(stats_ds)["A_mean"].sel(
    period=200, num_periods=5, theta=0, disorder_correlation="correlated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_stats_ds["R_mean"].sel(
    period=200, num_periods=5, theta=0, disorder_correlation="correlated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_norm_1_stats(stats_ds)["A_mean"].sel(
    period=200, num_periods=50, theta=0, disorder_correlation="correlated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_norm_1_stats(stats_ds)["A_mean"].sel(
    period=300, num_periods=5, theta=0, disorder_correlation="correlated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_norm_1_stats(stats_ds)["A_mean"].sel(
    period=300, num_periods=50, theta=0, disorder_correlation="correlated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_norm_1_stats(stats_ds)["A_mean"].sel(
    period=200, num_periods=5, theta=0, disorder_correlation="uncorrelated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_norm_1_stats(stats_ds)["A_mean"].sel(
    period=200, num_periods=50, theta=0, disorder_correlation="uncorrelated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_norm_1_stats(stats_ds)["A_mean"].sel(
    period=300, num_periods=5, theta=0, disorder_correlation="uncorrelated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = (restacked_stats_ds)["R_mean"].sel(
    period=300, num_periods=5, theta=0, disorder_correlation="uncorrelated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = restacked_norm_1_stats(stats_ds)["A_mean"].sel(
    period=300, num_periods=50, theta=0, disorder_correlation="uncorrelated"
).squeeze().hvplot(x="wavelength", by="delta")

In [ ]:
the_plot.opts(opts.Curve())

In [ ]:
the_plot = integrate_da(
    restacked_stats_ds["T_mean"].sel(disorder_correlation="uncorrelated").sel(delta=slice(0,0.5), theta=slice(0, 45)).squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
).hvplot(
    kind="quadmesh",
    x="delta",
    y="num_periods",
    row="period",
    # col="disorder_correlation",
)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="cividis", colorbar=False, clim=(0.4, 1)))
the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

## Plots for explaining how disorder works

### What do disordered structures look like?

In [ ]:
runs = [10, 20, 30]
correlated_example_plots = {run: wrap_plot_field(
    wavelength=680,
    excitonic_layer_thickness=50,
    period=200,
    num_periods=10,
    theta=0,
    delta=0.5,
    run=run,
    disorder_correlation='correlated',
    add_first_layer=True,
) for run in runs}

In [ ]:
the_plot = hv.NdLayout(correlated_example_plots).cols(1)
for run in runs[:2]:
    the_plot[run].opts(opts.Curve(xaxis="bare"))
the_plot.opts(opts.Curve(alpha=0, yaxis="bare", frame_height=50, xlim=(0, 2300)))

In [ ]:
hv.save(the_plot, filename=fig_path/"correlated_disorder_example", fmt="png", toolbar=False)

In [ ]:
runs = [10, 20, 30]
uncorrelated_example_plots = {run: wrap_plot_field(
    wavelength=680,
    excitonic_layer_thickness=50,
    period=200,
    num_periods=10,
    theta=0,
    delta=0.5,
    run=run,
    disorder_correlation='uncorrelated',
    add_first_layer=True,
) for run in runs}

In [ ]:
the_plot = hv.NdLayout(uncorrelated_example_plots).cols(1)
for run in runs[:2]:
    the_plot[run].opts(opts.Curve(xaxis="bare"))
the_plot.opts(opts.Curve(alpha=0, yaxis="bare", frame_height=50, xlim=(0, 2300)))

In [ ]:
hv.save(the_plot, filename=fig_path/"uncorrelated_disorder_example", fmt="png", toolbar=False)

### What do the disordered FOMs look like?

In [ ]:
compare_integrated_stats(
    var,
    "correlated",
    period=[100, 200, 300],
    delta=0.3,
    theta=0,
    wavelength=(480, 881),
    x="num_periods",
    by="period",
).opts(opts.Curve(ylim=(0, None)))

In [ ]:
compare_integrated_stats(
    "R",
    "uncorrelated",
    period=200,
    delta=[0, 0.1, 0.3, 0.5],
    theta=0,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))

In [ ]:
compare_integrated_stats(
    "R",
    "correlated",
    period=[200, 300],
    delta=[0, 0.5],
    theta=0,
    wavelength=(480, 881),
    x="num_periods",
    by=["period", "delta"],
).opts(opts.Curve(ylim=(0, None)))

In [ ]:
restacked_stats_ds.sel(period=300, delta=0, num_periods=10, disorder_correlation="correlated", theta=0).R_mean.squeeze().hvplot(x="wavelength")*lorentz_vlines([4, 6, 8], scale=1e-9)

In [ ]:
(compare_integrated_stats(
    "R",
    period=300,
    delta=[0, 0.1, 0.3, 0.5],
    theta=0,
    lorentz_line=6,
    window_radius=2,
    x="num_periods",
    by="delta",
).collate()).layout().opts(opts.Curve(ylim=(0, None)))

#### Reflectance

Generally it seems that the lower the reflectance peak, the wider the reflectance is spread out. Does the spreading counteract the dimming so that the integrated reflectance remains high, or even higher?

From the plots below, it would seem not (at normal incidence). There is a weak positive correlation between peak height and integrated reflectance.
* The correlation seems stronger at lower N.
* The periodic peak consistently has a higher max than the disordered peaks when N>10, but whether the integrated reflectance is higher depends on the number of layers. It's higher when N is low and generally lower when N is high.

##### Correlated

In [ ]:
var = ["R"]
run = slice(0, 1)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 200
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(
    var, run, disorder_correlation, num_periods, period, delta, theta
)

overlay_1 = (
    hv.Overlay(the_plots[:3])
    .select(run=slice(0, 8))
    .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(
    opts.Curve(ylim=(0, None))
)
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 1)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 200
delta = 0.3
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 1)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 200
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 1)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 200
delta = 1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 99)
disorder_correlation = "correlated"
N = num_periods = 10
P = period = 200
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 10
P = period = 200
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
var = ["R"]
run = slice(0, 99)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 200
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 200
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
var = ["R"]
run = slice(0, 1)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 200
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 1)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 200
delta = 0.3
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 1)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 200
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 1)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 200
delta = 1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

P=250

P=300

In [ ]:
var = ["R"]
run = slice(0, 99)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 300
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 99)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 300
delta = 0.3
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 300
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 99)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 300
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 300
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

Period=100, D=0.5

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 100
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 100
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 100
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

Period=100, D=1?

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 100
delta = 1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 100
delta = 1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 100
delta = 1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)

In [ ]:
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
reduced_overlay = overlay_1.select(run=0)

In [ ]:
reduced_overlay.NdOverlay.I.opts(opts.Curve(color=blue, alpha=1))

In [ ]:
temp_curve = reduced_overlay.NdOverlay.I.data

In [ ]:
peaks = find_peaks(temp_curve[var[0]])[0]

In [ ]:
wls = temp_curve.loc[peaks].wavelength.values

In [ ]:
vals = temp_curve.loc[peaks][var[0]].values

In [ ]:
arrows = [hv.Arrow(wl, val, f"{i+1}", 'v') for i, (wl, val) in enumerate(zip(wls, vals))]

In [ ]:
the_plot = (reduced_overlay*hv.Overlay(arrows)).opts(opts.Curve(ylim=(0, 0.15)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}_annotated",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 5)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 100
delta = 1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=1)

In [ ]:
reduced_overlay.NdOverlay.I.opts(opts.Curve(color=blue, alpha=1))

In [ ]:
temp_curve = reduced_overlay.NdOverlay.I.data.reset_index()

In [ ]:
peaks = find_peaks(temp_curve[var[0]])[0]

In [ ]:
wls = temp_curve.loc[peaks].wavelength.values

In [ ]:
vals = temp_curve.loc[peaks][var[0]].values

In [ ]:
arrows = [hv.Arrow(wl, val, f"{i+1}", 'v') for i, (wl, val) in enumerate(zip(wls, vals))]

In [ ]:
the_plot = (reduced_overlay*hv.Overlay(arrows)).opts(opts.Curve(ylim=(0, 0.03)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}_annotated",
    fmt="png",
    toolbar=False,
)

In [ ]:
the_plot_1 = wrap_plot_field(
    wavelength=778,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    # delta=delta,
    # run=5,
    # disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_2 = wrap_plot_field(
    wavelength=734,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=1,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_3 = wrap_plot_field(
    wavelength=778,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=1,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_4 = wrap_plot_field(
    wavelength=847,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=1,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot = (
    hv.Layout(
        [
            the_plot_1,
            the_plot_2,
            the_plot_3,
            the_plot_4,
        ]
    ).opts(opts.Curve(color=blue))
).cols(
    1
)  # autoranging, yeah!
the_plot

In [ ]:
var = ["R"]
run = slice(0, 5)
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 100
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=5)

In [ ]:
reduced_overlay.NdOverlay.I.opts(opts.Curve(color=blue, alpha=1))

In [ ]:
temp_curve = reduced_overlay.NdOverlay.I.data.reset_index()

In [ ]:
peaks = find_peaks(temp_curve[var[0]])[0]

In [ ]:
wls = temp_curve.loc[peaks].wavelength.values

In [ ]:
vals = temp_curve.loc[peaks][var[0]].values

In [ ]:
arrows = [hv.Arrow(wl, val, f"{i+1}", 'v') for i, (wl, val) in enumerate(zip(wls, vals))]

In [ ]:
the_plot = (reduced_overlay*hv.Overlay(arrows)).opts(opts.Curve(ylim=(0, 0.09)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}_annotated",
    fmt="png",
    toolbar=False,
)

In [ ]:
the_plot_1 = wrap_plot_field(
    wavelength=535,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    # delta=delta,
    # run=5,
    # disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_2 = wrap_plot_field(
    wavelength=562,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=5,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_3 = wrap_plot_field(
    wavelength=535,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=5,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_4 = wrap_plot_field(
    wavelength=508,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=5,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot = (
    hv.Layout(
        [
            the_plot_1,
            the_plot_2,
            the_plot_3,
            the_plot_4,
        ]
    ).opts(opts.Curve(color=blue))
).cols(
    1
)  # autoranging, yeah!
the_plot

Period=150, D=0.5

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 150
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 150
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 150
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

Period=150, D=1?

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 150
delta = 1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 150
delta = 1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 150
delta = 1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

##### Uncorrelated

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "uncorrelated"
N = num_periods = 5
P = period = 200
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "raw"

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var, run, disorder_correlation, num_periods, period, delta, theta, mode=mode
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,3))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "uncorrelated"
num_periods = 5
period = 200
delta = 0.3
theta = 0
excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(
    var, run, disorder_correlation, num_periods, period, delta, theta
)

overlay_1 = (
    hv.Overlay(the_plots[:3])
    .select(run=slice(0, 8))
    .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[5:], *the_plots[1:3]])
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,3))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "uncorrelated"
N = num_periods = 5
P = period = 200
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(
    var, run, disorder_correlation, num_periods, period, delta, theta
)

overlay_1 = (
    hv.Overlay(the_plots[:3])
    .select(run=slice(0, 8))
    .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(
    opts.Curve(ylim=(0, None))
)
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,3))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
the_plot_1 = wrap_plot_field(
    wavelength=538,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    # delta=delta,
    # run=5,
    # disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_2 = wrap_plot_field(
    wavelength=538,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=2,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_3 = wrap_plot_field(
    wavelength=707,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    # delta=delta,
    # run=1,
    # disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_4 = wrap_plot_field(
    wavelength=707,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=2,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot = (
    hv.Layout(
        [
            the_plot_1,
            the_plot_2,
            the_plot_3,
            the_plot_4,
        ]
    ).opts(opts.Curve(color=blue))
).cols(
    1
)  # autoranging, yeah!
the_plot

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 200
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "raw"

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var, run, disorder_correlation, num_periods, period, delta, theta, mode=mode
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

    overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(
        opts.Curve(ylim=(0, None))
    )
    display(overlay_2)

    max_vs_integral = the_plots[3]
    display(max_vs_integral)

    violin_plot = the_plots[4]
    display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,3))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "uncorrelated"
num_periods = 20
period = 200
delta = 0.3
theta = 0
excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(
    var, run, disorder_correlation, num_periods, period, delta, theta
)

overlay_1 = (
    hv.Overlay(the_plots[:3])
    .select(run=slice(0, 8))
    .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[5:], *the_plots[1:3]])
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,3))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 200
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(
    var, run, disorder_correlation, num_periods, period, delta, theta
)

overlay_1 = (
    hv.Overlay(the_plots[:3])
    .select(run=slice(0, 8))
    .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(
    opts.Curve(ylim=(0, None))
)
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,3))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
the_plot_1 = wrap_plot_field(
    wavelength=502,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    # delta=delta,
    # run=5,
    # disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_2 = wrap_plot_field(
    wavelength=502,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=2,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_3 = wrap_plot_field(
    wavelength=485,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=2,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_4 = wrap_plot_field(
    wavelength=524,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=2,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot = (
    hv.Layout(
        [
            the_plot_1,
            the_plot_2,
            the_plot_3,
            the_plot_4,
        ]
    ).opts(opts.Curve(color=blue))
).cols(
    1
)  # autoranging, yeah!
the_plot

P=200 N=50

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "uncorrelated"
N = num_periods = 50
P = period = 200
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "raw"

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var, run, disorder_correlation, num_periods, period, delta, theta, mode=mode
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

    overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(
        opts.Curve(ylim=(0, None))
    )
    display(overlay_2)

    max_vs_integral = the_plots[3]
    display(max_vs_integral)

    violin_plot = the_plots[4]
    display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,3))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "uncorrelated"
num_periods = 50
period = 200
delta = 0.3
theta = 0
excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(
    var, run, disorder_correlation, num_periods, period, delta, theta
)

overlay_1 = (
    hv.Overlay(the_plots[:3])
    .select(run=slice(0, 8))
    .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[5:], *the_plots[1:3]])
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,3))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["R"]
run = slice(0, 2)
disorder_correlation = "uncorrelated"
N = num_periods = 50
P = period = 200
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(
    var, run, disorder_correlation, num_periods, period, delta, theta
)

overlay_1 = (
    hv.Overlay(the_plots[:3])
    .select(run=slice(0, 8))
    .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(
    opts.Curve(ylim=(0, None))
)
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,3))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
the_plot_1 = wrap_plot_field(
    wavelength=752,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    # delta=delta,
    # run=5,
    # disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_2 = wrap_plot_field(
    wavelength=752,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=2,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_3 = wrap_plot_field(
    wavelength=734,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=2,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot_4 = wrap_plot_field(
    wavelength=770,
    excitonic_layer_thickness=excitonic_layer_thickness,
    period=period,
    num_periods=num_periods,
    theta=theta,
    delta=delta,
    run=2,
    disorder_correlation=disorder_correlation,
)

In [ ]:
the_plot = (
    hv.Layout(
        [
            the_plot_1,
            the_plot_2,
            the_plot_3,
            the_plot_4,
        ]
    ).opts(opts.Curve(color=blue))
).cols(
    1
)  # autoranging, yeah!
the_plot

P=250

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "uncorrelated"
num_periods = 50
period = 250
delta = 0.5
theta = 0
excitonic_layer_thickness = 50

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8))
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]])
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
var = ["R"]
run = slice(0, 99)
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 300
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 99)
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 300
delta = 0.3
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 300
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 99)
disorder_correlation = "uncorrelated"
N = num_periods = 50
P = period = 300
delta = 0.1
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 99)
disorder_correlation = "uncorrelated"
N = num_periods = 50
P = period = 300
delta = 0.3
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "uncorrelated"
N = num_periods = 50
P = period = 300
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

Period=100, D=0.5

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "uncorrelated"
N = num_periods = 5
P = period = 100
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 100
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "uncorrelated"
N = num_periods = 50
P = period = 100
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

Period=150, D=0.5

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "uncorrelated"
N = num_periods = 5
P = period = 150
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 150
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
var = ["R"]
run = slice(0, 999)
disorder_correlation = "uncorrelated"
N = num_periods = 50
P = period = 150
delta = 0.5
theta = 0
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plots = plot_singles(var, run, disorder_correlation, num_periods, period, delta, theta)

overlay_1 = hv.Overlay(the_plots[:3]).select(run=slice(0,8)).opts(opts.Scatter(alpha=1), clone=True)
display(overlay_1)

overlay_2 = hv.Overlay([*the_plots[1:3], *the_plots[5:]]).opts(opts.Curve(ylim=(0, None)))
display(overlay_2)

max_vs_integral = the_plots[3]
display(max_vs_integral)

violin_plot = the_plots[4]
display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

### vs N

#### RTA

In [ ]:
period = 300
theta = 0
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
delta = 0.5

In [ ]:
(
    integrate_da(
        restacked_stats_ds[["R_mean", "T_mean", "A_mean"]]
        .sel(
            disorder_correlation=disorder_correlation,
            period=period,
            delta=delta,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color=RTA_cycle))
    * integrate_da(
        restacked_pds[["R", "T", "A"]]
        .sel(
            period=period,
            excitonic_layer_thickness=excitonic_layer_thickness,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color=RTA_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[["R", "T", "A"]]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=theta,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color=RTA_cycle, line_dash="dotted"))
)

In [ ]:
period = 300
theta = slice(0, 45)
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
delta = 0.5
integration_dims = ["wavelength", "theta"]

In [ ]:
(
    integrate_da(
        restacked_stats_ds[["R_mean", "T_mean", "A_mean"]]
        .sel(
            disorder_correlation=disorder_correlation,
            period=period,
            delta=delta,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=integration_dims,
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color=RTA_cycle))
    * integrate_da(
        restacked_pds[["R", "T", "A"]]
        .sel(
            period=period,
            excitonic_layer_thickness=excitonic_layer_thickness,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=integration_dims,
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color=RTA_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[["R", "T", "A"]]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=theta,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim=integration_dims,
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color=RTA_cycle, line_dash="dotted"))
)

#### by Lambda

In [ ]:
for delta in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]:
    periods = [100, 150, 200, 250, 300]
    theta = 0
    variable = "R"
    disorder_correlation = "correlated"
    excitonic_layer_thickness = 50
    elt = excitonic_layer_thickness = 50
    disorder_code = disorder_correlation[0]
    D = str(delta).replace(".", "")

    the_plot = compare_integrated_stats(
        variable=variable,
        disorder_correlation=disorder_correlation,
        period=periods,
        delta=delta,
        theta=theta,
        num_periods=slice(5, 50),
        wavelength=(480, 881),
        x="num_periods",
        by="period",
        dash_cycle=hv.Cycle(default_dash_cycle.values[::-1])
    ).opts(opts.Curve(ylim=(0, None)))
    # display(the_plot)

    hv.save(
        the_plot,
        filename=fig_path
        /f"{variable}tot_DLOPC_period_comparison_N_elt{elt}_th{theta}_D{D}{disorder_code}",
        fmt="png",
        toolbar=False,
    )

In [ ]:
for delta in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
    periods = [100, 150, 200, 250, 300]
    theta = 0
    variable = "R"
    disorder_correlation = "uncorrelated"
    excitonic_layer_thickness = 50
    elt = excitonic_layer_thickness = 50
    disorder_code = disorder_correlation[0]
    D = str(delta).replace(".", "")

    the_plot = compare_integrated_stats(
        variable=variable,
        disorder_correlation=disorder_correlation,
        period=periods,
        delta=delta,
        theta=theta,
        num_periods=slice(5, 50),
        wavelength=(480, 881),
        x="num_periods",
        by="period",
        dash_cycle=hv.Cycle(default_dash_cycle.values[::-1])
    ).opts(opts.Curve(ylim=(0, None)))
    # display(the_plot)

    hv.save(
        the_plot,
        filename=fig_path
        /f"{variable}tot_DLOPC_period_comparison_N_elt{elt}_th{theta}_D{D}{disorder_code}",
        fmt="png",
        toolbar=False,
    )

In [ ]:
delta = 0
periods = [100, 150, 200, 250, 300]
theta = 0
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=periods,
    delta=delta,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="period",
    dash_cycle=hv.Cycle(default_dash_cycle.values[::-1])
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_period_comparison_N_elt{elt}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 1
periods = [100, 150, 200, 250, 300]
theta = 0
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=periods,
    delta=delta,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="period",
    dash_cycle=hv.Cycle(default_dash_cycle.values[::-1])
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_period_comparison_N_elt{elt}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.3
periods = [100, 150, 200, 250, 300]
theta = 0
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=periods,
    delta=delta,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="period",
    dash_cycle=hv.Cycle(default_dash_cycle.values[::-1])
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_period_comparison_N_elt{elt}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
periods = [100, 150, 200, 250, 300]
color_cycle = hv.Cycle(colorcet.glasbey[:len(periods)])
var = "R"
delta = 0.5

(
    integrate_da(
        restacked_stats_ds[f"{var}_mean"]
        .sel(
            disorder_correlation=disorder_correlation,
            period=periods,
            delta=delta,
            theta=0,
        )
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle))
    * integrate_da(
        restacked_pds[var]
        .sel(
            period=periods,
            excitonic_layer_thickness=excitonic_layer_thickness,
            theta=0,
        )
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[var]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=0,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color="black", line_dash="dotted"))
).opts(opts.Curve(ylabel=var, ylim=(0, None)))

In [ ]:
periods = [100, 150, 200, 250, 300]
color_cycle = hv.Cycle(colorcet.glasbey[:len(periods)])
var = "T"

(
    integrate_da(
        restacked_stats_ds[f"{var}_mean"]
        .sel(
            disorder_correlation=disorder_correlation,
            period=periods,
            delta=delta,
            theta=0,
        )
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle))
    * integrate_da(
        restacked_pds[var]
        .sel(
            period=periods,
            excitonic_layer_thickness=excitonic_layer_thickness,
            theta=0,
        )
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[var]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=0,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color="black", line_dash="dotted"))
).opts(opts.Curve(ylabel=var, ylim=(None, 1)))

In [ ]:
periods = [100, 150, 200, 250, 300]
color_cycle = hv.Cycle(colorcet.glasbey[:len(periods)])
var = "A"

(
    integrate_da(
        restacked_stats_ds[f"{var}_mean"]
        .sel(
            disorder_correlation=disorder_correlation,
            period=periods,
            delta=delta,
            theta=0,
        )
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle))
    * integrate_da(
        restacked_pds[var]
        .sel(
            period=periods,
            excitonic_layer_thickness=excitonic_layer_thickness,
            theta=0,
        )
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[var]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=0,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim="wavelength",
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color="black", line_dash="dotted"))
).opts(opts.Curve(ylabel=var, ylim=(0, None)))

#### by D

##### Correlated

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 100
theta = 0
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 150
theta = 0
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 200
theta = 0
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 250
theta = 0
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 300
theta = 0
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

##### Uncorrelated

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 100
theta = 0
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 150
theta = 0
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 200
theta = 0
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 250
theta = 0
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 300
num_periods=slice(0, 50)
theta = 0
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

##### Integrating over theta?

Integrating to 45 degrees.

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
num_periods=slice(0, 50)
theta = 0
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

for period in [100, 150, 200, 250, 300]:
    P = period
    for integrate_angle in [45, 86]:
        the_plot = compare_integrated_stats(
            variable=variable,
            disorder_correlation=disorder_correlation,
            num_periods=num_periods,
            period=period,
            delta=deltas,
            theta=theta,
            integrate_angle=integrate_angle,
            wavelength=(480, 881),
            x="num_periods",
            by="delta",
        ).opts(opts.Curve(ylim=(0, None)))
        # display(the_plot)

        hv.save(
            the_plot,
            filename=fig_path
            /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
            fmt="png",
            toolbar=False,
        )

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
num_periods=slice(0, 50)
theta = 0
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

for period in [100, 150, 200, 250, 300]:
    P = period
    for integrate_angle in [45, 86]:
        the_plot = compare_integrated_stats(
            variable=variable,
            disorder_correlation=disorder_correlation,
            num_periods=num_periods,
            period=period,
            delta=deltas,
            theta=theta,
            integrate_angle=integrate_angle,
            wavelength=(480, 881),
            x="num_periods",
            by="delta",
        ).opts(opts.Curve(ylim=(0, None)))
        # display(the_plot)

        hv.save(
            the_plot,
            filename=fig_path
            /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
            fmt="png",
            toolbar=False,
        )

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 300
num_periods=slice(0, 50)
theta = 0
integrate_angle = 45
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    integrate_angle=integrate_angle,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 250
num_periods=slice(0, 50)
theta = 0
integrate_angle = 45
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    integrate_angle=integrate_angle,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 250
num_periods=slice(0, 50)
theta = 0
integrate_angle = 45
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    integrate_angle=integrate_angle,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 200
num_periods=slice(0, 50)
theta = 0
integrate_angle = 45
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    integrate_angle=integrate_angle,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 200
num_periods=slice(0, 50)
theta = 0
integrate_angle = 45
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    integrate_angle=integrate_angle,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

Integrating to 86 degrees.

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 300
num_periods=slice(0, 50)
theta = 0
integrate_angle = 86
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    integrate_angle=integrate_angle,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 300
num_periods=slice(0, 50)
theta = 0
integrate_angle = 86
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    integrate_angle=integrate_angle,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 250
num_periods=slice(0, 50)
theta = 0
integrate_angle = 86
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    integrate_angle=integrate_angle,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 250
num_periods=slice(0, 50)
theta = 0
integrate_angle = 86
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    theta=theta,
    integrate_angle=integrate_angle,
    wavelength=(480, 881),
    x="num_periods",
    by="delta",
).opts(opts.Curve(ylim=(0, None)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    /f"{variable}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}-{integrate_angle}_D{disorder_code}",
    fmt="png",
    toolbar=False,
)

#### Comparing FOMs of disordered and periodic structures

In [ ]:
periods = [100, 150, 200, 250, 300]
color_cycle = hv.Cycle(colorcet.glasbey[: len(periods)])
var = "R"
theta = slice(0, 45)

(
    integrate_da(
        restacked_stats_ds[f"{var}_mean"]
        .sel(disorder_correlation=disorder_correlation, period=periods, delta=delta)
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle))
    * integrate_da(
        restacked_pds[var]
        .sel(
            period=periods,
            excitonic_layer_thickness=excitonic_layer_thickness,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[var]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=theta,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color="black", line_dash="dotted"))
).opts(opts.Curve(ylabel=var, ylim=(0, None)))

In [ ]:
periods = [100, 150, 200, 250, 300]
color_cycle = hv.Cycle(colorcet.glasbey[: len(periods)])
var = "T"
theta = slice(0, 45)

(
    integrate_da(
        restacked_stats_ds[f"{var}_mean"]
        .sel(disorder_correlation=disorder_correlation, period=periods, delta=delta)
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle))
    * integrate_da(
        restacked_pds[var]
        .sel(
            period=periods,
            excitonic_layer_thickness=excitonic_layer_thickness,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[var]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=theta,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color="black", line_dash="dotted"))
).opts(opts.Curve(ylabel=var, ylim=(None, 1)))

In [ ]:
periods = [100, 150, 200, 250, 300]
color_cycle = default_color_cycle[len(periods)]
var = "A"
theta = slice(0, 45)

(
    integrate_da(
        restacked_stats_ds[f"{var}_mean"]
        .sel(disorder_correlation=disorder_correlation, period=periods, delta=delta)
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle))
    * integrate_da(
        restacked_pds[var]
        .sel(
            period=periods,
            excitonic_layer_thickness=excitonic_layer_thickness,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[var]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=theta,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color="black", line_dash="dotted"))
).opts(opts.Curve(ylabel=var, ylim=(0, None)))

In [ ]:
periods = [100, 150, 200, 250, 300]
color_cycle = hv.Cycle(colorcet.glasbey[: len(periods)])
var = "R"
theta = slice(0, None)

(
    integrate_da(
        restacked_stats_ds[f"{var}_mean"]
        .sel(disorder_correlation=disorder_correlation, period=periods, delta=delta)
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle))
    * integrate_da(
        restacked_pds[var]
        .sel(
            period=periods,
            excitonic_layer_thickness=excitonic_layer_thickness,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[var]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=theta,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color="black", line_dash="dotted"))
).opts(opts.Curve(ylabel=var, ylim=(0, None)))

In [ ]:
periods = [100, 150, 200, 250, 300]
color_cycle = hv.Cycle(colorcet.glasbey[: len(periods)])
var = "T"
theta = slice(0, None)

(
    integrate_da(
        restacked_stats_ds[f"{var}_mean"]
        .sel(disorder_correlation=disorder_correlation, period=periods, delta=delta)
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle))
    * integrate_da(
        restacked_pds[var]
        .sel(
            period=periods,
            excitonic_layer_thickness=excitonic_layer_thickness,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[var]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=theta,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color="black", line_dash="dotted"))
).opts(opts.Curve(ylabel=var, ylim=(None, 1)))

In [ ]:
periods = [100, 150, 200, 250, 300]
color_cycle = hv.Cycle(colorcet.glasbey[: len(periods)])
var = "A"
theta = slice(0, None)

(
    integrate_da(
        restacked_stats_ds[f"{var}_mean"]
        .sel(disorder_correlation=disorder_correlation, period=periods, delta=delta)
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle))
    * integrate_da(
        restacked_pds[var]
        .sel(
            period=periods,
            excitonic_layer_thickness=excitonic_layer_thickness,
        )
        .sel(
            theta=theta,
        )
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods", by="period")
    .opts(opts.Curve(color=color_cycle, line_dash="dashed"))
    * integrate_da(
        crs_1[var]
        .assign_coords(
            {
                "num_periods": lambda x: x.total_excitonic_thickness
                / excitonic_layer_thickness
            }
        )
        .sel(
            theta=theta,
        )
        .sel(total_excitonic_thickness=slice(None, 2500))
        .squeeze(),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    .hvplot(x="num_periods")
    .opts(opts.Curve(color="black", line_dash="dotted"))

).opts(opts.Curve(ylabel=var, ylim=(0, None)))

## How do correlated and uncorrelated disorder differ?

### P=300

In [ ]:
var = "R"
P = period = 300
N = num_periods = 20
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.7, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = "R"
P = period = 300
N = num_periods = 5
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.1, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = "R"
P = period = 300
N = num_periods = 50
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.95, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = "R"
P = period = 300
N = num_periods = 10
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.33, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

### P=250

In [ ]:
var = "R"
P = period = 250
N = num_periods = 5
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.1, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = "R"
P = period = 250
N = num_periods = 10
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.33, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = "R"
P = period = 250
N = num_periods = 20
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.7, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = "R"
P = period = 250
N = num_periods = 50
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.95, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

### P=200

In [ ]:
var = "R"
P = period = 200
N = num_periods = 5
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.1, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = "R"
P = period = 200
N = num_periods = 10
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.33, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = "R"
P = period = 200
N = num_periods = 20
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.7, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = "R"
P = period = 200
N = num_periods = 50
theta = 0
elt = excitonic_layer_thickness = 50


the_plots = {
    delta: (restacked_stats_ds[f"{var}_mean"]
    .sel(theta=theta, num_periods=num_periods, period=period, delta=delta)
    .squeeze()
    .hvplot.line(x="wavelength", by="disorder_correlation")
    .opts(opts.Curve(ylabel=var, xticks=2, yticks=4, show_legend=False)) * hv.Text(550, 0.95, f"Delta={delta}", fontsize=18))
    for delta in [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}

for d in [0.2, 0.3, 0.5, 1]:
    the_plots[d].opts(opts.Curve(yaxis="bare"))

for d in [0.1, 0.2, 0.3]:
    the_plots[d].opts(opts.Curve(xaxis="bare"))

periodic_curve = (
    restacked_pds[var]
    .sel(
        theta=theta,
        num_periods=num_periods,
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
    )
    .squeeze()
    .hvplot(x="wavelength")
    .opts(color="black", line_dash="dashed")
)

layout = hv.NdLayout(the_plots)
layout *= periodic_curve
layout *= lorentz_vlines(0, scale=1e-9).opts(color=yellow, line_dash="dotted")

layout.cols(3).opts(opts.Curve(frame_width=400, frame_height=300))

In [ ]:
hv.save(
    layout,
    filename=fig_path
    / f"{var}_DLOPC_correlation_comparison_wl_P{P}_elt{elt}_N{N}_th{theta}_D0-1c-uc",
    fmt="png",
    toolbar=False,
)

### Old GridSpace approach

In [ ]:
# var = "A"
# period = 200
# num_periods = 5
# theta = 0
# excitonic_layer_thickness = 50

# restacked_stats_ds[f"{var}_mean"].sel(
#     theta=theta, num_periods=num_periods, period=period
# ).sel(delta=slice(0.1, None)).squeeze().hvplot.line(
#     x="wavelength", by="disorder_correlation", row="delta"
# ).opts(
#     opts.Curve(xticks=1, show_legend=False), opts.GridSpace(shared_xaxis=True)
# ) * restacked_pds[
#     var
# ].sel(
#     theta=theta,
#     num_periods=num_periods,
#     period=period,
#     excitonic_layer_thickness=excitonic_layer_thickness,
# ).squeeze().hvplot(
#     x="wavelength"
# ).opts(
#     color="black", line_dash="dashed"
# )

GridSpace for aggregated dataset - not really useful

In [ ]:
# var = "R"
# period = 220
# num_periods = 30
# theta = 0
# excitonic_layer_thickness = 40

# (
#     restacked_ads[var]
#     .sel(
#         disorder_correlation="correlated",
#         theta=theta,
#         num_periods=num_periods,
#         period=period,
#         excitonic_layer_thickness=excitonic_layer_thickness,
#     )
#     .sel(delta=slice(0.01, None))
#     .squeeze()
#     .hvplot.line(x="wavelength", by=["delta"], subplots=True, label="correlated")
#     .opts(
#         opts.Curve(
#             xticks=1,
#             show_legend=False,
#         ),
#         opts.GridSpace(shared_xaxis=True),
#     )
#     .cols(5)
#     * restacked_ads[var]
#     .sel(
#         disorder_correlation="uncorrelated",
#         theta=theta,
#         num_periods=num_periods,
#         period=period,
#         excitonic_layer_thickness=excitonic_layer_thickness,
#     )
#     .sel(delta=slice(0.01, None))
#     .squeeze()
#     .hvplot.line(x="wavelength", by=["delta"], subplots=True, label="uncorrelated")
#     .opts(
#         opts.Curve(
#             xticks=1,
#             show_legend=False,
#         ),
#         opts.GridSpace(shared_xaxis=True),
#     )
#     .cols(5)
#     * restacked_pds[var]
#     .sel(
#         theta=theta,
#         num_periods=num_periods,
#         period=period,
#         excitonic_layer_thickness=excitonic_layer_thickness,
#     )
#     .squeeze()
#     .hvplot(x="wavelength")
#     .opts(color="black", line_dash="dashed")
# ).opts(opts.Curve(frame_width=100))

## Does disorder make reflectance higher or lower?

In [ ]:
# the_plot = restack_plt_to_period(stats_ds)["R_mean"].sel(
#     period=300, num_periods=5, theta=0
# ).squeeze().hvplot(x="wavelength", by="delta", row="disorder_correlation")

# the_plot.opts(opts.Curve())

In [ ]:
# overlay = compare_integrated_stats(
#     "R",
#     wavelength=slice(480, 880),
#     num_periods=5,
#     period=300,
#     theta=0,
#     include=["DLOPC"],
#     x="wavelength",
#     y="delta",
#     by="disorder_correlation",
#     kind="quadmesh",
# )

# layout = hv.NdLayout(overlay.NdOverlay.I).opts(opts.QuadMesh(cmap="viridis"), opts.NdLayout(shared_axes=False))

# layout["uncorrelated"].opts(opts.QuadMesh(ylim=(0,0.5)))
# layout

In [ ]:
# the_plot = integrate_da(
#     restacked_stats_ds["R_mean"].sel(disorder_correlation="correlated").squeeze(),
#     dim=["wavelength", "theta"],
#     normalisation=1,
# ).hvplot(
#     kind="quadmesh",
#     x="delta",
#     y="num_periods",
#     row="period",
#     # col="disorder_correlation",
# )

# the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0.02, 0.125)))
# the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
# the_plot

In [ ]:
# hv.save(R_int_grid_plot, filename=fig_path/"R_dLOPC_2D_d_N", fmt="png", toolbar=False)

### Big loop

In [ ]:
# def load_singles_plots(variable, disorder_correlation, N, P, D, elt=50, theta=0, include=("runcurve_wl", "meancurve_wl", "max_vs_tot")):
#     disorder_code = disorder_correlation[0]
#     plots = [hv.RGB.load_image(fig_path / f"") for plot_type in include]
    
#     return plots

I've given up on using a DynamicMap-based approach to this because it seems scatter plots don't autorange for some reason. I've tried `framewise=True`, `xlim=(None, None)`, etc.

In [ ]:
## kdims = [
#     hv.Dimension("variable", values=list(singles_ds.data_vars.keys())),
#     hv.Dimension("disorder_correlation", values=singles_ds.disorder_correlation.data),
#     hv.Dimension("period", values=singles_ds.period.squeeze().data, default=200),
#     hv.Dimension("num_periods", values=singles_ds.num_periods.data),
#     hv.Dimension("delta", values=[0.1, 0.2, 0.3, 0.4, 0.5, 1]),
#     hv.Dimension("theta", range=(0, 86), default=0),
#     hv.Dimension("integrate_angle", range=(0, 86), default=0),
#     hv.Dimension("mode", values=["raw", "ef"], default="raw"),
# ]

# singles_dmap_D01_04 = hv.DynamicMap(partial(wrapped_plot_singles, run=slice(0,99)), kdims=kdims)

In [ ]:
var = ["R"]
theta = 0
elt = excitonic_layer_thickness = 50
mode = "raw"
wavelength_int_range = (None, None)

for period in tqdm([100, 150, 200, 250, 300]):
    for num_periods in [5, 20, 50]:
        for integrate_angle in [None, 45, 86]:
            for delta in [0.1, 0.3, 0.5, 1]:
                for disorder_correlation in ["correlated", "uncorrelated"]:
                    if delta == 1 and disorder_correlation == "uncorrelated":
                        pass
                    else:
                        P = period
                        N = num_periods
                        if delta < 0.5:
                            run = slice(0, 99)
                        else:
                            run = slice(0, 999)
                        if integrate_angle:
                            th = f"{theta}-{integrate_angle}"
                        else:
                            th = theta
                        if mode == "raw":
                            fom = var[0]
                        elif mode == "ef":
                            fom = f"{var[0]}ef"
                        disorder_code = disorder_correlation[0]
                        D = str(delta).replace(".", "")

                        with warnings.catch_warnings():
                            warnings.filterwarnings("ignore", "divide")
                            the_plots = plot_singles(
                                var,
                                run,
                                disorder_correlation,
                                num_periods,
                                period,
                                delta,
                                theta,
                                mode=mode,
                                wavelength_int_range=wavelength_int_range,
                                integrate_angle=integrate_angle,
                            )

                            overlay_1 = (
                                hv.Overlay(the_plots[:3])
                                .select(run=slice(0, 8))
                                .opts(
                                    opts.Curve(ylim=(0, None)),
                                    opts.Scatter(alpha=1),
                                    clone=True,
                                )
                            )
                            # display(overlay_1)

                            overlay_2 = hv.Overlay(
                                [the_plots[2], *the_plots[5:], the_plots[1]]
                            ).opts(opts.Curve(xlim=(480, 880), ylim=(0, None)))
                            # display(overlay_2)

                            max_vs_integral = the_plots[3]
                            # display(max_vs_integral)

                            hv.save(
                                overlay_1,
                                filename=fig_path
                                / f"{fom}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

                            hv.save(
                                overlay_2,
                                filename=fig_path
                                / f"{fom}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

                            hv.save(
                                max_vs_integral,
                                filename=fig_path
                                / f"{fom}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

In [ ]:
var = ["R"]
elt = excitonic_layer_thickness = 50
mode = "raw"
wavelength_int_range = (None, None)
N = num_periods = 5
theta = 0
integrate_angle = None
delta = 0.2
disorder_correlation = "uncorrelated"

for period in [200, 250]:
    P = period
    if delta < 0.5:
        run = slice(0, 99)
    else:
        run = slice(0, 999)
    if integrate_angle:
        th = f"{theta}-{integrate_angle}"
    else:
        th = theta
    if mode == "raw":
        fom = var[0]
    elif mode == "ef":
        fom = f"{var[0]}ef"
    disorder_code = disorder_correlation[0]
    D = str(delta).replace(".", "")

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", "divide")
        the_plots = plot_singles(
            var,
            run,
            disorder_correlation,
            num_periods,
            period,
            delta,
            theta,
            mode=mode,
            wavelength_int_range=wavelength_int_range,
            integrate_angle=integrate_angle,
        )

        overlay_1 = (
            hv.Overlay(the_plots[:3])
            .select(run=slice(0, 8))
            .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
        )
        # display(overlay_1)

        overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
            opts.Curve(xlim=(480, 880), ylim=(0, None))
        )
        # display(overlay_2)

        max_vs_integral = the_plots[3]
        # display(max_vs_integral)

        hv.save(
            overlay_1,
            filename=fig_path
            / f"{fom}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
            fmt="png",
            toolbar=False,
        )

        hv.save(
            overlay_2,
            filename=fig_path
            / f"{fom}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
            fmt="png",
            toolbar=False,
        )

        hv.save(
            max_vs_integral,
            filename=fig_path
            / f"{fom}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
            fmt="png",
            toolbar=False,
        )

In [ ]:
var = ["R"]
elt = excitonic_layer_thickness = 50
mode = "raw"
wavelength_int_range = (None, None)
N = num_periods = 50
theta = 0
integrate_angle = None
delta = 0.4
disorder_correlation = "uncorrelated"

for period in [100, 150, 200, 250, 300]:
    P = period
    if delta < 0.5:
        run = slice(0, 99)
    else:
        run = slice(0, 999)
    if integrate_angle:
        th = f"{theta}-{integrate_angle}"
    else:
        th = theta
    if mode == "raw":
        fom = var[0]
    elif mode == "ef":
        fom = f"{var[0]}ef"
    disorder_code = disorder_correlation[0]
    D = str(delta).replace(".", "")

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", "divide")
        the_plots = plot_singles(
            var,
            run,
            disorder_correlation,
            num_periods,
            period,
            delta,
            theta,
            mode=mode,
            wavelength_int_range=wavelength_int_range,
            integrate_angle=integrate_angle,
        )

        overlay_1 = (
            hv.Overlay(the_plots[:3])
            .select(run=slice(0, 8))
            .opts(opts.Curve(ylim=(0, None)), opts.Scatter(alpha=1), clone=True)
        )
        # display(overlay_1)

        overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
            opts.Curve(xlim=(480, 880), ylim=(0, None))
        )
        # display(overlay_2)

        max_vs_integral = the_plots[3]
        # display(max_vs_integral)

        hv.save(
            overlay_1,
            filename=fig_path
            / f"{fom}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
            fmt="png",
            toolbar=False,
        )

        hv.save(
            overlay_2,
            filename=fig_path
            / f"{fom}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
            fmt="png",
            toolbar=False,
        )

        hv.save(
            max_vs_integral,
            filename=fig_path
            / f"{fom}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
            fmt="png",
            toolbar=False,
        )

### Reflectance difference factor to compare against absorptance

#### Normal incidence

In [ ]:
variable = "R"
N = num_periods = 5
wavelength = (480, 881)
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable, ylim=(0, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{theta}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
N = num_periods = 20
wavelength = (480, 881)
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable, ylim=(0, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{theta}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
N = num_periods = 50
wavelength = (480, 881)
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable, ylim=(0, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{theta}",
    fmt="png",
    toolbar=False,
)

#### Integrated to 45

In [ ]:
variable = "R"
N = num_periods = 5
wavelength = (480, 881)
theta = 0
integrate_angle = 45
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable, ylim=(0, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
N = num_periods = 20
wavelength = (480, 881)
theta = 0
integrate_angle = 45
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable, ylim=(0, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
N = num_periods = 50
wavelength = (480, 881)
theta = 0
integrate_angle = 45
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable, ylim=(0, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

#### Integrated to 86

In [ ]:
variable = "R"
N = num_periods = 5
wavelength = (480, 881)
theta = 0
integrate_angle = 86
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True), opts.Curve(ylabel=variable, ylim=(None, 0.1))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
N = num_periods = 20
wavelength = (480, 881)
theta = 0
integrate_angle = 86
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable, ylim=(0, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
N = num_periods = 50
wavelength = (480, 881)
theta = 0
integrate_angle = 86
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable, ylim=(0, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

### Plots over theta

#### 2D plots

In [ ]:
the_plot = integrate_da(
    restacked_stats_ds["R_mean"].sel(disorder_correlation="correlated").squeeze(),
    dim=["wavelength"],
    normalisation=1,
).hvplot(
    kind="quadmesh",
    x="delta",
    y="theta",
    row="period",
    col="num_periods",
)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0, 0.6)))
the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
# the_plot = (
#     restacked_stats_ds["R_mean"].sel(disorder_correlation="correlated", num_periods=20).squeeze()
# ).hvplot(
#     kind="quadmesh",
#     x="wavelength",
#     y="theta",
#     row="period",
#     col="delta",
# )

In [ ]:
# the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False))#, clim=(0, 0.6)))
# the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
# the_plot

Period doesn't make much difference to the picture, so focus on N and D.

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 300
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_stats_ds["R_mean"].sel(disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0, 1), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="white", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"R_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_avg", fmt="png", toolbar=False)

In [ ]:
deltas = [0.1, 0.3, 0.5, 1]
P = period = 300
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
run = 0
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_stats_ds["R_mean"].sel(disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0, 1), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="white", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"R_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_avg", fmt="png", toolbar=False)

Interesting. Singles?

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 300
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
run = 0
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_sds["R"].sel(run=run, disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0, 1), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="white", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"R_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_r{run}", fmt="png", toolbar=False)

In [ ]:
deltas = [0.1, 0.3, 0.5, 1]
P = period = 300
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
run = 0
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_sds["R"].sel(run=run, disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0, 1), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="white", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"R_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_r{run}", fmt="png", toolbar=False)

 #### 1D plots

In [ ]:
variable = "R"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 300
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot.opts(opts.Curve(ylim=(0,None)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0, 0.12)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
variable = "R"
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 300
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot.opts(opts.Curve(ylim=(0,None)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0, 0.18)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
variable = "R"
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 300
deltas = [0, 0.1, 0.3, 0.5]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot.opts(opts.Curve(ylim=(0,None)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0, 0.12)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
variable = "R"
disorder_correlation = "uncorrelated"
N = num_periods = 50
P = period = 300
deltas = [0, 0.1, 0.3, 0.5]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot.opts(opts.Curve(ylim=(0,None)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0, 0.18)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

Interesting behaviour at P=250.

In [ ]:
variable = "R"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 250
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot.opts(opts.Curve(ylim=(0,None)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0, 0.1)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
variable = "R"
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 250
deltas = [0, 0.1, 0.3, 0.5]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot.opts(opts.Curve(ylim=(0,None)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0, 0.1)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

What about P=200?

In [ ]:
variable = "R"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 200
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot.opts(opts.Curve(ylim=(0,None)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0, 0.07)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
variable = "R"
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 200
deltas = [0, 0.1, 0.3, 0.5]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot.opts(opts.Curve(ylim=(0,None)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0, 0.07)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

End of section

In [ ]:
the_plot = integrate_da(
    restacked_stats_ds["R_mean"].sel(disorder_correlation="correlated").sel(theta=slice(0, 45)).squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
).hvplot(
    kind="quadmesh",
    x="delta",
    y="num_periods",
    row="period",
    # col="disorder_correlation",
)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0.02, 0.135)))
the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
the_plot = integrate_da(
    restacked_stats_ds["R_mean"]
    .sel(disorder_correlation="uncorrelated")
    .sel(delta=slice(0, 0.5))
    .squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
).hvplot(
    kind="quadmesh",
    x="delta",
    y="num_periods",
    row="period",
    # col="disorder_correlation",
)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0.02, 0.125)))
the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
the_plot = integrate_da(
    restacked_stats_ds["R_mean"].sel(disorder_correlation="uncorrelated").sel(delta=slice(0,0.5), theta=slice(0, 45)).squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
).hvplot(
    kind="quadmesh",
    x="delta",
    y="num_periods",
    row="period",
    # col="disorder_correlation",
)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0.02, 0.135)))
the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

### Is reflectance near the LO resonance easier to disrupt?

In [ ]:
variable = "R"
periods = [200, 250, 300]
wavelength = slice(None, None)
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]

match disorder_correlation:
    case "uncorrelated":
        deltas = [0, 0.1, 0.3, 0.5]
    case "correlated":
        deltas = [0.1, 0.3, 0.5, 1]

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"


temp_da = cross_section(
    restacked_stats_ds.sel(period=periods, disorder_correlation=disorder_correlation, delta=deltas),
    variable=f"{variable}_mean",
    wavelength=wavelength,
    theta=theta,
    integrate_angle=integrate_angle,
).squeeze()

the_plot = temp_da.hvplot(
    kind="quadmesh", x="wavelength", y="num_periods", row="period", col="delta", cmap=cmap
)

In [ ]:
the_plot = the_plot.redim(period=period_dim)
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(None, None)))
the_plot[200].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}_DLOPC_2D_wl_N_grid_P_D{disorder_code}_elt{elt}_th{th}_avg", fmt="png", toolbar=False)

In [ ]:
variable = "R"
periods = [200, 250, 300]
wavelength = slice(None, None)
theta = None
integrate_angle = None
N = num_periods = 20
elt = excitonic_layer_thickness = 50
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]

match disorder_correlation:
    case "uncorrelated":
        deltas = [0, 0.1, 0.3, 0.5]
    case "correlated":
        deltas = [0.1, 0.3, 0.5, 1]

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"


temp_da = cross_section(
    restacked_stats_ds.sel(period=periods, disorder_correlation=disorder_correlation, delta=deltas, num_periods=num_periods),
    variable=f"{variable}_mean",
    wavelength=wavelength,
    theta=theta,
    integrate_angle=integrate_angle,
).squeeze()

the_plot = temp_da.hvplot(
    kind="quadmesh", x="wavelength", y="theta", row="period", col="delta", cmap=cmap
)

In [ ]:
the_plot = the_plot.redim(period=period_dim)
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(None, None)))
the_plot[200].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}_DLOPC_2D_wl_th_grid_P_D{disorder_code}_N{N}_elt{elt}_avg", fmt="png", toolbar=False)

In [ ]:
variable = "R"
N = num_periods = 5
wavelength = slice(None, None)
theta = 0
integrate_angle = None
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = cross_section(
    restacked_stats_ds.sel(num_periods=num_periods),
    variable = f"{variable}_mean",
    wavelength = wavelength,
    theta=theta,
    integrate_angle=integrate_angle,
).squeeze()

temp_stats_ds = temp_stats_ds.max("wavelength")

In [ ]:
the_plot = (
    temp_stats_ds.sel(disorder_correlation="correlated").hvplot.line(
        x="delta", by="period", color=color_cycle, line_dash="dashed"
    )
    * temp_stats_ds.sel(disorder_correlation="uncorrelated").hvplot.line(
        x="delta", by="period", color=color_cycle, line_dash="solid"
    )
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
            .squeeze()
            .max("wavelength")
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True),
    opts.Curve(ylabel=f"{variable}_max", ylim=(0, None)),
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}max_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
N = num_periods = 5
wavelength = slice(None, None)
theta = 0
integrate_angle = None
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = cross_section(
    restacked_stats_ds.sel(num_periods=num_periods),
    variable = f"{variable}_mean",
    wavelength = wavelength,
    theta=theta,
    integrate_angle=integrate_angle,
).squeeze()

# temp_stats_ds = temp_stats_ds.max("wavelength")

In [ ]:
the_plot = (
    temp_stats_ds.sel(disorder_correlation="correlated").hvplot.line(
        x="wavelength", by="delta", row="num_periods", col="period", color=color_cycle, line_dash="dashed"
    )
    * temp_stats_ds.sel(disorder_correlation="uncorrelated").hvplot.line(
        x="wavelength", by="delta", row="num_periods", col="period", color=color_cycle, line_dash="solid"
    )
)
the_plot.opts(
    opts.Curve(ylabel=f"{variable}_max", ylim=(0, None), frame_width=400),
)
the_plot

In [ ]:
# this is not at all the right filename
# hv.save(
#     the_plot,
#     filename=fig_path
#     / f"{variable}max_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
#     fmt="png",
#     toolbar=False,
# )

## Does disorder make absorptance go up or down?

### Big loop

In [ ]:
var = ["A"]
theta = 0
elt = excitonic_layer_thickness = 50
mode = "ef"
wavelength_int_range = (None, None)

for period in tqdm([100, 150, 200, 250, 300]):
    for num_periods in [5, 20, 50]:
        for integrate_angle in [None, 45, 86]:
            for delta in [0.1, 0.3, 0.5, 1]:
                for disorder_correlation in ["correlated", "uncorrelated"]:
                    if delta == 1 and disorder_correlation == "uncorrelated":
                        pass
                    else:
                        P = period
                        N = num_periods
                        if delta < 0.5:
                            run = slice(0, 99)
                        else:
                            run = slice(0, 999)
                        if integrate_angle:
                            th = f"{theta}-{integrate_angle}"
                        else:
                            th = theta
                        if mode == "raw":
                            fom = var[0]
                        elif mode == "ef":
                            fom = f"{var[0]}ef"
                        disorder_code = disorder_correlation[0]
                        D = str(delta).replace(".", "")

                        with warnings.catch_warnings():
                            warnings.filterwarnings("ignore", "invalid value encountered in divide")
                            warnings.simplefilter("ignore")
                            the_plots = plot_singles(
                                var,
                                run,
                                disorder_correlation,
                                num_periods,
                                period,
                                delta,
                                theta,
                                mode=mode,
                                wavelength_int_range=wavelength_int_range,
                                integrate_angle=integrate_angle,
                            )

                            overlay_1 = (
                                hv.Overlay(the_plots[:3])
                                .select(run=slice(0, 8))
                                .opts(opts.Scatter(alpha=1), clone=True)
                            )
                            # display(overlay_1)

                            overlay_2 = hv.Overlay(
                                [the_plots[2], *the_plots[5:], the_plots[1]]
                            ).opts(opts.Curve(xlim=(480, 880), ylim=(None, None)))
                            # display(overlay_2)

                            max_vs_integral = the_plots[3]
                            # display(max_vs_integral)

                            hv.save(
                                overlay_1,
                                filename=fig_path
                                / f"{fom}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

                            hv.save(
                                overlay_2,
                                filename=fig_path
                                / f"{fom}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

                            hv.save(
                                max_vs_integral,
                                filename=fig_path
                                / f"{fom}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

Extras for particular figures I want to make.

In [ ]:
var = ["A"]
theta = 0
elt = excitonic_layer_thickness = 50
mode = "ef"
wavelength_int_range = (None, None)

for period in tqdm([200, 250]):
    for num_periods in [50]:
        for integrate_angle in [None]:
            for delta in [0.4]:
                for disorder_correlation in ["uncorrelated"]:
                    if delta == 1 and disorder_correlation == "uncorrelated":
                        pass
                    else:
                        P = period
                        N = num_periods
                        if delta < 0.5:
                            run = slice(0, 99)
                        else:
                            run = slice(0, 999)
                        if integrate_angle:
                            th = f"{theta}-{integrate_angle}"
                        else:
                            th = theta
                        if mode == "raw":
                            fom = var[0]
                        elif mode == "ef":
                            fom = f"{var[0]}ef"
                        disorder_code = disorder_correlation[0]
                        D = str(delta).replace(".", "")

                        with warnings.catch_warnings():
                            warnings.filterwarnings("ignore", "invalid value encountered in divide")
                            warnings.simplefilter("ignore")
                            the_plots = plot_singles(
                                var,
                                run,
                                disorder_correlation,
                                num_periods,
                                period,
                                delta,
                                theta,
                                mode=mode,
                                wavelength_int_range=wavelength_int_range,
                                integrate_angle=integrate_angle,
                            )

                            overlay_1 = (
                                hv.Overlay(the_plots[:3])
                                .select(run=slice(0, 8))
                                .opts(opts.Scatter(alpha=1), clone=True)
                            )
                            # display(overlay_1)

                            overlay_2 = hv.Overlay(
                                [the_plots[2], *the_plots[5:], the_plots[1]]
                            ).opts(opts.Curve(xlim=(480, 880), ylim=(None, None)))
                            # display(overlay_2)

                            max_vs_integral = the_plots[3]
                            # display(max_vs_integral)

                            hv.save(
                                overlay_1,
                                filename=fig_path
                                / f"{fom}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

                            hv.save(
                                overlay_2,
                                filename=fig_path
                                / f"{fom}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

                            hv.save(
                                max_vs_integral,
                                filename=fig_path
                                / f"{fom}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

Absorptance (not AEF) to demonstrate that it's not very informative to look at.

In [ ]:
var = ["A"]
theta = 0
elt = excitonic_layer_thickness = 50
mode = "raw"
wavelength_int_range = (None, None)

for period in tqdm([200]):
    for num_periods in [5, 20, 50]:
        for integrate_angle in [None]: #[None, 45, 86]:
            for delta in [0.3, 0.5, 1]:
                for disorder_correlation in ["correlated", "uncorrelated"]:
                    if delta == 1 and disorder_correlation == "uncorrelated":
                        pass
                    else:
                        P = period
                        N = num_periods
                        if delta < 0.5:
                            run = slice(0, 99)
                        else:
                            run = slice(0, 999)
                        if integrate_angle:
                            th = f"{theta}-{integrate_angle}"
                        else:
                            th = theta
                        if mode == "raw":
                            fom = var[0]
                        elif mode == "ef":
                            fom = f"{var[0]}ef"
                        disorder_code = disorder_correlation[0]
                        D = str(delta).replace(".", "")

                        with warnings.catch_warnings():
                            warnings.filterwarnings("ignore", "invalid value encountered in divide")
                            warnings.simplefilter("ignore")
                            the_plots = plot_singles(
                                var,
                                run,
                                disorder_correlation,
                                num_periods,
                                period,
                                delta,
                                theta,
                                mode=mode,
                                wavelength_int_range=wavelength_int_range,
                                integrate_angle=integrate_angle,
                            )

                            overlay_1 = (
                                hv.Overlay(the_plots[:3])
                                .select(run=slice(0, 8))
                                .opts(opts.Scatter(alpha=1), clone=True)
                            )
                            # display(overlay_1)

                            overlay_2 = hv.Overlay(
                                [the_plots[2], *the_plots[5:], the_plots[1]]
                            ).opts(opts.Curve(xlim=(480, 880), ylim=(None, None)))
                            # display(overlay_2)

                            max_vs_integral = the_plots[3]
                            # display(max_vs_integral)

                            hv.save(
                                overlay_1,
                                filename=fig_path
                                / f"{fom}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

                            hv.save(
                                overlay_2,
                                filename=fig_path
                                / f"{fom}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

                            hv.save(
                                max_vs_integral,
                                filename=fig_path
                                / f"{fom}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{th}_D{D}{disorder_code}",
                                fmt="png",
                                toolbar=False,
                            )

### Other stuff

In [ ]:
var = ["A"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 100
delta = 0.5
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "ef"
wavelength_int_range = (None, None)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var,
        run,
        disorder_correlation,
        num_periods,
        period,
        delta,
        theta,
        mode=mode,
        wavelength_int_range=wavelength_int_range,
        integrate_angle=integrate_angle,
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

    overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
        opts.Curve(ylim=(None, None))  # I don't understand why this is needed but otherwise the ylim defaults to (0, 1)
    )
    display(overlay_2)

    max_vs_integral = the_plots[3]
    display(max_vs_integral)

    violin_plot = the_plots[4]
    display(violin_plot)

In [ ]:
var = ["A"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 200
delta = 0.5
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "ef"
wavelength_int_range = (None, None)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var,
        run,
        disorder_correlation,
        num_periods,
        period,
        delta,
        theta,
        mode=mode,
        wavelength_int_range=wavelength_int_range,
        integrate_angle=integrate_angle,
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

    overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
        opts.Curve(ylim=(None, None))  # I don't understand why this is needed but otherwise the ylim defaults to (0, 1)
    )
    display(overlay_2)

    max_vs_integral = the_plots[3]
    display(max_vs_integral)

    violin_plot = the_plots[4]
    display(violin_plot)

In [ ]:
var = ["A"]
run = slice(0, 2)
disorder_correlation = "correlated"
N = num_periods = 50
P = period = 100
delta = 1
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "ef"
wavelength_int_range = (None, None)
match mode:
    case "ef":
        var_code = var[0] + "ef"
    case _:
        var_code = var[0]

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var,
        run,
        disorder_correlation,
        num_periods,
        period,
        delta,
        theta,
        mode=mode,
        wavelength_int_range=wavelength_int_range,
        integrate_angle=integrate_angle,
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

    overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
        opts.Curve(ylim=(None, None))  # I don't understand why this is needed but otherwise the ylim defaults to (0, 1)
    )
    display(overlay_2)

    max_vs_integral = the_plots[3]
    display(max_vs_integral)

    violin_plot = the_plots[4]
    display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=slice(0,2))
display(reduced_overlay)

In [ ]:
hv.save(
    reduced_overlay,
    filename=fig_path
    / f"{var_code}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
reduced_overlay = overlay_1.select(run=0)

In [ ]:
reduced_overlay.NdOverlay.I.opts(opts.Curve(color=red, alpha=1))

In [ ]:
temp_curve = reduced_overlay.NdOverlay.I.data

In [ ]:
peaks = find_peaks(temp_curve[var[0]])[0]

In [ ]:
wls = temp_curve.loc[peaks].wavelength.values

In [ ]:
vals = temp_curve.loc[peaks][var[0]].values

In [ ]:
arrows = [hv.Arrow(wl, val, f"{i+1}", 'v') for i, (wl, val) in enumerate(zip(wls, vals))]

In [ ]:
the_plot = (reduced_overlay*hv.Overlay(arrows)).opts(opts.Curve(ylim=(None, 1.17)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}_annotated",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["A"]
run = slice(0, 2)
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 100
delta = 1
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "ef"
wavelength_int_range = (None, None)
match mode:
    case "ef":
        var_code = var[0] + "ef"
    case _:
        var_code = var[0]

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var,
        run,
        disorder_correlation,
        num_periods,
        period,
        delta,
        theta,
        mode=mode,
        wavelength_int_range=wavelength_int_range,
        integrate_angle=integrate_angle,
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

    overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
        opts.Curve(ylim=(None, None))  # I don't understand why this is needed but otherwise the ylim defaults to (0, 1)
    )
    display(overlay_2)

    max_vs_integral = the_plots[3]
    display(max_vs_integral)

    violin_plot = the_plots[4]
    display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=1)

In [ ]:
reduced_overlay.NdOverlay.I.opts(opts.Curve(color=red, alpha=1))

In [ ]:
temp_curve = reduced_overlay.NdOverlay.I.data.reset_index()

In [ ]:
peaks = find_peaks(temp_curve[var[0]])[0]

In [ ]:
wls = temp_curve.loc[peaks].wavelength.values

In [ ]:
vals = temp_curve.loc[peaks][var[0]].values

In [ ]:
arrows = [hv.Arrow(wl, val, f"{i+1}", 'v') for i, (wl, val) in enumerate(zip(wls, vals))]

In [ ]:
the_plot = (reduced_overlay*hv.Overlay(arrows)).opts(opts.Curve(ylim=(None, 1.13)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}_annotated",
    fmt="png",
    toolbar=False,
)

In [ ]:
var = ["A"]
run = slice(0, 5)
disorder_correlation = "uncorrelated"
N = num_periods = 20
P = period = 100
delta = 0.5
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "ef"
wavelength_int_range = (None, None)
match mode:
    case "ef":
        var_code = var[0] + "ef"
    case _:
        var_code = var[0]

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var,
        run,
        disorder_correlation,
        num_periods,
        period,
        delta,
        theta,
        mode=mode,
        wavelength_int_range=wavelength_int_range,
        integrate_angle=integrate_angle,
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

    overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
        opts.Curve(ylim=(None, None))  # I don't understand why this is needed but otherwise the ylim defaults to (0, 1)
    )
    display(overlay_2)

    max_vs_integral = the_plots[3]
    display(max_vs_integral)

    violin_plot = the_plots[4]
    display(violin_plot)

In [ ]:
reduced_overlay = overlay_1.select(run=5)

In [ ]:
reduced_overlay.NdOverlay.I.opts(opts.Curve(color=red, alpha=1))

In [ ]:
temp_curve = reduced_overlay.NdOverlay.I.data.reset_index()

In [ ]:
peaks = find_peaks(temp_curve[var[0]], threshold=)[0]

In [ ]:
wls = temp_curve.loc[peaks].wavelength.values

In [ ]:
vals = temp_curve.loc[peaks][var[0]].values

In [ ]:
arrows = [hv.Arrow(wl, val, f"{i+1}", 'v') for i, (wl, val) in enumerate(zip(wls, vals))]

In [ ]:
the_plot = (reduced_overlay*hv.Overlay(arrows)).opts(opts.Curve(ylim=(None, 1.13)))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code}_DLOPC_runcurve_reduced_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}_annotated",
    fmt="png",
    toolbar=False,
)

### Plots by period

#### Loop

In [ ]:
period_legend_dim = hv.Dimension('period', label='Λ (nm)')

In [ ]:
wavelength = (None, None)
theta = 0
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

for variable in tqdm(["R", "A"]):
    for num_periods in [5, 20, 50]:
        for integrate_angle in [None, 45, 86]:
            N = num_periods
            if integrate_angle:
                th = f"{theta}-{integrate_angle}"
            else:
                th = theta
            temp_stats_ds = (
                restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
            )
            the_plot = (
                cross_section(
                    temp_stats_ds.sel(disorder_correlation="correlated"),
                    wavelength=wavelength,
                    theta=theta,
                    integrate_angle=integrate_angle,
                )
                .hvplot.line(
                    x="delta", by="period", color=color_cycle, line_dash="dashed", legend=False
                )
                .redim(period=period_legend_dim)
                * cross_section(
                    temp_stats_ds.sel(disorder_correlation="uncorrelated"),
                    wavelength=wavelength,
                    theta=theta,
                    integrate_angle=integrate_angle,
                )
                .hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
                .redim(period=period_legend_dim)
                * hv.HLine(
                    float(
                        cross_section(
                            crs_1[variable].sel(
                                total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                            ),
                            wavelength=wavelength,
                            theta=theta,
                            integrate_angle=integrate_angle,
                        )
                    )
                )
            )
            the_plot.opts(
                opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True),
                opts.Curve(ylabel=f"{variable}_tot"),
                opts.Overlay(legend_opts={"title": "Λ (nm)"}),
            )
            # display(the_plot)

            hv.save(
                the_plot,
                filename=fig_path
                / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
                fmt="png",
                toolbar=False,
            )

#### Normal incidence

In [ ]:
variable = "A"
N = num_periods = 5
wavelength = (None, None)
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    )
    .hvplot.line(
        x="delta", by="period", color=color_cycle, line_dash="dashed", legend=False
    )
    .redim(period=period_legend_dim)
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    )
    .hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    .redim(period=period_legend_dim)
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True),
    opts.Curve(ylabel=f"{variable}_tot"),
    opts.Overlay(legend_opts={"title": "Λ (nm)"}),
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{theta}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
N = num_periods = 20
wavelength = (480, 881)
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable)
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{theta}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
N = num_periods = 50
wavelength = (480, 881)
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable)
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{theta}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
N = num_periods = 5
wavelength = slice(None, None)
theta = 0
integrate_angle = None
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = cross_section(
    restacked_stats_ds.sel(num_periods=num_periods),
    variable = f"{variable}_mean",
    wavelength = wavelength,
    theta=theta,
    integrate_angle=integrate_angle,
).squeeze()

temp_stats_ds = temp_stats_ds.max("wavelength")

In [ ]:
the_plot = (
    temp_stats_ds.sel(disorder_correlation="correlated").hvplot.line(
        x="delta", by="period", color=color_cycle, line_dash="dashed"
    )
    * temp_stats_ds.sel(disorder_correlation="uncorrelated").hvplot.line(
        x="delta", by="period", color=color_cycle, line_dash="solid"
    )
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
            .squeeze()
            .max("wavelength")
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True),
    opts.Curve(ylabel=f"{variable}_max", ylim=(None, None)),
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}max_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

#### Integrated to 45

In [ ]:
variable = "A"
N = num_periods = 5
wavelength = (480, 881)
theta = 0
integrate_angle = 45
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True), opts.Curve(ylabel=variable, ylim=(None, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
N = num_periods = 20
wavelength = (480, 881)
theta = 0
integrate_angle = 45
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True), opts.Curve(ylabel=variable, ylim=(None, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
N = num_periods = 50
wavelength = (480, 881)
theta = 0
integrate_angle = 45
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True), opts.Curve(ylabel=variable, ylim=(None, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

#### Integrated to 86

In [ ]:
variable = "A"
N = num_periods = 5
wavelength = (480, 881)
theta = 0
integrate_angle = 86
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True), opts.Curve(ylabel=variable, ylim=(0.14, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
N = num_periods = 20
wavelength = (480, 881)
theta = 0
integrate_angle = 86
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True), opts.Curve(ylabel=variable, ylim=(0.33, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
N = num_periods = 50
wavelength = (480, 881)
theta = 0
integrate_angle = 86
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted", apply_ranges=True), opts.Curve(ylabel=variable, ylim=(0.485, None))
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

#### What is going on at P=200, D=1?

In [ ]:
var = ["A"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 200
delta = 1
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "ef"
wavelength_int_range = (None, None)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var,
        run,
        disorder_correlation,
        num_periods,
        period,
        delta,
        theta,
        mode=mode,
        wavelength_int_range=wavelength_int_range,
        integrate_angle=integrate_angle,
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

    overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
        opts.Curve(ylim=(None, None))  # I don't understand why this is needed but otherwise the ylim defaults to (0, 1)
    )
    display(overlay_2)

    max_vs_integral = the_plots[3]
    display(max_vs_integral)

    violin_plot = the_plots[4]
    display(violin_plot)

In [ ]:
var = ["A"]
run = slice(0, 999)
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 250
delta = 1
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")
mode = "ef"
wavelength_int_range = (None, None)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide")
    the_plots = plot_singles(
        var,
        run,
        disorder_correlation,
        num_periods,
        period,
        delta,
        theta,
        mode=mode,
        wavelength_int_range=wavelength_int_range,
        integrate_angle=integrate_angle,
    )

    overlay_1 = (
        hv.Overlay(the_plots[:3])
        .select(run=slice(0, 8))
        .opts(opts.Scatter(alpha=1), clone=True)
    )
    display(overlay_1)

    overlay_2 = hv.Overlay([the_plots[2], *the_plots[5:], the_plots[1]]).opts(
        opts.Curve(ylim=(None, None))  # I don't understand why this is needed but otherwise the ylim defaults to (0, 1)
    )
    display(overlay_2)

    max_vs_integral = the_plots[3]
    display(max_vs_integral)

    violin_plot = the_plots[4]
    display(violin_plot)

In [ ]:
hv.save(
    overlay_1,
    filename=fig_path
    / f"{var[0]}_DLOPC_runcurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    overlay_2,
    filename=fig_path
    / f"{var[0]}_DLOPC_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

hv.save(
    max_vs_integral,
    filename=fig_path
    / f"{var[0]}_DLOPC_max_vs_tot_P{P}_elt{elt}_N{N}_th{theta}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

# have no use for the violin plots yet

In [ ]:
variable = "A"
N = num_periods = 50
wavelength = (480, 881)
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
color_cycle = default_color_cycle[len(restacked_stats_ds.period)]

temp_stats_ds = (
    restacked_stats_ds[f"{variable}_mean"].sel(num_periods=num_periods).squeeze()
)
the_plot = (
    cross_section(
        temp_stats_ds.sel(disorder_correlation="correlated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="dashed")
    * cross_section(
        temp_stats_ds.sel(disorder_correlation="uncorrelated"),
        wavelength=wavelength,
        theta=theta,
        integrate_angle=integrate_angle,
    ).hvplot.line(x="delta", by="period", color=color_cycle, line_dash="solid")
    * hv.HLine(
        float(
            cross_section(
                crs_1[variable].sel(
                    total_excitonic_thickness=num_periods * excitonic_layer_thickness,
                ),
                wavelength=wavelength,
                theta=theta,
                integrate_angle=integrate_angle,
            )
        )
    )
)
the_plot.opts(
    opts.HLine(line_color="black", line_dash="dotted"), opts.Curve(ylabel=variable)
)
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{variable}tot_DLOPC_period_comparison_D_elt{elt}_N{N}_th{theta}",
    fmt="png",
    toolbar=False,
)

In [ ]:
the_plot = (
    restacked_ief(
        ds=stats_ds,
        ref=crs_1_stats_norm,
        common_dim="total_excitonic_thickness",
        method="vectorized_indexing",
        variable="A_mean",
        wavelength=(480, 881),
        integrate_angle=86,
    )
    .sel(disorder_correlation="correlated")
    .squeeze()
    .hvplot(
        kind="quadmesh",
        x="delta",
        y="num_periods",
        row="period",
        # col="disorder_correlation",
    )
)

In [ ]:
the_plot.opts(opts.QuadMesh(colorbar=False, cmap="RdBu_r", clim=(0.75, 1.25)))
the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
the_plot = (
    restacked_ief(
        ds=stats_ds,
        ref=crs_1_stats_norm,
        common_dim="total_excitonic_thickness",
        method="vectorized_indexing",
        variable="A_mean",
        wavelength=(480, 881),
        integrate_angle=45,
    )
    .sel(disorder_correlation="correlated")
    .squeeze()
    .hvplot(
        kind="quadmesh",
        x="delta",
        y="num_periods",
        row="period",
        # col="disorder_correlation",
    )
)

In [ ]:
the_plot.opts(opts.QuadMesh(colorbar=False, cmap="RdBu_r", clim=(0.9, 1.1)))
the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
the_plot = (
    restacked_ief(
        ds=stats_ds,
        ref=crs_1_stats_norm,
        common_dim="total_excitonic_thickness",
        method="vectorized_indexing",
        variable="A_mean",
        wavelength=(480, 881),
        integrate_angle=86,
    )
    .sel(disorder_correlation="uncorrelated")
    .sel(delta=slice(0, 0.5))
    .squeeze()
    .hvplot(
        kind="quadmesh",
        x="delta",
        y="num_periods",
        row="period",
        # col="disorder_correlation",
    )
)

In [ ]:
the_plot.opts(opts.QuadMesh(colorbar=False, cmap="RdBu_r", clim=(0.75, 1.25)))
the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
the_plot = (
    restacked_ief(
        ds=stats_ds,
        ref=crs_1_stats_norm,
        common_dim="total_excitonic_thickness",
        method="vectorized_indexing",
        variable="A_mean",
        wavelength=(480, 881),
        integrate_angle=45,
    )
    .sel(disorder_correlation="uncorrelated")
    .sel(delta=slice(0, 0.5))
    .squeeze()
    .hvplot(
        kind="quadmesh",
        x="delta",
        y="num_periods",
        row="period",
        # col="disorder_correlation",
    )
)

In [ ]:
the_plot.opts(opts.QuadMesh(colorbar=False, cmap="RdBu_r", clim=(0.9, 1.1)))
the_plot[100].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

### Plots over N

#### Absorptance enhancement

Testing

In [ ]:
deltas = [1]
P = period = 100
theta = 0
variable = "A"
disorder_correlation = "correlated"
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_1 = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    wavelength=(None, None),
    theta=theta,
    # kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
    include_std=True,
)
the_plot_1.opts(opts.HLine(apply_ranges=False))

In [ ]:
deltas = [0, 1]
P = period = 100
theta = 0
variable = "A"
disorder_correlation = "correlated"
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_1 = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    wavelength=(None, None),
    theta=theta,
    # kind="line",
    x="num_periods",
    by="delta",
    mode="raw",
    include_std=True,
)
the_plot_1.opts(opts.HLine(apply_ranges=False))

##### Mean

P=100

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 100
theta = 0
variable = "A"
disorder_correlation = "correlated"
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_1 = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    wavelength=(None, None),
    theta=theta,
    kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
)
the_plot_1.opts(opts.HLine(apply_ranges=False))

In [ ]:
hv.save(
    the_plot_1,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_Dc",
    fmt="png",
    toolbar=False,
)

P=200

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 200
theta = 0
variable = "A"
disorder_correlation = "correlated"
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_2 = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=theta,
    kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
)
the_plot_2

In [ ]:
hv.save(
    the_plot_2,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_Dc",
    fmt="png",
    toolbar=False,
)

P=300

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 300
theta = 0
variable = "A"
disorder_correlation = "correlated"
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_3 = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=theta,
    kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
)
the_plot_3

In [ ]:
hv.save(
    the_plot_3,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_Dc",
    fmt="png",
    toolbar=False,
)

All our ~powers~ periods combined

In [ ]:
the_big_plot = (
    hv.NdOverlay(
        {
            (100, 0): the_plot_1.select(delta=0),
            (100, 1): the_plot_1.select(delta=1),
            (200, 0): the_plot_2.select(delta=0),
            (200, 1): the_plot_2.select(delta=1),
            (300, 0): the_plot_3.select(delta=0),
            (300, 1): the_plot_3.select(delta=1),
        },
        kdims=[period_dim, delta_dim],
    )
)

In [ ]:
the_big_plot[:,0].opts(opts.Curve(line_dash="solid"))
the_big_plot[:,1].opts(opts.Curve(line_dash="dashed"))

the_big_plot[100,:].opts(opts.Curve(color=default_color_cycle.values[0]))
the_big_plot[200,:].opts(opts.Curve(color=default_color_cycle.values[1]))
the_big_plot[300,:].opts(opts.Curve(color=default_color_cycle.values[2]))

the_big_plot.opts(opts.NdOverlay(legend_position="right", legend_opts = {"title": ""}))

In [ ]:
hv.save(
    the_big_plot,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_period_delta_comparison_N_P_100-300_elt{elt}_th{theta}_D0-1c",
    fmt="png",
    toolbar=False,
)

### Plots over theta

#### 2D plots

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 300
variable = "A"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_norm_1_stats(stats_ds)[f"{variable}_mean"].sel(disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="RdBu_r", colorbar=False, clim=(0, 2), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="black", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_avg", fmt="png", toolbar=False)

In [ ]:
deltas = [0.1, 0.3, 0.5, 1]
P = period = 300
variable = "A"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_norm_1_stats(stats_ds)[f"{variable}_mean"].sel(disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="RdBu_r", colorbar=False, clim=(0, 2), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="black", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_avg", fmt="png", toolbar=False)

Interesting. Singles?

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 300
variable = "A"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
run = 0
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_norm_1(singles_ds)[f"{variable}"].sel(run=run, disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="RdBu_r", colorbar=False, clim=(0, 2), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="black", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_r{run}", fmt="png", toolbar=False)

In [ ]:
deltas = [0.1, 0.3, 0.5, 1]
P = period = 300
variable = "A"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
run = 0
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_norm_1(singles_ds)[f"{variable}"].sel(run=run, disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="RdBu_r", colorbar=False, clim=(0, 2), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="black", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_r{run}", fmt="png", toolbar=False)

In [ ]:
deltas = [0.1, 0.3, 0.5, 1]
P = period = 300
variable = "A"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
run = 0
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_sds[f"{variable}"].sel(run=run, disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="inferno", colorbar=False, clim=(0, 1), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="black", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_r{run}", fmt="png", toolbar=False)

Why is the AEF at high angles so much lower for P=100 than P=200-300?

In [ ]:
deltas = [0.1, 0.3, 0.5, 1]
P = period = 100
variable = "A"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_norm_1_stats(stats_ds)[f"{variable}_mean"].sel(disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="RdBu_r", colorbar=False, clim=(0, 2), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="black", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_avg", fmt="png", toolbar=False)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5]
P = period = 100
variable = "R"
disorder_correlation = "uncorrelated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_stats_ds["R_mean"].sel(disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0, 1), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="white", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"R_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_avg", fmt="png", toolbar=False)

In [ ]:
deltas = [0.1, 0.3, 0.5, 1]
P = period = 100
variable = "R"
disorder_correlation = "correlated"
excitonic_layer_thickness = 50
elt = excitonic_layer_thickness = 50
num_periods = [5, 10, 20, 50]
run = 0
disorder_code = disorder_correlation[0]

the_plot = (
    restacked_stats_ds["R_mean"].sel(disorder_correlation=disorder_correlation, period=period, num_periods=num_periods, delta=deltas).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="num_periods",
    col="delta",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0, 1), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="white", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"R_DLOPC_2D_wl_th_grid_N_D{disorder_code}_P{P}_elt{elt}_avg", fmt="png", toolbar=False)

Ah, it's because d_e=50 is pretty chunky compared to P=100.

In [ ]:
plts = [0, 50]
variable = "R"
disorder_correlation = "correlated"
elts = [10, 20, 30, 50]
num_periods = 50

the_plot = (
    periodic_ds[variable].sel(passive_layer_thickness=plts, num_periods=num_periods, excitonic_layer_thickness=elts).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="excitonic_layer_thickness",
    col="passive_layer_thickness",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0, 1), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="white", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
period = 100
variable = "R"
disorder_correlation = "correlated"
elts = [10, 20, 30, 50]
num_periods = 50

the_plot = (
    restacked_pds[variable].sel(period=period, num_periods=num_periods, excitonic_layer_thickness=elts).squeeze()
).hvplot(
    kind="quadmesh",
    x="wavelength",
    y="theta",
    row="excitonic_layer_thickness",
) * lorentz_vlines(0, scale=1e-9)

In [ ]:
the_plot = the_plot.redim(excitonic_layer_thickness=elt_dim)
the_plot.opts(opts.QuadMesh(cmap="viridis", colorbar=False, clim=(0, 1), xticks=2, yticks=[0, 40, 80]), opts.VLine(color="white", line_dash="dotted"))
the_plot[5].opts(opts.QuadMesh(colorbar=True, colorbar_position="left"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"R_LOPC_lambda_theta_2D_elt_grid", fmt="png", toolbar=None)

In [ ]:
period = 100
excitonic_layer_thickness = 50
passive_layer_thickness = period - excitonic_layer_thickness
num_periods = 20

In [ ]:
plots = OrderedDict({
    var: plot_R(
        variable=var,
        dataset=restacked_ds.sel(
            period=period,
            excitonic_layer_thickness=excitonic_layer_thickness,
            num_periods=num_periods,
        ).squeeze(),
        x="wavelength",
        y="theta",
    )
    for var in ["R", "Rs", "Rp"]
})

In [ ]:
grid = hv.GridSpace(plots, kdims=["Reflectance"], sort=False)*lorentz_vlines(0, scale=1e-9)

In [ ]:
grid.opts(opts.Image(colorbar=False))
grid["R"].opts(opts.Image(colorbar=True, colorbar_position="left"))

grid.opts(opts.Image(xticks=3, frame_height=120, frame_width=160), opts.VLine(line_dash="dotted", color="white"), opts.GridSpace(fontsize={"xlabel": 8}, toolbar="below"))

In [ ]:
# hv.save(grid, filename=fig_path/"R_lambda_theta_2D", fmt="png", toolbar=None)

In [ ]:
excitonic_layer_thickness = 30
theta = 0

In [ ]:
coords = {"excitonic_layer_thickness": excitonic_layer_thickness, "theta": theta}

In [ ]:
curves = {(P,N): restacked_ds[unpolarised_RTA].sel(num_periods=N, period=P, **coords).squeeze().hvplot.line(x="wavelength")*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted') for N in [50, 20, 10] for P in [200, 250, 300]}

In [ ]:
grid = hv.GridSpace(curves, kdims=[period_dim, "Number of periods"])

In [ ]:
# hacky workaround to bugs in hvplot/holoviews
for p in grid:
    p.opts(
        opts.Overlay(
            show_legend=False,
            show_grid=True,
        ),
        opts.Curve(
            color=hv.Cycle([blue, yellow, red]),  # set R/T/A->blue/yellow/red
            frame_width=200,
        ),
    )  # remove the excess legends

# grid[200, 50].opts(
#     opts.Overlay(show_legend=True, legend_position="left")
# )  # add the legend back in on a free side but this breaks the left alignment somehow

grid.opts(opts.GridSpace(show_legend=False, shared_xaxis=True, shared_yaxis=True, toolbar=None))

This is the code I would have liked to have used to save this image.

Unfortunately the `toolbar=False` argument doesn't work so I have to actually *use* the toolbar above to save it!

... Except this ALSO doesn't work, because the save button saves 9 individual images?! So I guess I have to use the below code and manually crop or something stupid.

In [ ]:
# hv.save(grid, filename=fig_path/f"N_Lambda_grid_2", fmt="png", toolbar=None)

#### 1D plots

##### Absorptance

(P=300)

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 300
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

(P=200)

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 200
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

(P=100)

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 100
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta"
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

##### Absorptance enhancement

(P=300) When the periodic structure would cause absorption suppression, disorder can turn that into enhancement...

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 300
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta",
    mode="ef",
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0.9, 1.1)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

(P=200) What about when it would cause enhancement?

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 200
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta",
    mode="ef",
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0.98, 1.1)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

(P=250) What about when we're on the LO resonance?

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 250
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta",
    mode="ef",
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(0.88, 1.1)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

(P=100) What about when we're far from the LO resonance?

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 100
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

the_plot = compare_integrated_stats(
    variable=variable,
    disorder_correlation=disorder_correlation,
    num_periods=num_periods,
    period=period,
    delta=deltas,
    wavelength=(480, 881),
    theta=None,
    kind="line",
    x="theta",
    by="delta",
    mode="ef",
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

### Optimal absorptance enhancement

#### Tests

It seems that disorder strictly reduces the optimum for my model...

In [ ]:
temp_da = (
    restacked_ief(
        stats_ds, ref=crs_1_stats_norm, common_dim="total_excitonic_thickness", wavelength=(None, None), lorentz_line=0, window_radius=1, integrate_angle=86,
    )["A_mean"]
    .sel(disorder_correlation="uncorrelated")
    .sel(delta=slice(0, 0.5))
    .squeeze()
)

the_plot = plot_optimum_over_dim(da=temp_da, x="delta", y="num_periods", dim="period")[
    0
].opts(opts.QuadMesh(cmap="inferno", clim=(1, None)))

In [ ]:
the_plot

In [ ]:
variable = "A"
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_da = (
    restacked_ief(
        stats_ds,
        ref=crs_1_stats_norm,
        common_dim="total_excitonic_thickness",
        wavelength=(None, None),
        # lorentz_line=0,
        # window_radius=5,
        integrate_angle=integrate_angle,
    )[f"{variable}_mean"]
    .sel(delta=slice(0, 1))
    .squeeze()
)

the_plot = plot_optimum_over_dim(
    da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
)[0].opts(opts.QuadMesh(cmap="inferno", clim=(1, None)), opts.Points(color=red))

the_plot

In [ ]:
restacked_

In [ ]:
(
    restacked_pds["R"]
    .sel(period=200, excitonic_layer_thickness=50, theta=0, num_periods=[5, 20, 50])
    .squeeze()
    .hvplot(x="wavelength", by="num_periods")
    * lorentz_vlines(-9, scale=1e-9)
)

In [ ]:
# find the lorentz line corresponding to a peak

temp_da = (
    restacked_ief(
        stats_ds,
        ref=crs_1_stats_norm,
        common_dim="total_excitonic_thickness",
        wavelength=slice(None, None),
        integrate_angle=integrate_angle,
    )[f"{variable}_mean"]
    .sel(delta=slice(0, 1))
    .squeeze()
)

(
    temp_da.sel(
        delta=0, disorder_correlation="correlated", period=200, num_periods=[5, 20, 50]
    ).hvplot(x="wavelength", by="num_periods")
    * lorentz_vlines(-7.5, scale=1e-9)
)

In [ ]:
variable = "A"
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_da = (
    restacked_ief(
        stats_ds,
        ref=crs_1_stats_norm,
        common_dim="total_excitonic_thickness",
        # wavelength=(None, None),
        lorentz_line=-7.5,
        window_radius=4,
        integrate_angle=integrate_angle,
    )[f"{variable}_mean"]
    .sel(delta=slice(0, 1))
    .squeeze()
)

the_plot = plot_optimum_over_dim(
    da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
)[0].opts(opts.QuadMesh(cmap="inferno", clim=(None, None)), opts.Points(color=red))

the_plot

In [ ]:
variable = "A"
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_da = (
    restacked_ief(
        stats_ds,
        ref=crs_1_stats_norm,
        common_dim="total_excitonic_thickness",
        # wavelength=(None, None),
        lorentz_line=0,
        window_radius=2,
        integrate_angle=integrate_angle,
    )[f"{variable}_mean"]
    .sel(delta=slice(0, 1))
    .squeeze()
)

the_plot = plot_optimum_over_dim(
    da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
)[0].opts(opts.QuadMesh(cmap="inferno", clim=(1, None)), opts.Points(color=red))

the_plot

In [ ]:
variable = "R"
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_da = (
    cross_section(
        restacked_stats_ds,
        wavelength=(None, None),
        # lorentz_line=0,
        # window_radius=1,
        integrate_angle=integrate_angle,
    )[f"{variable}_mean"]
    .sel(delta=slice(0, 1))
    .squeeze()
)

the_plot = plot_optimum_over_dim(
    da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
)[0].opts(opts.QuadMesh(cmap="viridis", clim=(0, None)), opts.Points(color=red))

the_plot

In [ ]:
variable = "A"
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_da = (
    cross_section(
        restacked_stats_ds,
        wavelength=(None, None),
        # lorentz_line=0,
        # window_radius=1,
        integrate_angle=integrate_angle,
    )[f"{variable}_mean"]
    .sel(delta=slice(0, 1))
    .squeeze()
)

the_plot = plot_optimum_over_dim(
    da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
)[0].opts(opts.QuadMesh(cmap="inferno", clim=(0, None)), opts.Points(color=red))

In [ ]:
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"opt_{variable}tot_max_DLOPC_D_N_th{th}_elt{elt}", fmt="png", toolbar=False)

#### Loops

In [ ]:
variable = "A"
theta = 0
elt = excitonic_layer_thickness = 50

for integrate_angle in [None, 45, 86]:
    if integrate_angle:
        th = f"{theta}-{integrate_angle}"
    else:
        th = theta
    temp_da = (
        restacked_ief(
            stats_ds,
            ref=crs_1_stats_norm,
            common_dim="total_excitonic_thickness",
            wavelength=(None, None),
            # lorentz_line=0,
            # window_radius=1,
            integrate_angle=integrate_angle,
        )[f"{variable}_mean"]
        .sel(delta=slice(0, 1))
        .squeeze()
    )

    the_plot = plot_optimum_over_dim(
        da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
    )[0].opts(opts.QuadMesh(cmap="inferno", clim=(None, None)), opts.Points(color=red))

    # display(the_plot)

    hv.save(
        the_plot,
        filename=fig_path / f"opt_{variable + 'ef'}tot_max_DLOPC_D_N_th{th}_elt{elt}",
        fmt="png",
        toolbar=False,
    )

In [ ]:
variable = "R"
theta = 0
elt = excitonic_layer_thickness = 50

for integrate_angle in [None, 45, 86]:
    if integrate_angle:
        th = f"{theta}-{integrate_angle}"
    else:
        th = theta
    temp_da = (
        cross_section(
            restacked_stats_ds,
            wavelength=(None, None),
            # lorentz_line=0,
            # window_radius=1,
            integrate_angle=integrate_angle,
        )[f"{variable}_mean"]
        .sel(delta=slice(0, 1))
        .squeeze()
    )

    the_plot = plot_optimum_over_dim(
        da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
    )[0].opts(opts.QuadMesh(cmap="viridis", clim=(0, None)), opts.Points(color=red))
    # display(the_plot)

    hv.save(
        the_plot,
        filename=fig_path / f"opt_{variable}tot_max_DLOPC_D_N_th{th}_elt{elt}",
        fmt="png",
        toolbar=False,
    )

### Unweighted integrated absorptance enhancement

My theory for the discrepancy with results in PCSCs (where disorder can increase absorptance enhancement) is that my absorption is narrow-band, so spectral selectivity is important. If I just integrate the spectral AEF:

In [ ]:
variable = "A"
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_da = (
    cross_section(
        restacked_norm_1_stats(stats_ds),
        wavelength=(None, None),
        # lorentz_line=0,
        # window_radius=1,
        integrate_angle=integrate_angle,
    )[f"{variable}_mean"]
    .sel(delta=slice(0, 1))
    .squeeze()
)

the_plot = plot_optimum_over_dim(
    da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
)[0].opts(opts.QuadMesh(cmap="inferno", clim=(None, None)), opts.Points(color=red))

In [ ]:
the_plot.opts(opts.Points(color=yellow))

In [ ]:
hv.save(the_plot, filename=fig_path/f"opt_{variable + 'ef'}_unweighted_tot_max_DLOPC_D_N_th{th}_elt{elt}", fmt="png", toolbar=False)

In [ ]:
variable = "A"
theta = 0
integrate_angle = None
elt = excitonic_layer_thickness = 50
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_da = (
    cross_section(
        restacked_norm_1_stats(stats_ds),
        # wavelength=(None, None),
        lorentz_line=-7,
        window_radius=3,
        integrate_angle=integrate_angle,
    )[f"{variable}_mean"]
    .sel(delta=slice(0, 1))
    .squeeze()
)

the_plot = plot_optimum_over_dim(
    da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
)[0].opts(opts.QuadMesh(cmap="inferno", clim=(None, None)), opts.Points(color=red))

the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"opt_{variable + 'ef'}_unweighted_tot_max_DLOPC_L-7-3_D_N_th{th}_elt{elt}_", fmt="png", toolbar=False)

#### P=200, N=5

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 5
P = period = 200
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]


temp_ds = (
    restacked_norm_1_stats(stats_ds)
    .sel(
        disorder_correlation=disorder_correlation,
        num_periods=num_periods,
        period=period,
        delta=deltas,
    )
    .squeeze()
)

In [ ]:
the_plot = cross_section(
    dataset=temp_ds,
    variable=f"{variable}_mean",
    wavelength=(480, 881),
    theta=None,
).hvplot(
    kind="line",
    x="theta",
    by="delta",
)
the_plot = (the_plot * hv.HLine(1)).opts(
    opts.Curve(
        ylabel=f"γ{variable}",
        color=default_color_cycle,
        line_dash=default_dash_cycle,
    ),
    opts.HLine(line_dash="dotted", color="black"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_unweighted_tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(1.04, 1.12)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_unweighted_tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

#### P=200, N=20

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 200
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]


temp_ds = (
    restacked_norm_1_stats(stats_ds)
    .sel(
        disorder_correlation=disorder_correlation,
        num_periods=num_periods,
        period=period,
        delta=deltas,
    )
    .squeeze()
)

In [ ]:
the_plot = cross_section(
    dataset=temp_ds,
    variable=f"{variable}_mean",
    wavelength=(480, 881),
    theta=None,
).hvplot(
    kind="line",
    x="theta",
    by="delta",
)
the_plot = (the_plot * hv.HLine(1)).opts(
    opts.Curve(
        ylabel=f"γ{variable}",
        color=default_color_cycle,
        line_dash=default_dash_cycle,
    ),
    opts.HLine(line_dash="dotted", color="black"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_unweighted_tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(1.04, 1.12)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_unweighted_tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

#### P=150

In [ ]:
variable = "A"
disorder_correlation = "correlated"
N = num_periods = 20
P = period = 150
deltas = [0, 0.1, 0.3, 0.5, 1]
theta = None
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]


temp_ds = (
    restacked_norm_1_stats(stats_ds)
    .sel(
        disorder_correlation=disorder_correlation,
        num_periods=num_periods,
        period=period,
        delta=deltas,
    )
    .squeeze()
)

In [ ]:
the_plot = cross_section(
    dataset=temp_ds,
    variable=f"{variable}_mean",
    wavelength=(480, 881),
    theta=None,
).hvplot(
    kind="line",
    x="theta",
    by="delta",
)
the_plot = (the_plot * hv.HLine(1)).opts(
    opts.Curve(
        ylabel=f"γ{variable}",
        color=default_color_cycle,
        line_dash=default_dash_cycle,
    ),
    opts.HLine(line_dash="dotted", color="black"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_unweighted_tot_DLOPC_delta_comparison_th_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

In [ ]:
the_plot.opts(opts.Curve(xlim=(0, 45), ylim=(1.04, 1.12)))

In [ ]:
hv.save(the_plot, filename=fig_path/f"{variable+'ef'}_unweighted_tot_DLOPC_delta_comparison_th0-45_P{P}_elt{elt}_N{N}_D{disorder_code}", fmt="png", toolbar=False)

### Max singles 2D

In [ ]:
variable = "A"
theta = 0
elt = excitonic_layer_thickness = 50

for integrate_angle in [None, 45, 86]:
    if integrate_angle:
        th = f"{theta}-{integrate_angle}"
    else:
        th = theta
    temp_da = (
        restacked_ief(
            singles_ds,
            ref=crs_1,
            common_dim="total_excitonic_thickness",
            wavelength=(None, None),
            # lorentz_line=0,
            # window_radius=1,
            integrate_angle=integrate_angle,
        )[variable]
        .sel(delta=slice(0, 1))
        .squeeze()
        .max("run")
        .compute()
    )

    the_plot = plot_optimum_over_dim(
        da=temp_da, x="delta", y="num_periods", dim=["period", "disorder_correlation"]
    )[0].opts(opts.QuadMesh(cmap="inferno", clim=(None, None)), opts.Points(color=red))

    display(the_plot)

    hv.save(
        the_plot,
        filename=fig_path / f"opt_{variable + 'ef'}tot_max_DLOPC_D_N_th{th}_elt{elt}_rmax",
        fmt="png",
        toolbar=False,
    )

##### Max

P=100

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 100
theta = 0
variable = "A"
disorder_correlation = "correlated"
runs = slice(None, None)
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_1 = compare_integrated_singles(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    run=runs,
    wavelength=(None, None),
    theta=theta,
    kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
    maximise="run",
)
the_plot_1.opts(opts.HLine(apply_ranges=False))

In [ ]:
hv.save(
    the_plot_1,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}_rmax",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 100
theta = 0
variable = "A"
disorder_correlation = "uncorrelated"
runs = slice(None, None)
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_1 = compare_integrated_singles(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    run=runs,
    wavelength=(None, None),
    theta=theta,
    kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
    maximise="run",
)
the_plot_1.opts(opts.HLine(apply_ranges=False))

In [ ]:
hv.save(
    the_plot_1,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}_rmax",
    fmt="png",
    toolbar=False,
)

P=200

In [ ]:
variable = "A"
theta = 0
deltas = [0, 0.1, 0.3, 0.5, 1]
disorder_correlation = "correlated"
runs = slice(None, None)
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]

for period in [150, 200, 250, 300]:
    for integrate_angle in [None, 45, 86]:
        if integrate_angle:
            th = f"{theta}-{integrate_angle}"
        else:
            th = theta
        P = period

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            periodic_optimum = (
                restacked_ief(
                    periodic_ds,
                    ref=crs_1,
                    common_dim="total_excitonic_thickness",
                    wavelength=(None, None),
                    theta=theta,
                    integrate_angle=integrate_angle,
                )["A"]
                .sel(excitonic_layer_thickness=excitonic_layer_thickness)
                .sel(num_periods=singles_ds.num_periods)
                .squeeze()
                .max("period")
                .hvplot(kind="scatter", x="num_periods", y="A")
            )
            # periodic_optimum_2 = (
            #     restacked_ief(
            #         periodic_ds,
            #         ref=crs_1,
            #         common_dim="total_excitonic_thickness",
            #         wavelength=(None, None),
            #         theta=theta,
            #         integrate_angle=integrate_angle,
            #     )["A"]
            #     .drop_sel({"excitonic_layer_thickness": 0})
            #     .sel(num_periods=singles_ds.num_periods)
            #     .squeeze()
            #     .max(["period", "excitonic_layer_thickness"])
            #     .hvplot(kind="scatter", x="num_periods", y="A")
            # )

        periodic_optimum.opts(
            opts.Scatter(color="black", marker="square", fill_alpha=0, size=10)
        )
        # periodic_optimum_2.opts(opts.Scatter(color="black", marker="circle"))

        the_plot = compare_integrated_singles(
            variable=variable,
            disorder_correlation=disorder_correlation,
            period=period,
            delta=deltas,
            run=runs,
            wavelength=(None, None),
            theta=theta,
            integrate_angle=integrate_angle,
            kind="line",
            x="num_periods",
            by="delta",
            mode="ef",
            maximise="run",
        )
        the_plot.opts(opts.HLine(apply_ranges=False))

        mega_plot = (
            periodic_optimum
            # *periodic_optimum_2
            * the_plot
        )
        display(mega_plot)

        hv.save(
            mega_plot,
            filename=fig_path
            / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{th}_D{disorder_code}_rmax_LOPC_opt",
            fmt="png",
            toolbar=False,
        )

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    periodic_optimum = (
        restacked_ief(
            periodic_ds,
            ref=crs_1,
            common_dim="total_excitonic_thickness",
            wavelength=(None, None),
            theta=theta,
        )["A"]
        .sel(excitonic_layer_thickness=excitonic_layer_thickness)
        .sel(num_periods=singles_ds.num_periods)
        .squeeze()
        .max("period")
        .hvplot(kind="scatter", x="num_periods", y="A")
    )

In [ ]:
periodic_optimum.opts(opts.Scatter(color="black", marker="square", fill_alpha=0, size=10))

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    periodic_optimum_2 = (
        restacked_ief(
            periodic_ds,
            ref=crs_1,
            common_dim="total_excitonic_thickness",
            wavelength=(None, None),
            theta=theta,
        )["A"]
        .drop_sel({"excitonic_layer_thickness": 0})
        .sel(num_periods=singles_ds.num_periods)
        .squeeze()
        .max(["period", "excitonic_layer_thickness"])
        .hvplot(kind="scatter", x="num_periods", y="A")
    )

In [ ]:
periodic_optimum_2.opts(opts.Scatter(color="black", marker="circle"))

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 200
theta = 0
variable = "A"
disorder_correlation = "correlated"
runs = slice(None, None)
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_2 = compare_integrated_singles(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    run=runs,
    wavelength=(None, None),
    theta=theta,
    kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
    maximise="run",
)
the_plot_2.opts(opts.HLine(apply_ranges=False))

In [ ]:
mega_plot = (periodic_optimum*periodic_optimum_2*the_plot_2)
mega_plot

In [ ]:
hv.save(
    the_plot_2,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}_rmax",
    fmt="png",
    toolbar=False,
)

In [ ]:
hv.save(
    mega_plot,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}_rmax_LOPC_opt",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 200
theta = 0
variable = "A"
disorder_correlation = "uncorrelated"
runs = slice(None, None)
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_2 = compare_integrated_singles(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    run=runs,
    wavelength=(None, None),
    theta=theta,
    kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
    maximise="run",
)
the_plot_2.opts(opts.HLine(apply_ranges=False))

In [ ]:
hv.save(
    the_plot_2,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}_rmax",
    fmt="png",
    toolbar=False,
)

P=300

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 300
theta = 0
variable = "A"
disorder_correlation = "correlated"
runs = slice(None, None)
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_3 = compare_integrated_singles(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    run=runs,
    wavelength=(None, None),
    theta=theta,
    kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
    maximise="run",
)
the_plot_3.opts(opts.HLine(apply_ranges=False))

In [ ]:
hv.save(
    the_plot_3,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}_rmax",
    fmt="png",
    toolbar=False,
)

In [ ]:
deltas = [0, 0.1, 0.3, 0.5, 1]
P = period = 300
theta = 0
variable = "A"
disorder_correlation = "uncorrelated"
runs = slice(None, None)
elt = excitonic_layer_thickness = 50
disorder_code = disorder_correlation[0]
# D = "-".join([str(deltas[0]), str(deltas[-1])])

the_plot_3 = compare_integrated_singles(
    variable=variable,
    disorder_correlation=disorder_correlation,
    period=period,
    delta=deltas,
    run=runs,
    wavelength=(None, None),
    theta=theta,
    kind="line",
    x="num_periods",
    by="delta",
    mode="ef",
    maximise="run",
)
the_plot_3.opts(opts.HLine(apply_ranges=False))

In [ ]:
hv.save(
    the_plot_3,
    filename=fig_path
    / f"{variable+'ef'}tot_DLOPC_delta_comparison_N_P{P}_elt{elt}_th{theta}_D{disorder_code}_rmax",
    fmt="png",
    toolbar=False,
)

## Does disorder reduce wavelength sensitivity?

Well, yes, but at the cost of reducing the magnitude of effects.

## Does disorder reduce angle sensitivity?

Yes, it tends to smooth out the reflectance and absorptance curves.

## Does disorder qualitatively change the optimality results?

In [ ]:
wrapped_2D_plot(variable="R", dataset=restacked_ads, lorentz_line=0, window_radius=5, period_start=0, period_stop=400, extra_plots=[], dim="period")

In [ ]:
wrapped_2D_plot(variable="R", dataset=restacked_ads, lorentz_line=0, window_radius=5, period_start=0, period_stop=400, extra_plots=[], dim=["period", "delta"])

Delta=0.15 (correlated) is enough to shift the position of the optimum R_per_osc at L0 significantly...

In [ ]:
# variable = "R_per_oscillator"
# theta = 0
# integrate_angle = None
# delta = 0.15
# disorder_correlation = "correlated"
# lorentz_line = 0
# window_radius = 0


# disorder_code = disorder_correlation[0]
# D = str(delta).replace(".", "")
# if "per_oscillator" in variable:
#     var_code = variable[0] + "po"
# else:
#     var_code = variable
# if integrate_angle:
#     th = f"{theta}-{integrate_angle}"
# else:
#     th = theta
# match variable[0]:
#     case "R":
#         cmap = "viridis"
#     case "T":
#         cmap = "cividis"
#     case "A":
#         cmap = "inferno"

# temp_da = (
#     cross_section(
#         restacked_ads,
#         # wavelength=(None, None),
#         lorentz_line=lorentz_line,
#         window_radius=window_radius,
#         integrate_angle=integrate_angle,
#     )[f"{variable}"]
#     .sel(delta=delta)
#     .sel(disorder_correlation=disorder_correlation)
#     .squeeze()
# )

# the_plot = plot_optimum_over_dim(
#     da=temp_da, x="excitonic_layer_thickness", y="num_periods", dim=["period"]
# )[0].opts(opts.QuadMesh(cmap=cmap, clim=(0, None)), opts.Points(color=red))

In [ ]:
tick_formatter = PrintfTickFormatter(format='%.0e')
# plot_opts = dict(colorbar=True, colorbar_opts={'formatter': formatter})
# hv.HeatMap([(0,0,0), (1,1,1)]).opts(plot=plot_opts)

In [ ]:
"%.0e" % (5.000e-30)

In [ ]:
variable = "R_per_oscillator"
lorentz_line = 0
window_radius = 0
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_ads
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(
    opts.Overlay(title=""),
    opts.QuadMesh(xlabel="d_e (nm)", colorbar_opts={"formatter": tick_formatter}),
)

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

Further from LO resonance?

In [ ]:
variable = "R_per_oscillator"
lorentz_line = 5
window_radius = 3
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_ads
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(opts.Overlay(title=""), opts.QuadMesh(xlabel="d_e (nm)"))

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

Integrating over theta?

In [ ]:
variable = "R_per_oscillator"
lorentz_line = 0
window_radius = 0
theta = 0
integrate_angle = 45

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_ads
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(opts.Overlay(title=""), opts.QuadMesh(xlabel="d_e (nm)"))

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

... and also R.

R-0-0-0-0: 0.47268407 (D=0.01c) vs 0.46824249 (D=0)

R-0-8-0-0: 0.23540381 (D=0.14c) vs 0.20471805 (D=0)

R-0-8-0-45: 0.17701249 (D=0.13c) vs 0.16186628 (D=0)

In [ ]:
variable = "R"
lorentz_line = 0
window_radius = 0
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_ds = cross_section(
    restacked_ads,
    variable=variable,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
)

In [ ]:
find_optimum_coords(temp_ds)

In [ ]:
find_optimum_coords(temp_ds.sel(delta=0, disorder_correlation="correlated"))

In [ ]:
P = 250
elt = 40
N = 30
the_plot = temp_ds.sel(period=P, excitonic_layer_thickness=elt, num_periods=N).squeeze().hvplot(
    x="delta", by="disorder_correlation"
).opts(opts.Curve(ylabel=var_code), opts.NdOverlay(title="", legend_position="bottom_left", legend_opts={"title": None}))
the_plot

In [ ]:
print(f"correlated: {(1-0.227/0.468)*100:.2f}%\nuncorrelated: {(1-0.347/0.468)*100:.2f}%")

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code}_DLOPC_1D_D_L0_P{P}_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
lorentz_line = 0
window_radius = 8
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if window_radius:
    L = f"{lorentz_line}-{window_radius}"
else:
    L = lorentz_line
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_ds = cross_section(
    restacked_ads,
    variable=variable,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
)

In [ ]:
find_optimum_coords(temp_ds)

In [ ]:
find_optimum_coords(temp_ds.sel(delta=0, disorder_correlation="correlated"))

In [ ]:
P = 290
elt = 80
N = 30
the_plot = temp_ds.sel(period=P, excitonic_layer_thickness=elt, num_periods=N).squeeze().hvplot(
    x="delta", by="disorder_correlation"
).opts(opts.Curve(ylabel=var_code), opts.NdOverlay(title="", legend_position="top_left", legend_opts={"title": None}))
the_plot

In [ ]:
print(f"correlated: {(1-0.235/0.205)*100:.2f}%\nuncorrelated: {(1-0.207/0.205)*100:.2f}%")

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code}_DLOPC_1D_D_L{L}_P{P}_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
lorentz_line = 0
window_radius = 8
theta = 0
integrate_angle = 45

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if window_radius:
    L = f"{lorentz_line}-{window_radius}"
else:
    L = lorentz_line
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_ds = cross_section(
    restacked_ads,
    variable=variable,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
)

In [ ]:
find_optimum_coords(temp_ds)

In [ ]:
find_optimum_coords(temp_ds.sel(delta=0, disorder_correlation="correlated"))

In [ ]:
P = 300
elt = 80
N = 30
the_plot = temp_ds.sel(period=P, excitonic_layer_thickness=elt, num_periods=N).squeeze().hvplot(
    x="delta", by="disorder_correlation"
).opts(opts.Curve(ylabel=var_code), opts.NdOverlay(title="", legend_position="top_left", legend_opts={"title": None}))
the_plot

In [ ]:
print(f"correlated: {(0.177/0.162-1)*100:.2f}%\nuncorrelated: {(0.160/0.162-1)*100:.2f}%")

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code}_DLOPC_1D_D_L{L}_P{P}_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "R"
lorentz_line = 0
window_radius = 0
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_ads
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(opts.Overlay(title=""), opts.QuadMesh(xlabel="d_e (nm)"))

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

Integrating over lambda?

In [ ]:
variable = "R"
lorentz_line = 0
window_radius = 3
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_ads
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(opts.Overlay(title=""), opts.QuadMesh(xlabel="d_e (nm)"))

In [ ]:
# find_optimum_coords(temp_ds)

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

Integrating over theta?

In [ ]:
variable = "R"
lorentz_line = 0
window_radius = 0
theta = 0
integrate_angle = 45

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_ads
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(opts.Overlay(title=""), opts.QuadMesh(xlabel="d_e (nm)"))

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

Integrating over theta and lambda?

In [ ]:
variable = "R"
lorentz_line = 0
window_radius = 8
theta = 0
integrate_angle = 45

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_ads
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(opts.Overlay(title=""), opts.QuadMesh(xlabel="d_e (nm)"))

In [ ]:
# find_optimum_coords(temp_ds)

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

With absorptance, it's hard to see any patterns because it saturates. Look at Aef instead.

In [ ]:
variable = "A"
lorentz_line = 0
window_radius = 0
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_ads
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(opts.Overlay(title=""), opts.QuadMesh(xlabel="d_e (nm)"))

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

Disorder can also shift the position of the optimal parameters for absorptance enhancement.

In [ ]:
variable = "A"
lorentz_line = 0
window_radius = 0
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta

temp_ds = cross_section(
    restacked_norm_1(aggregated_ds),
    variable=variable,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
)

In [ ]:
find_optimum_coords(temp_ds)

In [ ]:
find_optimum_coords(temp_ds.sel(delta=0, disorder_correlation="correlated"))

In [ ]:
P = 230
elt = 10
N = 10
the_plot = temp_ds.sel(period=P, excitonic_layer_thickness=elt, num_periods=N).squeeze().hvplot(
    x="delta", by="disorder_correlation"
).opts(opts.Curve(ylabel="γA"), opts.NdOverlay(title="", legend_position="bottom_left", legend_opts={"title": None}))
the_plot

In [ ]:
print(f"correlated: {(16.5/20.5-1)*100:.2f}%\nuncorrelated: {(19.1/20.5-1)*100:.2f}%")

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code + 'ef'}_DLOPC_1D_D_L0_P{P}_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
lorentz_line = 0
window_radius = 8
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
if window_radius:
    L = f"{lorentz_line}-{window_radius}"
else:
    L= lorentz_line

temp_ds = restacked_ief(
    aggregated_ds,
    ref=crs_1,
    common_dim="total_excitonic_thickness",
    variable=variable,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
)

In [ ]:
find_optimum_coords(temp_ds)

In [ ]:
find_optimum_coords(temp_ds.sel(delta=0, disorder_correlation="correlated"))

In [ ]:
P = 210
elt = 20
N = 5
the_plot = temp_ds.sel(period=P, excitonic_layer_thickness=elt, num_periods=N).squeeze().hvplot(
    x="delta", by="disorder_correlation"
).opts(opts.Curve(ylabel="γA_tot"), opts.NdOverlay(title="", legend_position="bottom_left", legend_opts={"title": None}))
the_plot

In [ ]:
print(f"correlated: {(16.3/17.1-1)*100:.2f}%\nuncorrelated: {(16.1/17.1-1)*100:.2f}%")

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code + 'ef'}_DLOPC_1D_D_L{L}_P{P}_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
lorentz_line = 0
window_radius = 8
theta = 0
integrate_angle = 45

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
if window_radius:
    L = f"{lorentz_line}-{window_radius}"
else:
    L= lorentz_line

temp_ds = restacked_ief(
    aggregated_ds,
    ref=crs_1,
    common_dim="total_excitonic_thickness",
    variable=variable,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
)

In [ ]:
find_optimum_coords(temp_ds)

In [ ]:
find_optimum_coords(temp_ds.sel(delta=0, disorder_correlation="correlated"))

In [ ]:
P = 210
elt = 30
N = 5
the_plot = temp_ds.sel(period=P, excitonic_layer_thickness=elt, num_periods=N).squeeze().hvplot(
    x="delta", by="disorder_correlation"
).opts(opts.Curve(ylabel="γA_tot"), opts.NdOverlay(title="", legend_position="bottom_left", legend_opts={"title": None}))
the_plot

In [ ]:
print(f"correlated: {(13.3/14.1-1)*100:.2f}%\nuncorrelated: {(13.4/14.1-1)*100:.2f}%")

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code + 'ef'}_DLOPC_1D_D_L{L}_P{P}_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

#### What if we ignore the absorption and illumination spectra?

Normal incidence

In [ ]:
variable = "A"
lorentz_line = 0
window_radius = 8
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
if window_radius:
    L = f"{lorentz_line}-{window_radius}"
else:
    L= lorentz_line

temp_ds = cross_section(
    restacked_norm_1(aggregated_ds),
    variable=variable,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
)

In [ ]:
find_optimum_coords(temp_ds)

In [ ]:
find_optimum_coords(temp_ds.sel(delta=0, disorder_correlation="correlated"))

Integrating over wavelength without accounting for absorption changes the position of the optima. Let's compare them...

In [ ]:
# original optimum
P = 210
elt = 20
N = 5
the_plot = temp_ds.sel(period=P, excitonic_layer_thickness=elt, num_periods=N).squeeze().hvplot(
    x="delta", by="disorder_correlation"
).opts(opts.Curve(ylabel="γA_tot"), opts.NdOverlay(title="", legend_position="bottom_left", legend_opts={"title": None}))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code + 'ef'}_unweighted_DLOPC_1D_D_L{L}_P{P}_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
P = 200
elt = 30
N = 5
the_plot = temp_ds.sel(period=P, excitonic_layer_thickness=elt, num_periods=N).squeeze().hvplot(
    x="delta", by="disorder_correlation"
).opts(opts.Curve(ylabel="γA_tot"), opts.NdOverlay(title="", legend_position="bottom_left", legend_opts={"title": None}))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code + 'ef'}_unweighted_DLOPC_1D_D_L{L}_P{P}_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

Integrated to 45 degrees

In [ ]:
variable = "A"
lorentz_line = 0
window_radius = 8
theta = 0
integrate_angle = 45

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
if window_radius:
    L = f"{lorentz_line}-{window_radius}"
else:
    L= lorentz_line

temp_ds = cross_section(
    restacked_norm_1(aggregated_ds),
    variable=variable,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
)

In [ ]:
find_optimum_coords(temp_ds)

In [ ]:
P = 210
elt = 30
N = 5
the_plot = temp_ds.sel(period=P, excitonic_layer_thickness=elt, num_periods=N).squeeze().hvplot(
    x="delta", by="disorder_correlation"
).opts(opts.Curve(ylabel="γA_tot"), opts.NdOverlay(title="", legend_position="bottom_left", legend_opts={"title": None}))
the_plot

In [ ]:
hv.save(
    the_plot,
    filename=fig_path
    / f"{var_code + 'ef'}_unweighted_DLOPC_1D_D_L{L}_P{P}_elt{elt}_N{N}_th{th}",
    fmt="png",
    toolbar=False,
)

In [ ]:
variable = "A"
lorentz_line = 5
window_radius = 0
theta = 0
integrate_angle = 0

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_norm_1(aggregated_ds)
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(opts.Overlay(title=""), opts.QuadMesh(xlabel="d_e (nm)"))

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code + 'ef'}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code + 'ef'}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code + 'ef'}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

Integrating over theta?

In [ ]:
variable = "A"
lorentz_line = 5
window_radius = 0
theta = 0
integrate_angle = 45

if "per_oscillator" in variable:
    var_code = variable[0] + "po"
else:
    var_code = variable
if integrate_angle:
    th = f"{theta}-{integrate_angle}"
else:
    th = theta
match variable[0]:
    case "R":
        cmap = "viridis"
    case "T":
        cmap = "cividis"
    case "A":
        cmap = "inferno"

temp_ds = restacked_norm_1(aggregated_ds)
dmap = wrapped_2D_plot(
    variable=variable,
    dataset=temp_ds,
    lorentz_line=lorentz_line,
    window_radius=window_radius,
    theta=theta,
    integrate_angle=integrate_angle,
    period_start=0,
    period_stop=400,
    extra_plots=[],
    dim="period",
    cmap=cmap,
).opts(opts.Overlay(title=""), opts.QuadMesh(xlabel="d_e (nm)"))

In [ ]:
dmap

In [ ]:
delta = 0
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code + 'ef'}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "correlated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code + 'ef'}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

In [ ]:
delta = 0.15
disorder_correlation = "uncorrelated"
disorder_code = disorder_correlation[0]
D = str(delta).replace(".", "")

the_plot = dmap.select(delta=delta, disorder_correlation=disorder_correlation)
display(the_plot)
hv.save(
    the_plot,
    filename=fig_path
    / f"opt_{var_code + 'ef'}_max_DLOPC_elt_N_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_D{D}{disorder_code}",
    fmt="png",
    toolbar=False,
)

## Does disorder make a difference on the level found in iridoplasts?

In [ ]:
R_CRS = integrate_da(
    crs_1["R"].sel(total_excitonic_thickness=400),
    dim="wavelength",
    normalisation=1,
).hvplot(x="theta", line_color="black", line_dash="dashed")

In [ ]:
the_plot = (
    integrate_da(
        restacked_ads["R"].sel(
            period=300,
            num_periods=20,
            excitonic_layer_thickness=20,
        ).sel(delta=slice(None,None,3)),
        dim="wavelength",
        normalisation=1,
    )
    .squeeze()
    .hvplot(x="theta", by="delta", groupby=["disorder_correlation"])
)

In [ ]:
the_plot.opts(opts.Curve())*R_CRS

In [ ]:
the_plot = (
    integrate_da(
        restack_plt_to_period(stats_ds)["R_mean"].sel(
            period=300,
            num_periods=5,
        ),
        dim="wavelength",
        normalisation=1,
    )
    .squeeze()
    .hvplot(
        x="theta",
        by="delta",
        groupby=[
            "disorder_correlation",
        ],
    )
)

In [ ]:
the_plot.opts(opts.Curve())#*R_CRS  # wrong thickness

Seems that disorder does smooth out the curves, and also brings the shape back to the shape of the CRS reflectance.

In [ ]:
the_plot = (
    integrate_da(
        restack_plt_to_period(stats_ds)["R_mean"], dim="wavelength", normalisation=1
    )
    .sel(period=300, num_periods=5, theta=0)
    .squeeze()
    .hvplot(x="delta", by="disorder_correlation")
)

In [ ]:
the_plot.opts()

In [ ]:
the_plot = (
    integrate_da(
        restack_plt_to_period(stats_ds)["R_mean"], dim=["wavelength", "theta"], normalisation=1
    )
    .sel(period=300, num_periods=5)
    .squeeze()
    .hvplot(x="delta", by="disorder_correlation")
)

In [ ]:
the_plot.opts()

In [ ]:
the_plot = (
    integrate_da(
        restack_plt_to_period(stats_ds)["R_mean"].sel(theta=slice(0, 45)), dim=["wavelength", "theta"], normalisation=1
    )
    .sel(period=300, num_periods=5)
    .squeeze()
    .hvplot(x="delta", by="disorder_correlation")
)

In [ ]:
the_plot.opts()

In [ ]:
the_plot = (
    integrate_da(
        restack_plt_to_period(stats_ds)["R_mean"].sel(theta=slice(0, 45)),
        dim=["wavelength", "theta"],
        normalisation=1,
    )
    - integrate_da(
        restack_plt_to_period(stats_ds)["R_mean"], dim="wavelength", normalisation=1
    ).sel(theta=0)
).sel(period=250, num_periods=10).squeeze().hvplot(x="delta", by="disorder_correlation")

In [ ]:
the_plot.opts()

In [ ]:
the_plot = (
    restacked_ief(
        stats_ds,
        ref=crs_1_stats_norm,
        common_dim="total_excitonic_thickness",
        variable="A_mean",
        wavelength=(480, 881),
        theta=None
    )
    .sel(
        period=300,
        num_periods=30,
    )
    .squeeze()
    .hvplot(
        x="theta",
        by="delta",
        groupby=[
            "disorder_correlation",
        ],
    )
)

In [ ]:
the_plot.opts(opts.Curve())*hv.HLine(1).opts(line_dash="dotted")

Seems that disorder does smooth out the curves, and also brings the shape back to the shape of the CRS reflectance.

What is the highest reflectance possible in the unperturbed LOPC at N=5, wavelength=700?

In [ ]:
wrapped_2D_plot(variable="R", dataset=restacked_pds, lorentz_line=1, period_start=0, period_stop=400, extra_plots=[], cmap="viridis", dim="period")

21%!

What about N=20? Only 51.6% at d_e=50! But have a look at `R_DLOPC_meancurve_wl_P250_elt50_N20_th0_D01c.png` and `R_DLOPC_runcurve_wl_P250_elt50_N20_th0_D01c.png`. Some of the runs go as high as 60% reflectance there! What's going on?

In [ ]:
wrapped_2D_plot(variable="R", dataset=restacked_pds, lorentz_line=1.2, period_start=0, period_stop=400, extra_plots=[], cmap="viridis", dim="period")

What's going on is that the maximum R really does skyrocket over a couple of nm. At 703 nm, max R at d_e=50, N=20 is 61.2%. This is a great illustration of how much a difference the dispersion can make.

# Cleanup

This depends on if the disorder is correlated or uncorrelated! I think in iridoplasts it is probably uncorrelated.

In [ ]:
# hv.archive.export()

In [ ]:
# hv.archive.last_export_status()